In [2]:
# standard imports
import urllib.request
import logging
import ssl
from urllib.request import quote,unquote
import requests
import joblib
import os
import re
import warnings
import json
import time
import argparse
import numpy as np
import openai
import tiktoken
import langchain

import pandas as pd
import concurrent.futures
from langchain.schema import Document
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
import random
# external imports
import faiss
from datetime import datetime
from sentence_transformers import SentenceTransformer
from azureml.core.model import Model
from azureml.core import Workspace
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [3]:
from azureml.core import Workspace
 
ws = Workspace.from_config()
default_keyvault = ws.get_default_keyvault()

In [4]:
from azureml.core import Workspace, Dataset
from azureml.core.model import Model

#ws = Workspace.from_config()
model = Model(ws,name="test-assist-code-search-faiss-index-test")
faiss_model_azure=model.download(exist_ok = True)

In [5]:
print(faiss_model_azure)

merged_faiss_db


In [6]:

# def restructure_document_object(self, matching_docs):
#     filtered_documents = []
#     for row in matching_docs:
#         input_doc_content = row.page_content + \
#             "\n\n" + row.metadata.get('unit_test_cases')
#         document_obj = {"input": row.page_content , "output":row.metadata.get('unit_test_cases')}
#         filtered_documents.append(document_obj)
#     return filtered_documents

In [7]:
# def few_shot_prompting_template(self,examples):
#     example_template = """
#     Input:
#         Task:
#         write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

#         Instructions:
#         - understand the code snippet below and write unit test cases
#         - consider every scenario for unit testing
#         - only write code
#         - no comments and explanations for the written code

#         code snippet:
#         ####

#         {input}

#         ####

#         Example output format:
#         //comments
#         <code>
#         </code>

#     Output:
#         {output}
#     """
#     example_prompt = PromptTemplate(
#         input_variables=["input", "output"], template=example_template
#     )
    
#     prompt = FewShotPromptTemplate(
#         examples=examples,
#         example_prompt=example_prompt,
#         suffix="",
#         input_variables=["input"],
#     )

#     examples_retreived = prompt.format(input="")
    
#     return examples_retreived



In [8]:

# if len(matching_docs) == 0:
#     system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Java script source code as input and you need to generate unit test cases in Jasmine framework as output "
#     value = [{"role": "System", "content": quote(system_message)},
#              {"role": "user", "content": quote(prompt)}]
    


In [9]:
examples = [
    {
        "input": "Who lived longer, Muhammad Ali or Alan Turing?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
    },
    {
        "input": "When was the founder of craigslist born?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
    },
    {
        "input": "Who was the maternal grandfather of George Washington?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
    },
    {
        "input": "Are both the directors of Jaws and Casino Royale from the same country?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
    }
]

In [10]:
# system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Java script source code as input and you need to generate unit test cases in Jasmine framework as output "
# value = [{"role": "System", "content": quote(system_message)},
#          {"role": "System", "content": quote(examples_retreived)},
#          {"role": "user", "content": quote(prompt)}]

In [48]:
class Inference:
    def __init__(self, model):
        self.model_path = model
        self.embeddings = SentenceTransformerEmbeddings(
            model_name="multi-qa-MiniLM-L6-cos-v1")
        self.db = FAISS.load_local(self.model_path, self.embeddings,allow_dangerous_deserialization=True)

        self.api_key_16k = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-35-16k-turbo-api-key")
        self.api_key = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-35-turbo-api-key")
        self.url = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-35-turbo-url")

    def restructure_document_object(self, matching_docs):
        filtered_documents = []
        for row in matching_docs:
            input_doc_content = row.page_content + \
                "\n\n" + row.metadata.get('unit_test_cases')
            input_code = self.code_clean(row.page_content)
            output_code= self.code_clean(row.metadata.get('unit_test_cases'))
            document_obj = {"input": input_code , "output":output_code}
            filtered_documents.append(document_obj)
        return filtered_documents 

    def few_shot_prompting_template(self,examples):
        example_template = '''
        Input:
            Task:
            write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

            Instructions:
            - understand the code snippet below and write unit test cases
            - consider every scenario for unit testing
            - only write code
            - no comments and explanations for the written code

            code snippet:
            ####

            "{input}"

            ####

            Expected Output Format:
            // Test case: Testing....
            <code>
            .
            .

        Output:
            {output}
        '''
        #print(type(examples[0]))
        
        examples_retreived = ""
        for i in examples:
            temp = example_template.format(input=i.get('input'),output=i.get('output')) + "\n\n"
            examples_retreived = examples_retreived + temp
            

        return examples_retreived
    
    def gpt_35_turbo_chat_completion(self,value):
        #self.allowSelfSignedHttps(True)
        #print(value)
        # value = [{"role": "user", "content": quote(prompt)}]
        # value_unquote = [{"role": "user", "content": prompt}]
        turbo_data = {
            'messages': value,
            'temperature': 0,
            'top_p': 0.1
        }
        #print(str(turbo_data))
        # turbo_data_unquote = {
        #     "messages": value_unquote,
        #     'temperature': 0,
        #     'top_p': 0.1
        # }
        # print(str(turbo_data_unquote))
        #print(str(turbo_data))
        turbo_body = str.encode(json.dumps(turbo_data))
        url = self.url
        api_key = self.api_key
        if not api_key:
            raise Exception("A key should be provided to invoke the endpoint")

        headers = {'Content-Type': 'application/json','api-key': api_key}

        #req = urllib.request.Request(url, turbo_body, headers)
        result =""
        prompt_response=""
        tokens_count = 0
        total_cost = 0.0
        model =""
        status =""
        status_code = 0
        error_message=""

        try:
            #req = urllib.request.Request(url, turbo_body, headers)
            response = requests.post(url=url,headers=headers,data=turbo_body)
            status_code = response.status_code
            response_json = response.json()
            #response_json = json.loads(response.text)
            
            #print(response_json)
            prompt_response = response_json['choices'][0]['message']['content']
            print(prompt_response)
            tokens_count = response_json['usage']['total_tokens']
            total_cost = (int(response_json['usage']['prompt_tokens'])/1000)*0.0005 + (int(response_json['usage']['completion_tokens'])/1000)*0.0015
            total_cost = round(total_cost,4)
            model = response_json['model']
            status = "success"

        except requests.exceptions.RequestException as error:  # This is the correct syntax
            
            status_code = error.errno
            status = "failed"
            print(error)
            raise(error)

        return prompt_response,tokens_count,total_cost,model,status,status_code,error_message


    def gpt_35_turbo_16k_chat_completion(self,value):
        #print(value)
        turbo_data = {
            'messages': value,
            'temperature': 0,
            'top_p': 0.1
        }
        #print(str(turbo_data))
        turbo_body = str.encode(json.dumps(turbo_data))
        url = self.url
        api_key = self.api_key_16k
        if not api_key:
            raise Exception("A key should be provided to invoke the endpoint")

        headers = {'Content-Type': 'application/json','api-key': api_key}

        result =""
        prompt_response=""
        tokens_count = 0
        total_cost = 0.0
        model =""
        status =""
        status_code = 0
        error_message=""

        try:
            start=datetime.now()

            #req = urllib.request.Request(url, turbo_body, headers)
            response = requests.post(url=url,headers=headers,data=turbo_body)
            status_code = response.status_code
            #response_json = json.loads(response.text)
            
            response_json = response.json()
            #print(response_json)
            prompt_response = response_json['choices'][0]['message']['content']
            print(prompt_response)
            tokens_count = response_json['usage']['total_tokens']
            total_cost = (int(response_json['usage']['prompt_tokens'])/1000)*0.0005 + (int(response_json['usage']['completion_tokens'])/1000)*0.0015
            total_cost = round(total_cost,4)
            model = response_json['model']
            status = "success"
            end=datetime.now()
            time_taken = round((end - start).total_seconds(),2)
            print(str(time_taken))
            
        except requests.exceptions.RequestException as error:  # This is the correct syntax
            end=datetime.now()
            time_taken = round((end - start).total_seconds(),2)
            print(str(time_taken))
            status_code = error.errno
            status = "failed"
            print(error)
            raise(error)

        return prompt_response,tokens_count,total_cost,model,status,status_code,error_message

    def num_tokens_from_string(self, string: str, encoding_name: str) -> int:
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    def code_clean(self,code):
        code = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', f"{{{'email'}}}", code)
        code = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', f"{{{'phone'}}}", code)
        code = re.sub(r'\b\d{3}[-]?\d{2}[-]?\d{4}\b', f"{{{'ssn '}}}", code)
        code = re.sub(r'\b\d{3}[ ]?\d{2}[ ]?\d{4}\b', f"{{{'ssn'}}}", code)
        code = re.sub(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', f"{{{'ip address'}}}", code)
        code = re.sub(r'//.*', '', code)
        code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        return code

    def parallel_api_call(self,final_list):
        # Define the number of threads (adjust as needed)
        num_threads = len(final_list)
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
            # Submit tasks to the executor
            futures = [executor.submit(self.gpt35turbo_langchain_generate, item) for item in final_list]
        
            # Wait for all tasks to complete
            concurrent.futures.wait(futures)
        
            # Get results
            results = [future.result() for future in futures]
            print("results: \n" + str(results))
            response = '\n'.join([x[0] for x in results])
            #response = '\n'.join([tuple(xi for xi in x[0] if xi is not None) for x in results])
            time_taken = max([x[1] for x in results])
            token_count = sum([x[2] for x in results])
            total_cost = sum([x[3] for x in results])
            model_used = ",".join(list(set([x[4] for x in results])))
            error = ",".join(list(set([x[5] for x in results])))
            error_type = ",".join(list(set([x[6] for x in results])))
            azure_time_taken = max([x[7] for x in results])
            #self_calculated_token_size = sum([x[8] for x in results])
        return response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken    
    
    def split_func(self,code):
        #code = self.code_clean(code)
        classes=code.split('\n')
        classlist=[]
        scriptlist=[]
        cla=''
        for i in range(len(classes)):
            #dividing classes
            if len(re.findall(r'Class\.create\(\)',classes[i]))>0:
                classlist.append(cla)
                cla=''
                cla+=classes[i]+'\n'
            else:
                cla+=classes[i]+'\n'
            if(i==len(classes)-1):
                classlist.append(cla)
                cla=''
        classlist.pop(0)
        final_classlist=[]
        for i in range(len(classlist)):   
            #counting number of functions in every class
            fu=re.findall(r':\s*function\s*\(',classlist[i])
            if(len(fu)>1):
                lines=classlist[i].split('\n')
                #getting all the script present before starting first function
                prefix=''
                ind=2
                for j in range(len(lines)):
                    if len(re.findall(r':\s*function\s*\(',lines[j]))>0:
                        ind=j
                        break
                    else:
                        prefix+='\n'+lines[j]
                    #not considering global variables
                temp_prefix=prefix.split('\n')
                prefix2=temp_prefix[1]+'\n'+temp_prefix[-1]
                class_name_line=temp_prefix[1].strip()
                suffix='    type: '+class_name_line.split(' ')[1]+'\n};'
                #exluding class defining code lines
                nlines=lines[ind:-2]
                func=[]
                temp=''
                br=0
                comment=False
                for j in range(len(nlines)):
                    if len(re.findall(r':\s*function\s*\(',nlines[j]))>0:
                        temp+='\n'+nlines[j]
                    if '{' in nlines[j]:
                        br+=nlines[j].count('{')
                        #print(nlines[j],end='\n\n')
                    if '}' in nlines[j]:
                        br-=nlines[j].count('}')
                    if br == 0 and '},' in nlines[j]:
                        if len(re.findall(r':\s*function\s*\(',nlines[j]))==0:
                            temp+='\n'+nlines[j]
                        #adding class defining code prior to function
                        temp=prefix2+temp+'\n'+suffix
                        # temp=temp+'\n'
                        #appending all functions snippets to list
                        func.append(temp)
                        temp=''
                    if len(temp)>0:
                        if len(re.findall(r':\s*function\s*\(',nlines[j]))>0:
                            continue
                        temp+='\n'+nlines[j]
                #replacing that paticular class string with list of function strings
                for script in func:
                    final_classlist.append(script)
            else:
                final_classlist.append(classlist[i])
        return final_classlist

    def gpt35turbo_large_script_generate(self,code):
        token_count = self.num_tokens_from_string(code,"cl100k_base")
        no_list = int(np.ceil(token_count/3000))
        list_func = self.split_func(code)
        # using list comprehension 
        num_rec = int(np.ceil(len(list_func)/no_list))
        list_func_split = [list_func[i:i + num_rec] for i in range(0, len(list_func), num_rec)]
        final_list=[]
        for i in list_func_split:
            if self.num_tokens_from_string('\n'.join(i),"cl100k_base")>4000:
                token_count_s = self.num_tokens_from_string('\n'.join(i),"cl100k_base")
                no_list_s = int(np.ceil(token_count_s/3000))
                list_func_s = self.split_func('\n'.join(i))
                # using list comprehension 
                num_rec_s = int(np.ceil(len(list_func_s)/no_list_s))
                list_func_split_s = [list_func_s[i:i + num_rec_s] for i in range(0, len(list_func_s), num_rec_s)]
                for j in list_func_split_s:
                    final_list.append('\n'.join(j))
            else:
                final_list.append('\n'.join(i))
        print("final_list:" + str(len(final_list)))
        response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken = self.parallel_api_call(final_list)
        
        return response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken


    def gpt35turbo_langchain_generate(self, code):

        start_time = datetime.now()
        prompt_response = None
        total_cost = 0.0
        tokens_count = 0
        model_used = ""
        error_message = ""
        error_type = ""
        azure_start_time = 0.0
        azure_end_time = 0.0
        input_prompt_size=0
        matching_docs=[]
        if len(code) <= 5:
            response = None
            error = "length of source code is too short to generate a response"
            error_type = "not_enough_tokens"

        elif len(code) > 5:

            #code = self.code_clean(code)
            prompt_template = PromptTemplate.from_template(
            '''
            Task:
            write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

            Instructions:
            - understand the code snippet below and write unit test cases
            - consider every scenario for unit testing
            - only write code
            - no comments and explanations for the written code

            code snippet:
            ####

            "{code}"

            ####

            Expected Output Format:
            // Test case: Testing....
            <code>
            .
            .

            ''')
            
            system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Javascript source code as input and you need to generate unit test cases in Jasmine framework as output "
           
            prompt = prompt_template.format(code = code)

            input_prompt_size = self.num_tokens_from_string(
                prompt, "cl100k_base")
            print("input prompt size: " + str(input_prompt_size))
            azure_start_time = datetime.now()
            if input_prompt_size < 1000:
                model_used = 'gpt-35-turbo'

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)
                if len(matching_docs) == 0:
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "user", "content": quote(prompt)}] 
                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_chat_completion(value)
                else:
                    matching_docs = self.restructure_document_object(matching_docs)
                    examples_retreived = self.few_shot_prompting_template(matching_docs)
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "system", "content": quote(examples_retreived)},
                             {"role": "user", "content": quote(prompt)}]

                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_chat_completion(value)


            elif input_prompt_size >= 1000 and input_prompt_size < 4000:
                model_used = 'gpt-35-turbo-16k'

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)
                
                if len(matching_docs) == 0:
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "user", "content": quote(prompt)}] 
                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_16k_chat_completion(value)
                else:
                    matching_docs = self.restructure_document_object(matching_docs)
                    examples_retreived = self.few_shot_prompting_template(matching_docs)
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "system", "content": quote(examples_retreived)},
                             {"role": "user", "content": quote(prompt)}]

                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_16k_chat_completion(value)

            else:
                model_used = 'gpt-35-turbo-16k'
                matching_docs = []
                value = [{"role": "system", "content": quote(system_message)},
                         {"role": "user", "content": quote(prompt)}] 
                prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_chat_completion(value)


        azure_end_time = datetime.now()

        end_time = datetime.now()
        azure_time_taken = round(
            (azure_end_time - azure_start_time).total_seconds(),
            2)  # type: ignore
        time_taken = round((end_time - start_time).total_seconds(), 2)
        return prompt_response, time_taken, tokens_count, total_cost, model_used, error_message, error_type, azure_time_taken




In [12]:

small_code = '''var ImpactSPMIntegrationService = Class.create();

ImpactSPMIntegrationService.prototype = {
    // Initialize tables used in integration
    initialize: function() {
        this.SPM_ENTITY_LINKAGE_DETAIL_TABLE = "x_snc_impact_spm_entity_linkage_detail";
        this.SPM_ENTITY_DETAIL_TABLE = "x_snc_impact_spm_entity_detail";
        this.SPM_ENTITY_TABLE = "x_snc_impact_spm_entity";
        this.SPM_INTEGRATION_SETUP_TABLE = "x_snc_impact_spm_integration_setup";
        this.CUSTOMER_CONTACTS_SPM_INTEGRATION_SETUP_RELATION_TABLE = "x_snc_impact_spm_m2m_x_snc_cust_c_x_snc_impac";
    }'''

In [13]:
medium_code = """
var TestGenerationUtil = Class.create();
TestGenerationUtil.tables = {
    test_assist_query: "x_snc_unit_test_ge_unit_test",
    test_assist_feedback: "x_snc_unit_test_ge_test_assist_feedback"
};
 
TestGenerationUtil.properties = {
    enable_test_assist: "x_snc_unit_test_ge.enable_test_assist",
    test_step_config_sys_id: "x_snc_unit_test_ge.test_step_config_sys_id",
    input_variable_sys_id: "x_snc_unit_test_ge.input_variable_sys_id"
};
 
TestGenerationUtil.event_name = "x_snc_unit_test_ge.run_test_assist_sync";
 
TestGenerationUtil.newGR = function(table_name) {
    var skipACL = true;
    return new(skipACL ? GlideRecord : GlideRecordSecure)(table_name);
};
 
TestGenerationUtil.testAssistEnabled = function() {
 
    var answer = false;
    var prop = gs.getProperty(TestGenerationUtil.properties.enable_test_assist, false) == "true" ? true : false;
    if (prop) {
        if (TestGenerationUtil.showTestAssistFunctionality()) {
            answer = true;
        }
    }
    return answer;
};
 
TestGenerationUtil.showTestAssistFunctionality = function() {
    var answer = true;
    if (!TestGenerationUtil.isTestAssistUser()) {
        answer = false;
    }
    return answer;
};
 
TestGenerationUtil.isTestAssistUser = function() {
    var answer = false;
    var roles = gs.getUser().getRoles();
    answer = (roles.indexOf('x_snc_unit_test_ge.user') > -1) || (roles.indexOf('x_snc_unit_test_ge.admin') > -1) ? true : false;
    return answer;
};
 
TestGenerationUtil.prototype = {
    initialize: function() {
        this.functionEndSyntax = '},';
        this.classEndSyntax = '};';
        this.scriptIncludeTableId = this.getTableId('sys_script_include');
    },
    getTableId: function(tableName) {
        var tableGr = new GlideRecord('sys_db_object');
        tableGr.addQuery('name', tableName);
        tableGr.query();
        if (tableGr.next()) {
            return tableGr.getUniqueValue();
        }
        return false;
    },
 
    createUnitTestRecord: function(scriptIncludeObj, sourceCode, documentRef, requestCategory) {
        try {
            var isCodeCoverageEnabled = gs.getProperty('x_snc_unit_test_ge.code_coverage.enabled');
            var atfUtil = new x_snc_unit_test_ge.UnitTestGenerationATFUtil();
            if (scriptIncludeObj == null && sourceCode == null && documentRef == null) {
                return false;
            } else {
                var unitTestGr = new GlideRecord('x_snc_unit_test_ge_unit_test');
                unitTestGr.initialize();
                if (scriptIncludeObj) {
                    var unitTestMappingRecord = atfUtil.getATFMappingRecord(scriptIncludeObj.getValue('sys_id'));
                    //gs.info("[TestAssistAjax] unitTestMappingRecord: " + unitTestMappingRecord);
                    if (unitTestMappingRecord) {
                        var deltaSourceCode = this.getDeltaSourceCode(scriptIncludeObj);
                        if (deltaSourceCode)
                            unitTestGr.setValue('source_code', deltaSourceCode);
                        else {
                            var errorMessage = "No new code changes found, all tests are up to date.";
                            gs.addErrorMessage(gs.getMessages(errorMessage));
                            return false;
                        }
                    } else {
                        unitTestGr.setValue('source_code', scriptIncludeObj.getValue('script'));
                    }
                    unitTestGr.setValue('document_id', scriptIncludeObj.getValue('sys_id'));
                    unitTestGr.setValue('reference_table', this.scriptIncludeTableId + '');
                } else if (sourceCode) {
                    unitTestGr.setValue('source_code', sourceCode);
                    unitTestGr.setValue('document_id', documentRef);
                    unitTestGr.setValue('reference_table', this.getTableId('x_snc_unit_test_ge_github_repo_request'));
 
                }
                var gdt = new GlideDateTime();
                unitTestGr.setValue('start_time', gdt);
                unitTestGr.setValue('request_category', requestCategory);
                unitTestGr.setValue('type', 'unit_test');
                unitTestGr.setValue('user', gs.getUserID());
                unitTestGr.setValue('client', gs.getProperty('instance_name'));
                unitTestGr.setValue('is_current', true);
                var rec = unitTestGr.insert();
                //gs.info("[TestAssistAjax] rec: " + rec + ", unitTestGr.isValidRecord(): " + unitTestGr.isValidRecord());
                if (rec) {
                    var existingUnitTestsGr = new GlideRecord('x_snc_unit_test_ge_unit_test');
                    existingUnitTestsGr.addQuery('document_id', unitTestGr.getValue('document_id'));
                    existingUnitTestsGr.addQuery('sys_id', '!=', unitTestGr.getUniqueValue());
                    existingUnitTestsGr.addQuery('is_current', true);
                    existingUnitTestsGr.query();
                    if (existingUnitTestsGr.hasNext()) {
                        existingUnitTestsGr.setValue('is_current', false);
                        existingUnitTestsGr.updateMultiple();
                    }
                    return unitTestGr;
                }
                return false;
            }
        } catch (e) {
            gs.error("[TestAssistAjax] createUnitTestRecord: " + e.toString());
        }
 
    },
 
    extractFunctionLineNumbers: function(code) {
        var functionRegex = /(\w+):[\s]*function\(/g;
        var lines = code.split('\n');
        var functions = [];
        var match;
 
        while ((match = functionRegex.exec(code)) !== null) {
            var functionName = match[1];
            var startLine = this.getFunctionStartLine(lines, match.index);
            //gs.info("match.index: " + match.index);
            var endLine = this.getFunctionEndLine(lines, match.index);
            functions.push({
                name: functionName,
                startLine: startLine,
                endLine: endLine
            });
        }
 
        return functions;
    },
 
    getFunctionStartLine: function(lines, index) {
        var line = 0;
        var charCount = 0;
 
        while (charCount <= index) {
            charCount += lines[line].length + 1;
            line++;
        }
 
        return line;
    },
 
    getFunctionEndLine: function(lines, index) {
        var line = 0;
        try {
            line = this.getFunctionStartLine(lines, index);
 
            while (!lines[line - 1].includes(this.functionEndSyntax)) { //&& !lines[line - 1].includes('}')
                line++;
                //gs.info("Line count: " + line);
            }
 
        } catch (e) {
            gs.error("Exception: " + e.message);
        }
 
        return line;
    },
 
    getDeltaSourceCode: function(scriptInlcudeRef) {
        var codeCoverageRecord = this.getCodeCoverageRecord(scriptInlcudeRef.getValue('sys_id'));
        if (codeCoverageRecord) {
            //gs.info("codeCoverageRecord: " + codeCoverageRecord.getValue('sys_id'));
            var coveredLines = (codeCoverageRecord.getValue('covered_lines')).split(','); //(codeCoverageRecord.getValue('lines')).split(',');
            var executableLines = this.getExecutableLineNumbers(scriptInlcudeRef.getValue('script')).split(','); //new global.TestGenerationUtilGlobal().getExecutableLines(scriptInlcudeRef.getValue('script')); //(new SNC.JSCoverageSourceParser().parse(scriptInlcudeRef.getValue('script'))).replace('[', '').replace(']', '').split(', ');
            var uncoveredLines = executableLines.filter(function(line) {
                return coveredLines.indexOf(line) == -1;
            });
 
            //gs.info("coveredLines: Length: " + coveredLines.length + "\n Line Numbers: " + coveredLines);
            //gs.info("executableLines: " + executableLines.length + "\n Line Numbers: " + executableLines);
            //gs.info("uncoveredLines: " + uncoveredLines);
            //uncoveredLines = [90, 100, 128, 129, 130];
            var functionLineNumbers = this.extractFunctionLineNumbers(scriptInlcudeRef.getValue('script'));
            //gs.info("functionLineNumbers: " + JSON.stringify(functionLineNumbers) + ", length: " + functionLineNumbers.length);
 
            var deltaFunctionsObj = [];
            var deltaFunctions = [];
            for (var i = 0; i < uncoveredLines.length; i++) {
                for (var j = 0; j < functionLineNumbers.length; j++) {
                    var obj = functionLineNumbers[j];
                    if (uncoveredLines[i] >= obj.startLine && uncoveredLines[i] <= obj.endLine) {
                        if (deltaFunctions.indexOf(obj.name) == -1) {
                            deltaFunctions.push(obj.name);
                            deltaFunctionsObj.push(obj);
                        }
                    }
                }
            }
 
            var scriptArray = (scriptInlcudeRef.getValue('script')).split("\n");
            var sourceCode = '';
 
            //gs.info("Script Line Numbers: " + scriptArray.length);
            for (var k = 0; k < deltaFunctionsObj.length; k++) {
                var functionObj = deltaFunctionsObj[k];
                sourceCode = sourceCode + (this.getSubsetArray(scriptArray, functionObj.startLine, functionObj.endLine)).join("\n").trim() + "\n";
            }
            sourceCode = this.addTemplateScript(scriptInlcudeRef, sourceCode);
            //gs.info(sourceCode);
            //this.createUnitTestRecord(null, sourceCode, null);
        } else {
            sourceCode = scriptInlcudeRef.getValue('script');
        }
 
 
        return sourceCode;
 
    },
 
    addTemplateScript: function(scriptInlcudeRef, sourceCode) {
        var serverSideScriptTemplateStart = 'var <scriptName> = Class.create();' + '\n' + '<scriptName>.prototype = {' + '\n';
        var clientSideScriptTemplateStart = 'var <scriptName> = Class.create();' + '\n' + '<scriptName>.prototype = Object.extendsObject(global.AbstractAjaxProcessor, {' + '\n';
        var clientSideScriptTemplateEnd = "type: '<scriptName>'" + "\n" + "});";
        var serverSideScriptTemplateEnd = "type: '<scriptName>'" + "\n" + "};";
 
        var isClientCallable = scriptInlcudeRef.getValue('client_callable');
        var codeTemplateStart;
        var codeTemplateEnd;
        var scriptCode;
        if (isClientCallable == true || isClientCallable == 'true') {
            codeTemplateStart = clientSideScriptTemplateStart;
            codeTemplateEnd = clientSideScriptTemplateEnd;
        } else {
            codeTemplateStart = serverSideScriptTemplateStart;
            codeTemplateEnd = serverSideScriptTemplateEnd;
        }
        scriptCode = codeTemplateStart + sourceCode + "\n" + codeTemplateEnd;
        scriptCode = (scriptCode.replaceAll('<scriptName>', scriptInlcudeRef.getValue('name')));
 
        return scriptCode;
    },
 
    getSubsetArray: function(array, startIndex, endIndex) {
        return array.slice(startIndex - 1, endIndex + 1);
    },
 
    isInRange: function(jsonArray, number) {
        for (var i = 0; i < jsonArray.length; i++) {
            var obj = jsonArray[i];
            if (number >= obj.startLine && number <= obj.endLine) {
                return true;
            }
        }
        return false;
    },
 
    getExecutableLineNumbers: function(code) {
        var lines = code.split('\n');
        var executableLineNumbers = [];
 
        var inMultilineComment = false;
 
        for (var i = 0; i < lines.length; i++) {
            var line = lines[i].trim();
            // Exclude empty lines
            if (line !== '') {
                if (!inMultilineComment) {
                    // Exclude single-line comments and multiline comment start lines
                    if (!line.startsWith('//') && !line.startsWith('/*') && line != '{' && line != '}' && line != this.functionEndSyntax && line != this.classEndSyntax) {
                        executableLineNumbers.push(i + 1);
                    }
                }
                // Check for multiline comment start and end
                if (line.includes('/*')) {
                    inMultilineComment = true;
                }
                if (line.includes('*/')) {
                    inMultilineComment = false;
                }
            }
        }
        return executableLineNumbers + "";
    },
 
    getCodeCoverageRecord: function(scriptIncludeId) {
        //var codeCoverageRec = new GlideRecord('sys_js_code_coverage');
        var codeCoverageRec = new GlideRecord('x_snc_code_coverge_coverage_file_records');
        //codeCoverageRec.addQuery('script_doc_id', scriptIncludeId);
        codeCoverageRec.addQuery('script_name', 'CONTAINS', scriptIncludeId);
        codeCoverageRec.orderByDesc('sys_created_on');
        codeCoverageRec.query();
        if (codeCoverageRec.next()) {
            return codeCoverageRec;
        }
        return false;
    },
 
    getInstance: function() {
        var instance = gs.getProperty("x_snc_unit_test_ge.instance_url");
        if (gs.nil(instance)) {
            return;
        }
        instance = instance.trim().toLowerCase();
        if (instance.indexOf("https://") == -1 && instance.indexOf("http://") == -1) {
            instance = "https://" + instance;
        }
        return instance;
    },
 
    scheduleSync: function() {
        var result = 'failed';
        try {
            this.queueEvent();
        } catch (e) {
            gs.error("[Test Assist] scheduleSync Error: " + e.toString());
            result = 'failed';
        }
        return result;
    },
 
    queueEvent: function(expiration_numeric) {
        if (expiration_numeric) {
            var gdt = new GlideDateTime();
            gdt.setNumericValue(expiration_numeric * 1000);
            gs.eventQueueScheduled(TestGenerationUtil.event_name, null, null, null, gdt);
        } else {
            gs.eventQueue(TestGenerationUtil.event_name, null, null);
        }
    },
 
    runTestAssistSync: function() {
        try {
            var instance = this.getInstance();
            if (gs.nil(instance)) {
                // Instance URL not defined
                gs.error("[Test Assist] runTestAssistSync: Instance URL not defined");
                return;
            }
 
            var payload = {};
            payload.instance_name = gs.getProperty('instance_name');
 
            // Last check date/time
            var last_check = gs.getProperty("x_snc_unit_test_ge.last_updated");
            var last_check_GDT = undefined;
            if (!gs.nil(last_check)) {
                last_check = parseInt(last_check);
                payload.last_check = last_check;
                last_check_GDT = new GlideDateTime();
                last_check_GDT.setNumericValue(parseInt(last_check));
            }
 
            // current date/time
            var currentGDT = new GlideDateTime();
            var last_check_end = parseInt(currentGDT.getNumericValue());
            payload.last_check_end = last_check_end;
 
            // Test Assist Feedback
            var test_assist_feedback = this.getTestAssistFeedback(last_check_GDT, currentGDT);
            if (test_assist_feedback) {
                payload.test_assist_feedback = test_assist_feedback;
            }
 
            var test_assist_user_action = this.getTestAssistUserAction(last_check_GDT, currentGDT);
            if (test_assist_user_action) {
                payload.test_assist_user_action = test_assist_user_action;
            }
 
            //gs.info("[Test Assist] runTestAssistSync payload: " + JSON.stringify(payload));
            var user_name = gs.getProperty('x_snc_unit_test_ge.test_assist_api_user_name');
            var user_password = gs.getProperty('x_snc_unit_test_ge.test_assist_api_user_password');
            var syncPOST = new sn_ws.RESTMessageV2("x_snc_unit_test_ge.Test Assist", "Sync");
            syncPOST.setBasicAuth(user_name, user_password);
            syncPOST.setStringParameter("instance", instance);
            syncPOST.setRequestBody(JSON.stringify(payload));
 
            var sync_response = syncPOST.execute();
            //gs.info("[Test Assist] runTestAssistSync body: " + sync_response.getStatusCode());
            var sync_result = {
                status_code: sync_response.getStatusCode(),
                ratelimit_reset: sync_response.getHeader("X-RateLimit-Reset"),
                body: JSON.parse(sync_response.getBody())
            };
 
            if ((sync_result.status_code == 429) && (sync_result.body.error.message == "Rate limit exceeded")) {
                // Rate limit exceeded
                this.queueEvent(sync_result.ratelimit_reset);
                return;
            } else if (sync_result.status_code == 200) {
                //gs.info("[Test Assist] runTestAssistSync body: " + JSON.stringify(sync_result));
                gs.setProperty("x_snc_unit_test_ge.last_updated", parseInt(currentGDT.getNumericValue() + 1000));
            }
        } catch (e) {
            gs.error("[Test Assist] runTestAssistSync Error: " + e.toString());
            result = "failed";
        }
    },
 
 
    getTestAssistUserAction: function(last_check_GDT, currentGDT) {
        var queryGR = TestGenerationUtil.newGR(TestGenerationUtil.tables.test_assist_query);
        var encodedQuery = 'progress_idISNOTEMPTY^test_created=true^sys_updated_on<' + currentGDT;
        if (!gs.nil(last_check_GDT)) {
            encodedQuery += "^sys_updated_on>=" + last_check_GDT;
        }
        queryGR.addEncodedQuery(encodedQuery);
        queryGR.query();
        if (!queryGR.hasNext()) {
            return;
        }
        var user_actions = [];
        while (queryGR.next()) {
            var test_modified = queryGR.getValue("modified");
            var test_script_modified = '';
            if (test_modified == true || test_modified == 'true') {
                test_modified = true;
                test_script_modified = queryGR.getValue("test_script_modified");
            } else {
                test_modified = false;
            }
            user_actions.push({
                progress_id: queryGR.getValue("progress_id") + '',
                test_created: true,
                manual_edits: test_modified,
                test_script_modified: test_script_modified
            });
        }
 
        return user_actions;
    },
 
    getTestAssistFeedback: function(last_check_GDT, currentGDT) {
        var queryGR = TestGenerationUtil.newGR(TestGenerationUtil.tables.test_assist_query);
        var encodedQuery = 'status=completed^progress_idISNOTEMPTY^feedbackISNOTEMPTY^sys_updated_on<' + currentGDT;
        if (!gs.nil(last_check_GDT)) {
            encodedQuery += "^sys_updated_on>=" + last_check_GDT;
        }
        queryGR.addEncodedQuery(encodedQuery);
        queryGR.query();
        if (!queryGR.hasNext()) {
            return;
        }
        var feedbacks = {
            query_records: [],
            feedback_records: {}
        };
        while (queryGR.next()) {
            var feedbackGR = queryGR.feedback.getRefRecord();
            var userGR = queryGR.user.getRefRecord();
            if (feedbackGR) {
                var query_record = {
                    progress_id: queryGR.getValue("progress_id"),
                    feedback_id: queryGR.getValue("feedback")
                };
                var feedback_record = {
                    user_rating: feedbackGR.getValue("user_rating"),
                    user_feedback: feedbackGR.getValue("user_feedback"),
                    user_name: userGR.getValue('user_name'),
                    feedback_id: query_record.feedback_id
                };
                feedbacks.query_records.push(query_record);
                feedbacks.feedback_records[query_record.feedback_id] = feedback_record;
            }
        }
 
        return feedbacks;
    },
 
    type: 'TestGenerationUtil'
};
"""

In [14]:
large_code = '''
var AutoAgentHandler = Class.create();
AutoAgentHandler.prototype = {
    initialize: function() {
        this.className = "AutoAgentHandler";
    },
    /**
     * @author Ram Krushna Mishra
     * @description internal method to create trigger
     * @task 
     * @param {<String>} <triggerScript> - <description>
     * @param {<String>} <triggerTime> - <description>
     * @param {<String>} <triggerName> - <description>
     * @returns {<String>} <sys id of the inserted record>
     */
    _createTrigger: function(triggerScript, triggerTime, triggerName) {
        if (JSUtil.nil(triggerScript) || JSUtil.nil(triggerTime) || JSUtil.nil(triggerName)) {
            gs.info("Invalid input provided while creating trigger triggerScript : " + triggerScript + ",triggerTime :" + triggerTime + ",triggerName : " + triggerName, this.className);
            return '';
        }

        var scheduler = new ScheduleOnce();
        scheduler.setLabel(triggerName);
        scheduler.script = triggerScript;
        scheduler.setTime(triggerTime);
        return scheduler.schedule();
    },
    /**
     * @author Ram Krushna Mishra
     * @description create trigger to be executed after the provided delay
     * @task
     * @param {<Glide Object>} <autoAgentConfigGr> - <GlideRecord of the auto agent configuration>
     * @param {<Glide Object>} <taskGr> - <GlideRecord of the task>
     * @param {<Sys Id>} <userId> - <Sys id of the user who intiated the transaction>
     * @param {<Boolean>} Flag to indicate that an active flow was deactivated
     * @returns {<Sys Id>} <Sys id of the created trigger record>
     */
    createTriggerWithDelay: function(autoAgentConfigGr, taskGr, userId, isFlowDeactivated) {
        if (JSUtil.nil(autoAgentConfigGr) || !autoAgentConfigGr.isValidRecord() || JSUtil.nil(taskGr)) {
            gs.info("Invalid input provided while creating trigger with delay ", this.className);
            return '';
        }
        // Calculate the next action time
        var nowDate = new GlideDateTime();
        var delay = autoAgentConfigGr.getValue('u_transfer_delay_in_seconds') ? parseInt(autoAgentConfigGr.getValue('u_transfer_delay_in_seconds')) : 0;
        //Trigger the flow immediately if active flow was deactivated or there is no transfer delay
        if (isFlowDeactivated || JSUtil.nil(delay))
            delay = 0;
        nowDate.addSeconds(delay);

        var triggerScript = "new AutoAgentHandler().executeAutoAgentFlow('" + taskGr.getUniqueValue() + "','" + taskGr.getTableName() + "','" + userId + "','" + autoAgentConfigGr.getUniqueValue() + "');",
            triggerName = AutoAgentConstants.DELAY_TRIGGER_PREFIX + taskGr.getValue('number') + '_' + gs.nowDateTime();
        //Adding logs for additonal troubleshooting
        gs.info('createTriggerWithDelay: Creating auto agent trigger with delay for taskId: ' + taskGr.getValue('number') + ' , autoAgentConfigGr : ' + autoAgentConfigGr.getValue('sys_id'), this.className);
        // Create Trigger
        return this._createTrigger(triggerScript, nowDate, triggerName);

    },

    /**
     * @author Vaibhav Kadam
     * @description create trigger to evaluate and execute auto agent flow 
     * @param {Sys id} Sys id of the task record
     * @param {String} Table Name of the task 
     * @param {Sys Id} Sys Id of the user
     * @param {Boolean} Interactive session or not
     * @returns {<sys id>} <description>
     */
    triggerAutoAgentFlow: function(taskId, tableName, userId, isInteractiveSession, isAssignedToChangedFromAA) {

        //Adding logs for additional troubleshooting
        gs.info('Inside triggerAutoAgentFlow for taskId : ' + taskId, this.className);
        if (JSUtil.nil(taskId) || JSUtil.nil(userId)) {
            gs.info("Invalid triggerAutoAgentFlow : Creating auto agent trigger for taskId: ", this.className);
            return '';
        }
        var triggerScript = "",
            triggerName = AutoAgentConstants.MAIN_TRIGGER_PREFIX + taskId,
            triggertime = new GlideDateTime();
        if (isAssignedToChangedFromAA) {
            triggerScript = "new AutoAgentHandler().cancelFlowWhenAssignedToChangesFromAA('" + taskId + "', false);";

        } else {
            triggerScript = "new AutoAgentHandler().executeAutoAgentFlow('" + taskId + "','" + tableName + "','" + userId + "');";
            if (JSUtil.notNil(isInteractiveSession) && !isInteractiveSession)
                triggertime.addSeconds(90); //Add 90 seconds delay to ensure there are no parallel updates happening on case when update is done via email reply
            else
                triggertime.addSeconds(30); //Adding 30 seconds delay to ensure there are no parallel updates happening on case
        }
        return this._createTrigger(triggerScript, triggertime, triggerName);
    },
    /**
     * @author Ram Krushna Mishra
     * @description get trasnfer delay from autoconfign record
     * @task
     * @param {<String>} <autoAgentConfigId> - <sysId of the auto agent configuration>
     * @returns {<String>} <transfer delay in seconds>
     */
    _getAutoAgentConfigRecord: function(autoAgentConfigId) {
        var autoAgentConfigDetails = new AutoAgentDao(AutoAgentConstants.CONFIGURATION_TABLE).getGlideRecordById(autoAgentConfigId);
        var autoAgentConfigGr = JSUtil.notNil(autoAgentConfigDetails.record) ? autoAgentConfigDetails.record : null;
        if (JSUtil.nil(autoAgentConfigGr)) {
            gs.info("Not able to fetch glide record with  autoAgentConfigId : " + autoAgentConfigId, this.className);
            return null;
        }
        return autoAgentConfigGr;
    },
    /**
     * @author Ram Krushna Mishra
     * @description get reference to auto agent definitions by order
     * @task
     * @param {<String>} <autoAgentConfigId> - <sysId of the auto agent configuration>
     * @returns {<GlideRecord>} <Reference to Auto Agent Definitions>
     */
    _getAutoAgentDefinitions: function(autoAgentConfigId) {
        if (JSUtil.nil(autoAgentConfigId))
            return null;

        var definitionQuery = 'u_auto_agent_configuration=' + autoAgentConfigId + '^u_active=true';
        var availabilityDetails = new AutoAgentDao(AutoAgentConstants.AVAILABILITY_TABLE).getGlideRecordByEncodedQuery(definitionQuery, 'u_order', false);
        var availabilityGr = JSUtil.notNil(availabilityDetails.record) ? availabilityDetails.record : null;
        if (JSUtil.nil(availabilityGr)) {
            gs.info("Not able to fetch availability records with  autoAgentConfigId : " + autoAgentConfigId, this.className);
            return null;
        }
        return availabilityGr;
    },
    /**
     * @author Ram Krushna Mishra
     * @description Execute Auto agent flow and evaluate the entry condition again and assign to auto agent if true. If any global exit condition is satisfied, do not assign and end the flow.
     * @param {Sys Id} sys id of the Task record 
     * @param {String} Table name of the task record
     * @param {Sys Id} sys id of the user record
     * @param {Sys Ids} sys ids of the auto agent configuration record
     * @returns {void} 
     */
    executeAutoAgentFlow: function(taskSysId, tableName, userId, autoAgentConfigIds) {
        gs.info("Inside executeAutoAgentFlow for " + taskSysId + " and autoAgentConfigIds " + autoAgentConfigIds, this.className);
        try {
            if (JSUtil.nil(taskSysId) || JSUtil.nil(tableName)) {
                gs.info("Invalid inputs provided to executeAutoAgentFlow ", this.className);
                return null;
            }
            var taskGr = this._getTaskRec(taskSysId, tableName);
            if (JSUtil.nil(taskGr)) {
                gs.info("Unable to fetch task record to executeAutoAgentFlow ", this.className);
                return null;
            }
            if (taskGr.u_time_zone_realignment != '') {
                return null;
            }

            // If AA is disable for record make sure all executions are completed.
            if (taskGr.getValue('u_aa_status') === AutoAgentConstants.AA_STATUS.DISABLED) {
                gs.info("AutoAgent: Unable to to start due case " + taskGr.number + " status is disabled", this.className);
                this.disableAA(taskGr, false);
                return null;
            }

            var fromDelayTrigger = false; //Flag to verify if this function is called from the delay trigger code

            if (JSUtil.notNil(autoAgentConfigIds)) {
                fromDelayTrigger = true;
            }

            var exitDetails = '';
            var skipAllResult = this.getValidSkipAllScenario(taskGr);

            if (JSUtil.notNil(skipAllResult)) {
                exitDetails = this.handleSkipAllExitScenario(taskGr, skipAllResult);
                this.handleTaskAssignment(taskGr, exitDetails, userId);

                // Set AA Status as disabled for skiped tasks
                this.disableAA(taskGr, true);
                return;
            }

            //If there is any active log record, evaluate the corresponding exit condition
            var activeLogRecord = this.getActiveExecutionLog(taskGr.getUniqueValue());
            if (JSUtil.notNil(activeLogRecord))
                exitDetails = this.handleActiveLogRecords(taskGr, activeLogRecord);
            //check if there are any parallel flows to be evaluated
            if (JSUtil.notNil(exitDetails) && exitDetails.hasOwnProperty('parallel_flows') && JSUtil.notNil(exitDetails.parallel_flows)) {
                autoAgentConfigIds += exitDetails.parallel_flows;
            }
            //If there is an active log record and no parallel flows exists, then return
            if (JSUtil.notNil(exitDetails) && exitDetails.hasOwnProperty('has_active_log') && exitDetails.has_active_log && JSUtil.nil(exitDetails.parallel_flows))
                return;

            //Determine whether the active log record was deactivated or not
            var isFlowDeactivated = JSUtil.notNil(exitDetails) && exitDetails.hasOwnProperty('auto_config_id') && JSUtil.notNil(exitDetails.auto_config_id);

            // Evaluate the entry condition and assign to auto agent, if true
            var startDetails = this.evaluateEntryConditions(taskGr, autoAgentConfigIds, fromDelayTrigger, false, isFlowDeactivated);

            if (isFlowDeactivated && !startDetails.new_flow_triggered) {
                try {
                    this.handleTaskAssignment(taskGr, exitDetails, userId);
                } catch (ex) {
                    gs.info("handleExitScenario: " + ex.message + " at" + ex.stack);
                }

                // Move AA Status back to inactive and update work_notes with mention to the user
                this.handleExitUserWorknoteMention(taskGr.getUniqueValue(), taskGr.getTableName());
            }

        } catch (ex) {
            var message = ex.getMessage();
            gs.info("Exception in executeAutoAgentFlow method. Exception Details : " + message + ex.stack, this.className);
        }
    },
    /**
     * @author Lucas Caixeta
     * @description return the first skip all scenario that match for the record
     * @param <GlideRecord> taskGr - Task that will be evaluated agains the skip all conditions
     * @returns <Object|null> The skip all result or null if not found
     */
    getValidSkipAllScenario: function(taskGr) {
        if (JSUtil.nil(taskGr) || !taskGr.isValidRecord()) {
            return;
        }

        // Evaluate skip all conditions, if true then exit
        var skipAllConfigurationGr = this.getSkipAllConfigurations(taskGr.getTableName());
        if (JSUtil.notNil(skipAllConfigurationGr)) {
            return this.isValidSkipAllScenario(taskGr, skipAllConfigurationGr);
        }
        return null;
    },
    /**
     * @author Lucas Caixeta
     * @description Validate if task as any valid skip all scenario
     * @param <GlideRecord> taskGr - Task that will be evaluated agains the skip all conditions
     * @returns <boolean> true if any result is found
     */
    hasValidSkipAllScenario: function(taskGr) {
        return JSUtil.notNil(this.getValidSkipAllScenario(taskGr));
    },
    /**
     * @author Lucas Caixeta
     * @description Return the name of all matching skip all configurations for specific task
     * @param <GlideRecord> taskGr - Task that will be evaluated agains the skip all conditions
     * @returns <Object|null> The skip all result or null if not found
     */
    getAllValidSkipAllScenarioNames: function(taskGr) {
        if (JSUtil.nil(taskGr) || !taskGr.isValidRecord()) {
            return;
        }
        // Evaluate skip all conditions, if true then exit
        var names = [];
        var skipAllConfigurationGr = this.getSkipAllConfigurations(taskGr.getTableName());
        if (JSUtil.notNil(skipAllConfigurationGr)) {
            do {
                var result = this.evalCondition(taskGr, skipAllConfigurationGr.getValue('u_conditions'));
                if (result) {
                    names.push(skipAllConfigurationGr.getDisplayValue());
                }
            }
            while (skipAllConfigurationGr.next());
        }
        return names;
    },
    /**
     * @author Lucas Schirm
     * @description Enable Needs attention and add work_note with user mention to it
     * @param <GlideRecord> Task case glide record
     * @return void
     */
    handleExitUserWorknoteMention: function(taskId, tableName) {
        if (JSUtil.nil(taskId) || JSUtil.nil(tableName))
            return;

        var taskGr = new GlideRecord(tableName);
        taskGr.get(taskId);

        gs.info("handleExitScenario: Task " + taskGr.number + " was set to inactive. Adding work note with user mention.");

        // if (taskGr.getValue('assgined_to') === AutoAgentConstants.AUTOAGENT_USER)
        //     return;

        taskGr.setValue('u_aa_status', AutoAgentConstants.AA_STATUS.INACTIVE);
        taskGr.setValue('needs_attention', true);
        taskGr.work_notes = gs.getMessage('ns.autoagent.exit.flow.notes', taskGr.getDisplayValue('assigned_to'));
        taskGr.update();
    },

    /**
     * @author Ram Krushna Mishra
     * @description Assign to the Case to Autoagent
     * @task 
     * @param {<Sys Id>} <taskId> - <time after which the trigger should get executed>
     * @param {<type>} <name> - <description>
     * @param {<type>} <name> - <description>
     * @returns {<type>} <description>
     */
    assignTaskToUser: function(taskId, tableName, userId) {
        if (JSUtil.nil(taskId) || JSUtil.nil(tableName))
            return false;

        var inputParams = {
            'assigned_to': userId
        };

        var taskGrDetails = new AutoAgentDao(tableName).updateById(inputParams, taskId);
        if (JSUtil.nil(taskGrDetails.sys_id)) {
            gs.info("Unable to assign user " + userId + " to task " + taskId + " of table " + tableName, this.className);
            return false;
        }
        return true;
    },
    /**
     * @author Ram Krushna Mishra
     * @description Retrieve Task GlideRecord from Task Id
     * @task 
     * @param {<String>} <taskId> - <sys id of the task>
     * @param {<String>} <tableName> - <table Name>
     * @returns {<GlideRecord>} <case Gliderecord>
     */
    _getTaskRec: function(taskId, tableName) {
        if (JSUtil.nil(taskId) || JSUtil.nil(tableName))
            return null;

        var taskRecDetails = new AutoAgentDao(tableName).getGlideRecordById(taskId);
        var taskGr = JSUtil.notNil(taskRecDetails.record) ? taskRecDetails.record : null;
        if (JSUtil.nil(taskGr)) {
            gs.info("Unable to fetch glide record with  taskId : " + taskId + ", tableName :" + tableName, this.className);
            return null;
        }
        return taskGr;
    },
    /**
     * @author Ram Krushna Mishra
     * @description Verify if Last Assignee is available using WFM API
     * @task 
     * @param {<autoAgentConfigId>} Sys_id of auto agent config record
     * @param {<taskId>} Sys_id of task record
     * @returns {<Boolean>} <isAvailable>
     */
    isLastAssigneeAvailable: function(autoAgentConfigId, taskGr) {

        if (JSUtil.nil(autoAgentConfigId) || JSUtil.nil(taskGr) || !taskGr.isValidRecord())
            return {
                answer: false
            };
        var periodInMins = 0,
            timeAvailableInMins = 0,
            availabilityDef = '';

        var autoAgentConfigGr = this._getAutoAgentConfigRecord(autoAgentConfigId);
        if (JSUtil.nil(autoAgentConfigGr))
            return {
                answer: false,
                errorMessage: ""
            };

        // Get the Last Assignee User ID
        var lastAssigneeUserId = taskGr.getValue(AutoAgentConstants.TASK_ASSIGNEE_FIELD_MAP[taskGr.getTableName()]);
        if (JSUtil.nil(lastAssigneeUserId))
            return {
                answer: false,
                errorMessage: ""
            };

        var availabilityGrs = this._getAutoAgentDefinitions(autoAgentConfigId);
        do {
            var condition = availabilityGrs.getValue('u_conditions');
            if (this.evalCondition(taskGr, condition)) {
                var periodInMs = availabilityGrs.u_total_duration.getGlideObject().getNumericValue(),
                    timeAvailableInMs = availabilityGrs.u_available_duration.getGlideObject().getNumericValue();
                periodInMins = periodInMs / 60000;
                timeAvailableInMins = timeAvailableInMs / 60000;
                availabilityDef = availabilityGrs.getValue('u_name');
                break;
            }
        } while (availabilityGrs.next());

        if (periodInMins == 0 || timeAvailableInMins == 0)
            return {
                answer: false,
                errorMessage: ""
            };

        var retVal = new AutoAgentIntegrationUtil().fetchAgentAvailability(lastAssigneeUserId, timeAvailableInMins, periodInMins);
        return {
            answer: retVal.agentAvailable,
            errorMessage: retVal.errorMessage,
            statusCode: retVal.status_code,
            availabilityDefinition: availabilityDef
        };
    },

    /**
     * @author Ram Krushna Mishra
     * @description evaluate the condition
     * @task 
     * @param {<String>} <delay> - <time after which the trigger should get executed>
     * @param {<type>} <name> - <description>
     * @param {<type>} <name> - <description>
     * @returns {<type>} <description>
     */
    evalCondition: function(sourceGr, queryCond) {
        if (JSUtil.nil(sourceGr) || !sourceGr.isValidRecord() || JSUtil.nil(queryCond))
            return false;

        return GlideFilter.checkRecord(sourceGr, queryCond);
    },

    /**
     * @author Vaibhav Kadam
     * @description Get active execution log for a task
     * @param {<Sys ID>} Sys_id of the task record
     * @param {<Sys Id>} Sys_id of the auto agent configuration record
     * @return valid GlideRecord or null
     * @example 
     * @see 
     */
    getActiveExecutionLog: function(taskId, autoAgentConfigId) {
        if (JSUtil.nil(taskId))
            return null;

        var encodedQuery = 'u_execution_status=' + AutoAgentConstants.EXECUTION_STATUS.IN_PROGRESS + '^u_task=' + taskId;
        if (JSUtil.notNil(autoAgentConfigId))
            encodedQuery += '^u_entry_configuration=' + autoAgentConfigId;

        var executionLogDetails = new AutoAgentDao(AutoAgentConstants.EXECUTION_TABLE).getGlideRecordByEncodedQuery(encodedQuery, '	sys_created_on', false),
            executionLogGr = JSUtil.notNil(executionLogDetails.record) ? executionLogDetails.record : null;

        if (!JSUtil.nil(executionLogGr)) {
            gs.info("Found active execution log record for task " + taskId + " and auto agent config " + autoAgentConfigId, this.className);
        }
        return executionLogGr;
    },

    /**
     * @author Vaibhav Kadam
     * @description Trigger a subflow associated with the auto agent config
     * @param {Glide Record} Glide record of auto agent config record
     * @param {<Sys ID>} Sys_id of the case record
     * @return valid GlideRecord or null
     * @example 
     * @see 
     */
    triggerSubFlow: function(autoAgentConfigGr, taskSysId) {
        try {
            if (JSUtil.nil(autoAgentConfigGr) || !autoAgentConfigGr.isValidRecord() || JSUtil.nil(taskSysId)) {
                gs.info("Invalid inputs provided to trigger subflow for task ", this.className);
                return null;
            }

            //Get the name of the subflow to be triggered
            var subflowName = this._getSubFlowDetails(autoAgentConfigGr.getValue('u_subflow'));
            if (JSUtil.nil(subflowName)) {
                return null;
            }

            // SFSTRY0098670: Use new run_as configuration to start subflow
            var runAsUser = autoAgentConfigGr.getValue('u_run_as'),
                originalUser = '';
            //Impersonate as auto agent user
            if (JSUtil.notNil(runAsUser)) {
                originalUser = this._impersonate(runAsUser);
            }
            //Build the inputs to be passed to the subflow
            var flowInputs = AutoAgentConstants.SUBFLOW_COMMON_INPUTS;
            flowInputs['task_id'] = taskSysId;
            flowInputs['auto_agent_configuration_id'] = autoAgentConfigGr.getUniqueValue();

            //trigger the subflow
            var result = sn_fd.FlowAPI.getRunner()
                .subflow(subflowName)
                .inBackground()
                .withInputs(flowInputs)
                .run();
            //DeImpersonate as auto agent user
            if (JSUtil.notNil(originalUser)) {
                originalUser = this._impersonate(originalUser);
            }
            //retrun the context id of the subflow which is triggered
            return result.getContextId();
        } catch (ex) {
            var message = ex.getMessage();
            gs.info("Exception in triggerSubFlow method. Exception details : " + message + ex.stack, this.className);
        }
    },
    /**
     * @author Vaibhav Kadam
     * @description Get subflow details associated with the auto agent config
     * @param {<Sys ID>} sys_id of the subflow
     * @return name of the subflow
     * @example 
     * @see 
     */
    _getSubFlowDetails: function(subflowId) {
        if (JSUtil.nil(subflowId))
            return null;

        var subflowDetails = new AutoAgentDao(AutoAgentConstants.SYS_HUB_FLOW_TABLE).getGlideRecordById(subflowId),
            subflowGr = JSUtil.notNil(subflowDetails.record) ? subflowDetails.record : null;

        if (JSUtil.nil(subflowGr))
            return null;

        var subflowName = String(subflowGr.sys_scope.scope) + '.' + subflowGr.getValue('internal_name');
        return subflowName;
    },
    /**
     * @author Vaibhav Kadam
     * @description Impersonate as auto agent user
     * @param {<Sys ID>} sys_id of the user to be impersonated
     * @return sysid of the user that was logged in prior to impersonation
     * @example 
     * @see 
     */
    _impersonate: function(userSysId) {
        if (!userSysId) {
            return null;
        }

        var originalUser = gs.getSession().impersonate(userSysId);
        return originalUser;
    },
    /**
     * @author Vaibhav Kadam
     * @description Cancel any active subflows for a case
     * @param {<Sys Id>} Sys id of the sys_flow_context table
     * @return void
     * @example 
     * @see 
     */
    cancelSubFlow: function(flowContextId) {
        try {
            gs.info('cancelAutoAgentSubFlow: Cancelling flowContextId: ' + flowContextId, this.className);
            sn_fd.FlowAPI.cancel(flowContextId, 'Canceling Sub Flow as one of exit conditions for auto agent is met');
        } catch (ex) {
            var message = ex.getMessage();
            gs.info("Exception in cancelSubFlow method. Exception details : " + message + ex.stack, this.className);
        }
    },
    /**
     * @author Vaibhav Kadam
     * @description Update record in u_auto_agent_execution_log table
     * @param {<Sys Id>} Sys id of the task record
     * @param {<Sys Id>} sys_id of the auto agent configuration table
     * @param {object} object with key value pairs of field and field value to be updated
     * @return void
     * @example 
     * @see 
     */
    updateExecutionLog: function(taskSysId, fieldDetails, autoAgentConfigId) {
        if (JSUtil.nil(taskSysId) || JSUtil.nil(fieldDetails))
            return null;

        var executionLogId = '',
            activeExecutionLogGr = this.getActiveExecutionLog(taskSysId, autoAgentConfigId);
        if (JSUtil.notNil(activeExecutionLogGr)) {
            do {
                var executionLogDetails = new AutoAgentDao(AutoAgentConstants.EXECUTION_TABLE).updateById(fieldDetails, activeExecutionLogGr.getUniqueValue());
                executionLogId = executionLogDetails.sys_id;
                if (JSUtil.nil(executionLogId)) {
                    gs.info("Unable to update execution Log record for task " + taskSysId + " and fields " + JSON.stringify(fieldDetails) + " Error details : " + executionLogDetails.errorMessage, this.className);
                }
            } while (activeExecutionLogGr.next());
        }
        return executionLogId;
    },

    /**
     * @author Ragunath Sigicherla
     * @description Get Exit Configuration with Skip All as true for given table.
     * @param {<String>} TableName - Table Name for which the SkipAll need to be verified
     * @return {boolean}
     * @example 
     * @see 
     */
    getSkipAllConfigurations: function(tableName) {
        var skipAllConfigurationGr = null,
            encodedQuery = "u_active=true^u_state=approved^u_type=exit^u_skip_all=true^u_table=" + tableName,
            configDetails = new AutoAgentDao(AutoAgentConstants.CONFIGURATION_TABLE).getGlideRecordByEncodedQuery(encodedQuery, 'u_order', false);

        skipAllConfigurationGr = JSUtil.notNil(configDetails.record) ? configDetails.record : null;

        return skipAllConfigurationGr;

    },

    /**
     * @author Vaibhav Kadam
     * @description Check if Auto agent is enabled
     * @param {String} Table name of the task record
     * @return {boolean} Returns whether the auto agent flow is enabled for a specific table or not
     * @example 
     * @see 
     */
    isAutoAgentFlowEnabled: function(tableName) {
        var flowEnablementDetails = gs.getProperty('ns.autoagent.enabled');
        if (flowEnablementDetails && JSON.parse(flowEnablementDetails).hasOwnProperty(tableName))
            return JSON.parse(flowEnablementDetails)[tableName];
        else
            return false;
    },

    /**
     * @author Vaibhav Kadam
     * @description Evalaute if it is a valid scenario to trigger auto agent flow for a specific record
     * @param {Glide Object} Glide record of a task  
     * @return {boolean} Returns if it is a valid scenario to evaluate auto agent flow conditions
     * @example 
     * @see 
     */
    isValidScenario: function(taskGr) {
        if (JSUtil.nil(taskGr) || !taskGr.isValidRecord())
            return false;

        var tableName = taskGr.getTableName();
        if (!this.isAutoAgentFlowEnabled(tableName))
            return false;

        var fieldDetails = gs.getProperty('ns.autoagent.trigger.fields');
        if (JSUtil.nil(fieldDetails) || !JSON.parse(fieldDetails).hasOwnProperty(tableName))
            return false;

        var fieldsList = JSON.parse(fieldDetails)[tableName],
            encodedQuery = '';
        //Build the encoded query to check if any of the fields have changed
        fieldsList.forEach(function(element) {
            if (JSUtil.nil(encodedQuery))
                encodedQuery = element + 'VALCHANGES';
            else
                encodedQuery += '^OR' + element + 'VALCHANGES';
        });

        if (JSUtil.nil(encodedQuery))
            return false;

        var isChildOfMajorCase = false,
            parentId = taskGr.getValue('parent');

        if (JSUtil.notNil(parentId))
            isChildOfMajorCase = this._isChildOfMajorCase(parentId);

        var result = this.evalCondition(taskGr, encodedQuery) && !isChildOfMajorCase;
        //Adding additional logs for debugging
        gs.info('AutoAgent:isValidScenario for task ' + taskGr.number + ' - Result: ' + result, this.className);
        return result;
    },

    /**
     * @author Vaibhav Kadam
     * @description Validate if case's parent is a major case or not
     * @param {Sys Id} sys_id fo the task recor
     * @return {Boolean} Returns true or false
     * @example 
     * @see 
     */
    _isChildOfMajorCase: function(parentId) {
        var encodedQuery = 'sys_id=' + parentId;
        var parentCaseDetails = new AutoAgentDao(AutoAgentConstants.CASE_TABLE).getGlideRecordByEncodedQuery(encodedQuery, '', '', true);
        var parentCaseRecord = JSUtil.notNil(parentCaseDetails.record) ? parentCaseDetails.record : null;
        if (JSUtil.notNil(parentCaseRecord)) {
            return parentCaseRecord.ref_sn_customerservice_case.u_major_case;
        }
        return false;
    },

    /**
     * @author Vaibhav Kadam
     * @description Deactivate log and cancel activate subflow if any
     * @param {Sys Id} sys_id fo the task record
     * @param {Sys Id} sys_id of the configuration record
     * @param {String} Error message from WFM api call, if any
     * @return {Sys Id} Returns sys_id of the record which is updated
     * @example 
     * @see 
     */
    deactivateLog: function(taskId, exitConfigId) {
        var fields = {};
        fields["u_exit_configuration"] = exitConfigId;
        fields["u_execution_status"] = AutoAgentConstants.EXECUTION_STATUS.COMPLETED;
        return this.updateExecutionLog(taskId, fields);
    },

    /**
     * @author Vaibhav Kadam
     * @description Validate if it is a valid Skip all scenario
     * @param {Glide Record} task record
     * @param {Glide Record} Auto agent configuration record
     * @return {sys id} Returns sys id if it is a valid scenario 
     * @example 
     * @see 
     */
    isValidSkipAllScenario: function(taskGr, skipAllConfigurationGr) {
        if (JSUtil.notNil(skipAllConfigurationGr)) {
            do {
                var result = this.evalCondition(taskGr, skipAllConfigurationGr.getValue('u_conditions'));
                if (result) {
                    return skipAllConfigurationGr.getUniqueValue();
                }
            }
            while (skipAllConfigurationGr.next());
        }
        return null;
    },

    /**
     * @author Vaibhav Kadam
     * @description Hanlde Exit scenario for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {Sys Id} sys_id of the exit configuration record
     * @return {void}
     * @example 
     * @see 
     */
    handleSkipAllExitScenario: function(taskGr, exitConfigurationId) {
        //Check if there are any active log records
        var executionLogGr = this.getActiveExecutionLog(taskGr.getUniqueValue());
        if (JSUtil.notNil(executionLogGr)) {
            return this.handleExitScenario(taskGr, executionLogGr, exitConfigurationId);
        }
        return {
            auto_config_id: '',
            execution_log_id: ''
        };
    },

    /**
     * @author Vaibhav Kadam
     * @description Hanlde Exit scenario for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {GlideRecord} GlideRecord of the Execution Log record
     * @param {Sys Id} sys_id of the exit configuration record
     * @return {Object} object with additional exit details
     * @example 
     * @see 
     */
    handleExitScenario: function(taskGr, executionLogGr, exitConfigurationId) {
        var entryConfigId = '',
            executionLogId = '';
        if (JSUtil.notNil(executionLogGr) && executionLogGr.isValidRecord() && JSUtil.notNil(exitConfigurationId)) {
            do {
                entryConfigId = executionLogGr.getValue('u_entry_configuration');
                //Cancel Sublfow, if any
                if (JSUtil.notNil(executionLogGr.getValue('u_subflow_context'))) {
                    gs.info('handleExitScenario: cancelling the subflow context for executionLog: ' + executionLogGr.getValue('sys_id'), this.className);
                    this.cancelSubFlow(executionLogGr.getValue('u_subflow_context'));
                }
                //Deactivate execution log record
                executionLogId = this.deactivateLog(taskGr.getUniqueValue(), exitConfigurationId);
            }
            while (executionLogGr.next());
        }
        return {
            auto_config_id: entryConfigId,
            execution_log_id: executionLogId
        };
    },
    /***
     * @author Lucas Schirm
     * @description Check if assigned_to is an auto agent
     * @param {GlideRecord} - task record
     * @return boolean
     */
    isAssignedToAutoAgent: function(taskGr) {
        // if taskgr is not a valid record, return false
        if (JSUtil.nil(taskGr) || !taskGr.isValidRecord()) {
            return false;
        }

        return taskGr.assigned_to && taskGr.assigned_to.u_type != 'SNC Employee' || taskGr.assigned_to == AutoAgentConstants.AUTOAGENT_USER;
    },
    /**
     * @author Vaibhav Kadam
     * @description Hanlde user assignment scenario for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {Object} Object with auto agent configuration sys id and execution log sys id
     * @return {void}
     * @example 
     * @see 
     */
    handleTaskAssignment: function(taskRecordGr, additionalInfo, userId) {
        var taskGr = this._getTaskRec(taskRecordGr.getUniqueValue(), taskRecordGr.getTableName());
        //Do not change the assigned to if state is closed
        if (taskGr.getValue('state') == HICaseConstants.states.CLOSED)
            return;

        // SFSTRY0098670: If assigned to is not an Auto Agent user it should ignore function
        if (!this.isAssignedToAutoAgent(taskGr)) {
            return;
        }

        //SFSTRY0042914: Do not check the availability of last assignee who is updating the case
        var last_assignee = taskGr.getValue(AutoAgentConstants.TASK_ASSIGNEE_FIELD_MAP[taskGr.getTableName()]),
            assignee = "";
        // if (taskGr.assigned_to != AutoAgentConstants.AUTOAGENT_USER) { ////autoagent sysID
        //     taskGr.work_notes = gs.getMessage('ns.autoagent.case.not.reassigned.notes');
        //     taskGr.update();
        //     return;
        // }
        if (last_assignee == userId)
            assignee = last_assignee;
        else {
            //Get user availability
            var userAvailablityDetails = this.isLastAssigneeAvailable(additionalInfo.auto_config_id, taskGr),
                validRestCall = userAvailablityDetails.hasOwnProperty('statusCode') && AutoAgentConstants.VALID_HTTP_CODES.indexOf(userAvailablityDetails.statusCode) > -1;
            if (userAvailablityDetails.answer)
                assignee = last_assignee;
            else if (JSUtil.notNil(userAvailablityDetails.errorMessage) && JSUtil.notNil(additionalInfo.execution_log_id)) {
                var fields = {};
                if (!validRestCall)
                    fields["u_execution_status"] = AutoAgentConstants.EXECUTION_STATUS.COMPLETED_WITH_ERRORS;
                fields["u_message"] = userAvailablityDetails.errorMessage;
                var executionLogDetails = new AutoAgentDao(AutoAgentConstants.EXECUTION_TABLE).updateById(fields, additionalInfo.execution_log_id);
                //Assign to last assignee even in error scenario
                if (!validRestCall)
                    assignee = last_assignee;
            }
            //Update worknotes if assignee is unavailable and it is a valid rest call
            if (userAvailablityDetails.hasOwnProperty('availabilityDefinition') && !userAvailablityDetails.answer && validRestCall) {
                this._updateTaskWorkNotes(taskGr, userAvailablityDetails.availabilityDefinition);
            }
        }
        //Assign back to the user
        this.assignTaskToUser(taskGr.getUniqueValue(), taskGr.getTableName(), assignee);
    },

    /**
     * @author Vaibhav Kadam
     * @description Hanlde user assignment scenario for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {GlideRecord} GlideRecord of the execution log table
     * @return {Object} Object with additional exit details
     * @example 
     * @see 
     */
    handleActiveLogRecords: function(taskGr, executionLogGr) {
        var exitDetails = {},
            hasActiveRecord = false,
            activeCount = 0,
            parallelFlowList = [];
        if (JSUtil.notNil(taskGr) && taskGr.isValidRecord() && JSUtil.notNil(executionLogGr) && executionLogGr.isValidRecord()) {
            do {
                activeCount++;
                //Evaluate exit configuration associated with the Entry configuration
                var isValidExit = false,
                    exitConditionQuery = String(executionLogGr.u_entry_configuration.u_exit_configuration.u_conditions);
                if (JSUtil.notNil(exitConditionQuery))
                    isValidExit = this.evalCondition(taskGr, exitConditionQuery);
                if (isValidExit) {
                    gs.info("Found valid exit for task " + taskGr.number + " with entry config " + executionLogGr.u_entry_configuration.getDisplayValue(), this.className);
                    //Handle exit scenario
                    exitDetails = this.handleExitScenario(taskGr, executionLogGr, String(executionLogGr.u_entry_configuration.u_exit_configuration));
                    activeCount--;
                } else {
                    if (JSUtil.notNil(String(executionLogGr.u_entry_configuration.u_parallel_flows)))
                        parallelFlowList.push(String(executionLogGr.u_entry_configuration.u_parallel_flows));
                }
            } while (executionLogGr.next());
        }

        exitDetails.parallel_flows = parallelFlowList.toString();
        exitDetails.has_active_log = activeCount > 0 ? true : false;
        return exitDetails;
    },

    /**
     * @author Vaibhav Kadam
     * @description Evaluate Entry config conditions for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {String} List of auto agent configuration ids
     * @param {Boolean} Flag to determine if this is triggered from sys_trigger record logic
     * @param {Boolean} Flag to determine if this is triggered from parallel flow logic
     * @param {Boolean} Flag to determine if any active flow was deactivated
     * @return {Sys Id} Sys Id of the user record
     * @example 
     * @see 
     */
    evaluateEntryConditions: function(taskGr, entryConfigIds, fromTrigger, isParallelFlow, isFlowDeactivated) {
        var taskAssignee = '',
            newFlowTriggered = false,
            isValidStart = false,
            startDetails = {},
            encodedQuery = "u_active=true^u_state=approved^u_type=entry^u_conditionsISNOTEMPTY^u_table=" + taskGr.getTableName();
        if (JSUtil.notNil(entryConfigIds) && isParallelFlow)
            encodedQuery += '^sys_idIN' + entryConfigIds;

        var configDetails = new AutoAgentDao(AutoAgentConstants.CONFIGURATION_TABLE).getGlideRecordByEncodedQuery(encodedQuery, 'u_order', false),
            entryConfigGr = JSUtil.notNil(configDetails.record) ? configDetails.record : null;
        if (JSUtil.notNil(entryConfigGr)) {
            do {
                isValidStart = this.evalCondition(taskGr, entryConfigGr.getValue('u_conditions'));
                if (isValidStart && fromTrigger) {
                    var subFlowContextId = '';
                    taskAssignee = entryConfigGr.getValue('u_auto_agent_user');
                    //Trigger flow and add an entry in execution log table if doesn't exist already
                    var activeLogRecord = this.getActiveExecutionLog(taskGr.getUniqueValue(), entryConfigGr.getUniqueValue());
                    if (JSUtil.nil(activeLogRecord)) {
                        //Trigger subflow if any
                        if (JSUtil.notNil(entryConfigGr.getValue('u_subflow'))) {
                            subFlowContextId = this.triggerSubFlow(entryConfigGr, taskGr.getUniqueValue());
                        }
                        //add an entry in execution log table
                        this._createExecutionLog(taskGr.getUniqueValue(), entryConfigGr.getUniqueValue(), subFlowContextId);

                        // Update AA Status to active state
                        this.setAAStatusActive(taskGr);
                    } else
                        newFlowTriggered = true; //this will help to check when an active record already exists

                } else if (isValidStart) {
                    this.createTriggerWithDelay(entryConfigGr, taskGr, gs.getUserID(), isFlowDeactivated);
                }

                if (isValidStart & JSUtil.nil(entryConfigGr.u_parallel_flows)) {
                    gs.info("Found valid start for task " + taskGr.number + " with entry config " + entryConfigGr.getDisplayValue(), this.className);
                    break;
                }
                //Handle parallel flows, if any
                if (isValidStart & JSUtil.notNil(entryConfigGr.u_parallel_flows)) {
                    startDetails = this.handleParallelFlows(taskGr, entryConfigGr.u_parallel_flows);
                    taskAssignee = startDetails.task_assignee;
                    if (!newFlowTriggered)
                        newFlowTriggered = startDetails.new_flow_triggered;
                    break;
                }
            }
            while (entryConfigGr.next());
            //Assign the task to auto agent
            if (JSUtil.notNil(taskAssignee) && taskGr.getValue('assigned_to') != taskAssignee)
                this.assignTaskToUser(taskGr.getUniqueValue(), taskGr.getTableName(), taskAssignee);
            // Based on multiple field updates we might enter into this logic more than once. We need to set the flag only when an active flow is deactivated and a new one is triggered
            if (!newFlowTriggered)
                newFlowTriggered = isValidStart && isFlowDeactivated;
        }
        return {
            task_assignee: taskAssignee,
            new_flow_triggered: newFlowTriggered
        };
    },
    /**
     * @author Vaibhav Kadam
     * @description Handle parallel flows to be executed
     * @param {GlideRecord} GlideRecord of the task record
     * @param {String} List of auto agent configuration ids
     * @return {Sys Id} Sys Id of the user record
     * @example 
     * @see 
     */
    handleParallelFlows: function(taskGr, entryConfigIds) {
        var entryConfigIdArr = entryConfigIds.split(","),
            startDetails = {};
        for (var i = 0; entryConfigIdArr.length; i++) {
            startDetails = this.evaluateEntryConditions(taskGr, entryConfigIdArr[i], false, true);
        }
        return startDetails;
    },

    /**
     * @author Vaibhav Kadam
     * @description Create an execution log record
     * @param {Sys Id} Sys Id of the task record
     * @param {Sys Id} Sys Id of the auto agent entry configuration
     * @return {Sys Id} Sys Id of the flow context
     * @example 
     * @see 
     */
    _createExecutionLog: function(taskId, entryConfigId, subFlowContextId) {
        var fields = {};
        fields["u_entry_configuration"] = entryConfigId;
        fields["u_execution_status"] = AutoAgentConstants.EXECUTION_STATUS.IN_PROGRESS;
        fields["u_task"] = taskId;
        fields["u_subflow_context"] = subFlowContextId;
        var executionLogDetails = new AutoAgentDao(AutoAgentConstants.EXECUTION_TABLE).insert(fields);
    },

    /**
     * @author Vaibhav Kadam
     * @description Update case worknotes
     * @param {Glide Object} Gliderecord of the task record
     * @param {String} Name of the agent availability definition configuration
     * @return {Sys Id} Sys Id of the task record
     * @example 
     * @see 
     */
    _updateTaskWorkNotes: function(taskGr, availabilityDefinition) {
        var msgArray = [];
        msgArray.push(taskGr.getDisplayValue(AutoAgentConstants.TASK_ASSIGNEE_FIELD_MAP[taskGr.getTableName()]));
        msgArray.push(availabilityDefinition);
        var fields = {};
        fields["work_notes"] = gs.getMessage('ns.autoagent.last.assignee.unavailable.notes', msgArray);

        var executionLogDetails = new AutoAgentDao(taskGr.getTableName()).updateById(fields, taskGr.getUniqueValue());
    },

    /**
     * @author Adil J Patel
     * @description Method to get TSE's next available shift & time remaining in current shift
     * @param {String} userId
     * @return {Object} An object with 'shiftInfo' property that contains the message of TSE's availability
     * @example 
     * @see 
     */
    getAgentShiftTiming: function(userId) {
        // Error response if TSE is not available
        var errorResponse = {
            "shiftInfo": gs.getMessage('could_not_available')
        };

        try {
            if (userId == null)
                return errorResponse;
            var res = new AutoAgentIntegrationUtil().fetchAgentShiftAvailability(userId);
            var parsedResponse = JSON.parse(res);

            if (parsedResponse.status == "fail") {
                return errorResponse;
            }

            var message = "";
            var onShiftStartTime;
            var localOnShiftStartTime;
            var nextAvailableShiftStartTime;
            var localNextAvailableShiftStartTime;
            var timeRemainingInCurrentShift;
            var currentShiftDuration;

            // TSE's current shift start time
            if (parsedResponse.onShift.start_time) {
                onShiftStartTime = new GlideDateTime(parsedResponse.onShift.start_time);
                localOnShiftStartTime = onShiftStartTime.getDisplayValue();
            }

            // Time remaining in current shift
            if (parsedResponse.onShift.time_remaining) {
                timeRemainingInCurrentShift = parsedResponse.onShift.time_remaining;
            }

            // Current shift duration
            if (parsedResponse.onShift.duration_time) {
                currentShiftDuration = parsedResponse.onShift.duration_time;
            }

            // TSE's next shift start time
            if (parsedResponse.nextAvailableShift_details.start_time) {
                nextAvailableShiftStartTime = new GlideDateTime(parsedResponse.nextAvailableShift_details.start_time);
                localNextAvailableShiftStartTime = nextAvailableShiftStartTime.getDisplayValue();
            }
            if (onShiftStartTime || nextAvailableShiftStartTime) {
                if (timeRemainingInCurrentShift === currentShiftDuration && localOnShiftStartTime) { // Current shift didn't start yet
                    message = gs.getMessage('next_available') + localOnShiftStartTime;
                } else {
                    if (timeRemainingInCurrentShift > 0) {
                        var dur = new GlideDuration(timeRemainingInCurrentShift * 60 * 1000);
                        message += gs.getMessage('time_remaining') + dur.getByFormat('HH:mm');
                    }
                    if (localNextAvailableShiftStartTime) {
                        message += (message.length > 0 ? ',' : '') + gs.getMessage('next_available') + localNextAvailableShiftStartTime;
                    }
                }
            } else {
                return errorResponse;
            }
            return {
                "shiftInfo": message ? message : gs.getMessage('could_not_available')
            };
        } catch (ex) {
            gs.error(ex);
            return errorResponse;
        }

    },

    /**
     * @author Lucas Schirm
     * @description move task AA status to Active
     * @param <GlideRecord> taskGr - task that will be updated
     * @retunr void
     **/
    setAAStatusActive: function(taskGr) {
        // Check if is a valid record
        if (!taskGr || !taskGr.isValidRecord()) {
            return false;
        }

        // Only update if status is different from ACTIVE
        if (taskGr.getValue('u_aa_status') !== AutoAgentConstants.AA_STATUS.ACTIVE) {
            taskGr.setValue('u_aa_status', AutoAgentConstants.AA_STATUS.ACTIVE);
            taskGr.update();
        }
    },

    /**
     * @author Lucas Schirm
     * @description Enable AA for case record
     * @param <GlideRecord> taskGR - Case record
     * @return void
     **/
    enableAA: function(taskGr) {
        // check if case is valid
        if (!taskGr || !taskGr.isValidRecord()) {
            return false;
        }

        // Change state of the field to Inactive, case it matches to any of the configuration Start it will change it to Active.
        if (taskGr.getValue('u_aa_status') !== AutoAgentConstants.AA_STATUS.INACTIVE) {
            var previousStatus = taskGr.getDisplayValue('u_aa_status') || AutoAgentConstants.AA_STATUS.DISABLED;
            taskGr.setValue('u_aa_status', AutoAgentConstants.AA_STATUS.INACTIVE);
            taskGr.work_notes = gs.getMessage('ns.autoagent.aa_status.flow.notes', [gs.getUserDisplayName(), taskGr.getDisplayValue('u_aa_status'), previousStatus]);
            taskGr.update();
        }
    },

    /**
     * @author Lucas Schirm
     * @description Disable AA for case record
     * @param <GlideRecord> taskGr - Case that auto agent will be disabled
     * @return void
     **/
    disableAA: function(taskGr, skipAll) {
        // check if case is valid
        if (!taskGr || !taskGr.isValidRecord()) {
            return false;
        }

        if (taskGr.getValue('u_aa_status') !== AutoAgentConstants.AA_STATUS.DISABLED) {
            var message;
            var previousStatus = taskGr.getDisplayValue('u_aa_status') || AutoAgentConstants.AA_STATUS.INACTIVE;
            taskGr.setValue('u_aa_status', AutoAgentConstants.AA_STATUS.DISABLED);

            if (JSUtil.nil(skipAll)) {
                message = gs.getMessage('ns.autoagent.aa_status.flow.notes', [gs.getUserDisplayName(), taskGr.getDisplayValue('u_aa_status'), previousStatus]);
            } else {
                message = this.getBannerMessage(taskGr);
            }

            taskGr.work_notes = message;
            taskGr.update();
        }

        this.cancelFlowWhenStatusIsDisabled(taskGr);
    },

    /**
     * @author Lucas Schirm
     * @description Cancel existing flow/logs when case is disabled
     * @param <GlideRecord> caseGr - the case task sys_id
     * @param <String> reasonMessage - the worknotes message added to the task.
     **/
    cancelFlowWhenStatusIsDisabled: function(taskGr, reasonMessage) {
        // check if case is valid
        if (!taskGr || !taskGr.isValidRecord()) {
            return false;
        }

        if (taskGr.getValue('u_aa_status') !== AutoAgentConstants.AA_STATUS.DISABLED) {
            return false;
        }

        if (reasonMessage !== false) {
            reasonMessage = reasonMessage ? reasonMessage : "ns.autoagent.disabled.flow.notes";
        }

        var taskId = taskGr.getUniqueValue();

        // Reassign task to last_assignee
        var userId = taskGr.getValue(AutoAgentConstants.TASK_ASSIGNEE_FIELD_MAP[taskGr.getTableName()]);

        var triggerScript = "",
            triggerName = AutoAgentConstants.MAIN_TRIGGER_PREFIX + taskId,
            triggertime = new GlideDateTime();

        triggerScript = "new AutoAgentHandler().cancelFlowWhenAssignedToChangesFromAA('" + taskId + "', '" + userId + "', '" + reasonMessage + "');";
        return this._createTrigger(triggerScript, triggertime, triggerName);
    },

    /**
     * @author Atul Tripathi
     * @description Canceling existing flow/logs when case is assigned to agent from auto-agent.
     * @param {Sys Id} sys Id of Task 
     * @param <String> workNoteMessageId - A sys_ui_message ID that will be added to the case worknote
     * @return 
     * @example 
     * @see 
     */
    cancelFlowWhenAssignedToChangesFromAA: function(taskId, userId, workNoteMessageId) {
        if (JSUtil.nil(taskId)) {
            gs.info("Invalid cancelFlowWhenAssignedToChangesFromAA : Creating auto agent trigger for taskId: " + taskId, this.className);
            return;
        }

        var taskGr = new GlideRecord('sn_customerservice_case');
        taskGr.get(taskId);

        var executionLogGr = this.getActiveExecutionLog(taskId);
        var entryConfigId = '',
            executionLogId = '';
        if (JSUtil.notNil(executionLogGr)) {
            do {
                entryConfigId = executionLogGr.getValue('u_entry_configuration');

                //Cancel Sublfow, if any
                if (JSUtil.notNil(executionLogGr.getValue('u_subflow_context'))) {
                    gs.info('handleExitScenario: cancelling the subflow context for executionLog: ' + executionLogGr.getValue('sys_id'), this.className);
                    this.cancelSubFlow(executionLogGr.getValue('u_subflow_context'));
                }
                //Deactivate execution log record
                executionLogId = this.deactivateLog(taskId, "");
            }

            while (executionLogGr.next());

            // Only ignore message if strictly set to false
            if (workNoteMessageId !== false) {
                // If no message is passed, use default message
                var defaultMessage = 'ns.autoagent.cancel.flow.notes';
                workNoteMessageId = workNoteMessageId ? workNoteMessageId : defaultMessage;
                taskGr.work_notes = gs.getMessage(workNoteMessageId);
                taskGr.update();
            }

        }

        var exitResult = {
            auto_config_id: entryConfigId,
            execution_log_id: executionLogId
        };

        if (userId !== false) {
            this.handleTaskAssignment(taskGr, exitResult, userId);
        }
    },

    /**
     * @author Lucas Caixeta
     * @description Inactivated impacted cases when Configuration changes
     * @param <Glide Record> current - Current Auto Agent Configuration
     * @param <Object> previous - Object with previous configuration of the record
     * @param <String> previous_state - Previous state of the configuration
     * @return void
     */
    invalidateImpactedCases: function(current, previous, previous_state) {

        // Inactivate all current configuration records
        var table = current.getValue('u_table');
        var conditions = current.getValue('u_conditions');
        this.bulkUpdateAAStatus(table, conditions, AutoAgentConstants.AA_STATUS.INACTIVE);

        // If previous state of the configuration was approved, check if table or conditions change
        // If yes, inactivate cases for the previous configuration as well
        if (previous_state === 'approved') {
            var changed_fields = previous;
            var tableOrConditionChanged = false;
            for (var x in changed_fields) {
                // validate if table changed
                if (changed_fields[x].field_name === 'u_table') {
                    tableOrConditionChanged = true;
                    table = changed_fields[x].previous_value;
                }

                // validate if conditions changed.
                if (changed_fields[x].field_name === 'u_conditions') {
                    tableOrConditionChanged = true;
                    conditions = changed_fields[x].previous_value;
                }
            }

            if (tableOrConditionChanged) {
                this.bulkUpdateAAStatus(table, conditions, AutoAgentConstants.AA_STATUS.INACTIVE);
            }
        }
    },

    /***
     * @author Lucas Caixeta
     * @description Update AA for multiple records so they can be evaluated
     * @param <String> table - table name
     * @param <String> condition - the query conditions to update
     * @param <String> aa_status - the status to udpate to
     */
    bulkUpdateAAStatus: function(table, condition, aa_status) {
        var bulkUpdateGr = new GlideRecord(table);
        bulkUpdateGr.addEncodedQuery(condition);

        bulkUpdateGr.setValue('u_aa_status', aa_status);
        if (bulkUpdateGr.canWrite()) {
            return bulkUpdateGr.updateMultiple();
        }

        return false;
    },

    /**
     * @author Lucas Caixeta
     * @description Run pending scripts (ATF use only)
     * @param <Strign> sys_id - case sys_id
     **/
    ATFRunPendingScripts: function(taskId, checkIfExtraSchedule) {
        var taskGr = new GlideRecord('sn_customerservice_case');
        taskGr.get(taskId);

        if (!taskGr) {
            return;
        }

        // 		TriggerAutoAgent_4d96ea9c4779b9504832eb36736d430f
        // 		AutoAgent_CS6765320_2023-10-06 07:52:58
        //nameLIKETriggerAutoAgent_^ORnameLIKEAutoAgent_
        var grTrigger = new GlideRecord('sys_trigger');
        grTrigger.addEncodedQuery('nameLIKETriggerAutoAgent_' + taskId + '^ORnameLIKEAutoAgent_' + taskGr.number + '^state=0^ORDERBYDESCsys_created_on');
        grTrigger.query();

        while (grTrigger.next()) {
            gs.info("ATFAAstatus: run scheduled " + grTrigger.getDisplayName() + " for task " + taskGr.number);
            grTrigger.next_action.setDateNumericValue(0);
            grTrigger.update();
        }

        if (checkIfExtraSchedule) {
            gs.sleep(5000);
            return this.ATFRunPendingScripts(taskId, false);
        }

        return;
    },
    /**
     * @author Lucas Caixeta
     * @description Generate message to show on the banner / add to the work_notes
     * @param <GlideRecord> taskGr - Case Glide Record
     **/
    getBannerMessage: function(taskGr) {
        var configuration_names = this.getAllValidSkipAllScenarioNames(taskGr);

        // If any find any name, add a message
        if (configuration_names && configuration_names.length) {
            return gs.getMessage('ns.autoagent.skipall.banner', [configuration_names.join(',')]);
        }

        return;
    },

    /**
     * @author vivekrupender.gopu
     * @description Use this method to fetch any preferred message for a given case/user/flow. If no preferred message exists, default message is returned.
     * @param {String} flowKey - The unique identifier for a given AutoAgent Flow
     * @param {String} caseSysID - SysID of the case to check preferred Auto Agent response
     */
    getPreferredAutoAgentResponse: function(flowKey, caseSysID) {
		var caseGR = new GlideRecord('sn_customerservice_case');
        caseGR.get(caseSysID);
		
        var autoCloseUtils = new global.AutoCloseCaseUtil();
		var caseContactLanguage = autoCloseUtils.getCaseEmailLangPreference(caseGR);
		var comments = gs.getMessageLang('ns.autoagent.' + flowKey, caseContactLanguage);
        var query = 'u_auto_agent_flow=' + flowKey + '^u_case=' + caseSysID;

        if (caseGR.getValue('assigned_to') == AutoAgentConstants.AUTOAGENT_USER)
            query += '^u_user=' + caseGR.getValue('u_last_assignee');
        else
            query += '^u_user=' + caseGR.getValue('assigned_to');

        var AAResponseGR = new GlideRecord('u_auto_agent_response');
        AAResponseGR.addEncodedQuery(query);
        AAResponseGR.query();
        if (AAResponseGR.next())
            comments = '[code]' + AAResponseGR.getValue('u_preferred_message') + '[/code]';

        return comments;
    },

    type: 'AutoAgentHandler'
};
'''

In [15]:
tc_1 = """
var ValueNormalizationUtil = Class.create();
ValueNormalizationUtil.prototype = {
	initialize: function (companyId) {
		this.cacheClass = new global.SuccessPlatformCacheMan("normalization");
		this.normUnitCacheKey = "normUnitKey";
		this.companyId = companyId ? companyId.toString() : "";
		this.aclUtil = new x_snc_cusp.AclUtil();
	},

	//Get Normalized metric values based on the companyId, SHG, Instance and Product parameters
	getNormalizationMetricValues: function (shgId, instanceId, productId) {
		var unitDef = this._getUnitDefinition(productId);
		if (unitDef === null){
			return null;
		}
		// Generate a cache key based on the input parameters
		var cacheKey = this._getCacheKey(shgId, instanceId, unitDef.Id);

		// Check if the result is already in the cache
		var cachedResult = this._getCache(cacheKey);

		if (cachedResult !== undefined && cachedResult !== null) {
			// Return the cached result if it exists
			return cachedResult;
		} else {
			// If the result is not in the cache, fetch it and store it in the cache
			var result = this._getNormalizationData(shgId, instanceId, productId);
			if (result == null) {
				return null;
			}
			//Save the records to the cache only when there are normalization records present
			if(result && result.normalizationData && result.normalizationData.length > 0){
				this._putInCache(cacheKey, result);
			}
			return result;
		}
	},

	//Update normalization metrics
	updateNormalizationMetricValues: function (requestData) {
		var shgId = requestData.shgId;
		var instanceId = requestData.instanceId;
		var productId = requestData.productId;
		var unitTitle = requestData.normUnitTitle;
		var normalizationData = requestData.normalizationData;

		if (!normalizationData)
			throw new Error("Required data is not present in the request body");

		var unitDef = this._getUnitDefinition(productId);
		if (unitDef === null){
			throw new Error("There is no unit definition for this product " + productId);
		}
		var normInstanceId = this._updateNormalizationInstance(shgId, instanceId, productId, unitTitle, unitDef);

		for (var i = 0; i < normalizationData.length; i++) {

			var year = normalizationData[i].year;
			var quarters = normalizationData[i].quartersData;

			for (var j in quarters) {
				var normDataGr = new GlideRecordSecure(ImpactBizKPITables.NORMALIZATION_DATA);

				if (quarters[j].sysId) {
					//Update existing record
					normDataGr.addQuery("sys_id", quarters[j].sysId);
					normDataGr.query();
					if (normDataGr.next()) {
						if (quarters[j].delete)
							normDataGr.deleteRecord();
						else {
							normDataGr.setValue("value", (quarters[j].value === "" || quarters[j].value === null || isNaN(+quarters[j].value)) ? 'NULL' : quarters[j].value);
							//If the scan data is present and not matching the user entered data then update is_override flag to true
							//or if scan data is not present then mark is_override flag to true
							if((!quarters[j].scanData && quarters[j].value != "") || (quarters[j].scanData && quarters[j].value != quarters[j].scanData)){
								normDataGr.is_override = true;
							}else if(quarters[j].scanData && quarters[j].value == quarters[j].scanData){
								normDataGr.is_override = false;
							}
							normDataGr.update();
						}
					}
				} else if (!quarters[j].delete) {
					//Create new record
					normDataGr.initialize();
					normDataGr.value = (quarters[j].value === "" || quarters[j].value === null || isNaN(+quarters[j].value)) ? 'NULL' : quarters[j].value;
					normDataGr.normalization_unit_instance = normInstanceId;
					normDataGr.quarter = this._getQuarterIdByName(quarters[j].title);
					normDataGr.is_override = true;
					normDataGr.insert();
				}
			}
		}
		return true;
	},

	getNormalizationApplicableOutcomes: function (productSysId, shg_id) {
		var objectivesMap = {};
        var objectives = [];
		var outcomeSysIDMap={};

		var outcomeInstGR = new GlideRecordSecure(ImpactBizKPITables.OUTCOME_INSTANCE);
		outcomeInstGR.addQuery("objective.company.sys_id",this.companyId);
		outcomeInstGR.addQuery("objective.stakeholder_group",shg_id);
		outcomeInstGR.addQuery("objective.user_defined",false);
		outcomeInstGR.addQuery("outcome.product","CONTAINS",productSysId);
		outcomeInstGR.addQuery("outcome.normalizable",true);
		outcomeInstGR.orderBy('objective.title');
		outcomeInstGR.query();
		while(outcomeInstGR.next()){
			outcomeSysIDMap[outcomeInstGR.outcome.toString()] ="";
		}

        var outcomeGr = new GlideRecordSecure(ImpactBizKPITables.VALUE_OUTCOME);
        outcomeGr.addEncodedQuery("productLIKE"+productSysId);
        outcomeGr.addQuery("normalizable", true);
        outcomeGr.query();
		while (outcomeGr.next()) {
            var outcome = {};
            //outcome.sysId = outcomeGr.getUniqueValue().toString();
			//outcome.products = outcomeGr.getDisplayValue("product");
            outcome.title = outcomeGr.getDisplayValue("title");
            outcome.metric = this._getMetric(outcomeGr.getUniqueValue());
			if (outcomeSysIDMap.hasOwnProperty(outcomeGr.getUniqueValue().toString())){
				outcome.inValueBlueprint = true;       
			}
			else {
				outcome.inValueBlueprint = false;
			}
            if (!objectivesMap[outcomeGr.objective]) {
                objectivesMap[outcomeGr.objective] = {
                    title: outcomeGr.objective.title.toString(),
                    outcomes: []
                };
            }
            objectivesMap[outcomeGr.objective].outcomes.push(outcome);
        }
        for (var key in objectivesMap) {
            objectives.push(objectivesMap[key]);
        }

        return objectives;
		
	},


	getNormalizationProducts: function (shgId) {

		var prodArr = [];
		var prodObj = {};
		var productDisplayValue = "";
		var productSysID = "";

		var outcomeInstDataGR = new GlideRecordSecure(ImpactBizKPITables.OUTCOME_INSTANCE);
		outcomeInstDataGR.addQuery("objective.company", this.companyId);
		if (!gs.nil(shgId)) {
			outcomeInstDataGR.addQuery("objective.stakeholder_group", shgId);
		}
		outcomeInstDataGR.addQuery("outcome.normalizable", true);
		outcomeInstDataGR.query();
		while (outcomeInstDataGR.next()) {
			productDisplayValue = '';
			productSysID = '';
			if (outcomeInstDataGR.getElement('user_defined')) {
				if (!gs.nil(outcomeInstDataGR.objective.products)) {
					productDisplayValue = outcomeInstDataGR.getDisplayValue('objective.products').toString().trim();
					productSysID = outcomeInstDataGR.getElement('objective.products').toString().trim();
				}
			} else {
				if (!gs.nil(outcomeInstDataGR.outcome.product)) {
					productDisplayValue = outcomeInstDataGR.getDisplayValue('outcome.product').toString().trim();
					productSysID = outcomeInstDataGR.getElement('outcome.product').toString().trim();
				}
			}
			if (!gs.nil(productDisplayValue) && !gs.nil(productSysID)) {
				if (productDisplayValue.indexOf(',') > 0) {
					var listOfProductNames = productDisplayValue.split(',');
					var listOfProductSysIDs = productSysID.split(',');
					for (var i = 0; i < listOfProductNames.length; i++) {
						productDisplayValue = listOfProductNames[i].toString().trim();
						productSysID = listOfProductSysIDs[i].toString().trim();
						if (!(productDisplayValue in prodObj)) {
							prodArr.push({
								"label": productDisplayValue,
								"value": productSysID,
								"selected": false
							});
							prodObj[productDisplayValue] = productSysID;
						}
					}
				} else {
					if (!(productDisplayValue in prodObj)) {
						prodArr.push({
							"label": productDisplayValue,
							"value": productSysID,
							"selected": false
						});
						prodObj[productDisplayValue] = productSysID;
					}
				}
			}
		}
		return prodArr;
	},

	getNormalizationInstance: function (shgId) {
		var instanceArray = [];
		var instanceGR;
		if (!gs.nil(shgId)) {
			// from dept  
			instanceGR = new GlideRecordSecure("u_instance_dept_mapping_m2m");
			instanceGR.addQuery("u_company", this.companyId);
			instanceGR.addQuery("u_account_department", shgId);
			instanceGR.query();
			while (instanceGR.next()) {
				instanceArray.push({
					"label": instanceGR.u_instance.getDisplayValue(),
					"selected": false,
					"value": instanceGR.u_instance.toString()
				});
			}
		} else {
			// cmdb_ci_service
			instanceGR = new GlideRecordSecure("cmdb_ci_service");
			instanceGR.addQuery("company", this.companyId);
			instanceGR.addQuery("used_for", "Production");
			instanceGR.query();
			while (instanceGR.next()) {
				instanceArray.push({
					"label": instanceGR.snc_instance_name.toString(),
					"selected": false,
					"value": instanceGR.getUniqueValue()
				});
			}
		}

		return instanceArray;
	},

	getNormalizationFactor: function (shgId, productId, actualsStartQtr, baseLineStartQtr) {

		var result = {
			instances: [],
			normFactorListByInstance: {}
		};
		// Define a regular expression pattern for "Year Quarter" - "2022 Q4"
		var pattern = /^\d{4}\sQ[1-4]$/;

		if (!pattern.test(actualsStartQtr) || !pattern.test(baseLineStartQtr))
			return result;

		var actualQuarters = this._getNextFourQuarters(actualsStartQtr);
		var baseLineQuarters = this._getNextFourQuarters(baseLineStartQtr);

		var unitDef = this._getUnitDefinition(productId);
		if (unitDef == null) {
			return result;
		}
		var unitDefId = unitDef.Id;

		//Iterate through the grouped instances and pass the instanceId
		var normalizationUnitInstanceGr = new GlideRecordSecure(ImpactBizKPITables.NORMALIZATION_UNIT_INSTANCE);
		normalizationUnitInstanceGr.addQuery("company", this.companyId);
		normalizationUnitInstanceGr.addQuery("normalization_unit", unitDefId);

		if (shgId)
			normalizationUnitInstanceGr.addQuery("stakeholder_group", shgId);

		normalizationUnitInstanceGr.query();

		while (normalizationUnitInstanceGr.next()) {
			var instanceId = normalizationUnitInstanceGr.getValue("instance");

			//If there's no instance Id available then ignore that record
			if (!instanceId)
				continue;

			var instanceTitle = normalizationUnitInstanceGr.instance.snc_instance_name.toString();
			var actualsTotal = 0;
			var actualsCounter = 0;
			var baseLineTotal = 0;
			var baseLineCounter = 0;

			var normalizationValues = this.getNormalizationMetricValues(shgId, instanceId, productId);
			var normalizationData = normalizationValues ? normalizationValues.normalizationData : [];

			for (var i = 0; i < normalizationData.length; i++) {

				var yearData = normalizationData[i];
				var quartersData = yearData.quartersData;

				for (var j = 0; j < quartersData.length; j++) {

					var quarter = quartersData[j];
					var quarterTitle = quarter.title;

					if (actualQuarters.indexOf(quarterTitle) !== -1 && quarter.value !== "" && !isNaN(quarter.value)) {

						actualsTotal += parseFloat(quarter.value);
						actualsCounter++;

					} else if (baseLineQuarters.indexOf(quarterTitle) !== -1 && quarter.value !== "" && !isNaN(quarter.value)) {

						baseLineTotal += parseFloat(quarter.value);
						baseLineCounter++;
					}					
				}			
			}

			//Only if there's data present for atleast one quarter in baseline period and actuals period, add the
			//normalization factor in the response
			if (actualsCounter && baseLineCounter) {
				var normObj = {
					normalizationFactor: null,
					baseLineAvg: parseFloat((baseLineTotal / baseLineCounter).toFixed(2)),
					actualsAvg: parseFloat((actualsTotal / actualsCounter).toFixed(2)),
					instanceId: instanceId
				};
				normObj.normalizationFactor = parseFloat((normObj.actualsAvg / normObj.baseLineAvg).toFixed(2));
				result.normFactorListByInstance[instanceId] = normObj;

				result.instances.push({
					value: instanceId,
					label: instanceTitle,
					selected: false
				});
			}			
		}		
		return result;
	},

	clearNormalizationUnitCache: function () {
		var cachedResult = this._getCache(this.normUnitCacheKey);
		if (cachedResult !== undefined && cachedResult !== null) {
			this._removeCache(this.normUnitCacheKey);	
		}	
	},

	clearNormalizationUnitDataCache: function (shgId, instanceId, unitDefId) {
		var cacheKey = this._getCacheKey(shgId, instanceId, unitDefId);
		var cachedResult = this._getCache(cacheKey);
		if (cachedResult !== undefined && cachedResult !== null) {
			this._removeCache(cacheKey);	
		}	
	},

	_getNormalizationData: function (shgId, instanceId, productId) {
		//Get the unit definition id using the product ID and apply it for the normalization unit instance so in future if we have multiple units it would still be working
		var unitDef = this._getUnitDefinition(productId);
		if (unitDef == null) {
			return null;
		}
		var unitDefId = unitDef.Id;

		//Initialize the result object
		var result = {
			normUnitActualTitle: unitDef.name,
			shgId: shgId,
			instanceId: instanceId,
			productId: productId,
			customizable: unitDef.isCustomizable,
			normalizationData: []
		};

		var normalizationInstanceId = "";
		var yearsData = {};

		var normalizationUnitInstanceGr = new GlideRecordSecure(ImpactBizKPITables.NORMALIZATION_UNIT_INSTANCE);
		normalizationUnitInstanceGr.addQuery("company", this.companyId);
		normalizationUnitInstanceGr.addQuery("normalization_unit", unitDefId);

		if (shgId)
			normalizationUnitInstanceGr.addQuery("stakeholder_group", shgId);

		if (instanceId)
			normalizationUnitInstanceGr.addQuery("instance", instanceId);

		normalizationUnitInstanceGr.query();

		if (normalizationUnitInstanceGr.next()) {
			normalizationInstanceId = normalizationUnitInstanceGr.getUniqueValue();
			result.normInstanceId = normalizationInstanceId;
			result.normUnitTitle = normalizationUnitInstanceGr.title.toString();
		}

		if (!normalizationInstanceId)
			return result; //Zero state

		var normalizationDataGr = new GlideRecordSecure(ImpactBizKPITables.NORMALIZATION_DATA);
		normalizationDataGr.addQuery("normalization_unit_instance", normalizationInstanceId);
		normalizationDataGr.orderBy("quarter");
		normalizationDataGr.query();

		while (normalizationDataGr.next()) {
			var year = this._getYearFromQuarterRecord(normalizationDataGr.quarter);
			var quarterData = {
				title: normalizationDataGr.quarter.title.toString(),
				scanData: gs.nil(normalizationDataGr.scan_value) ? '' : parseInt(normalizationDataGr.scan_value),
				value: gs.nil(normalizationDataGr.value) ? '' : parseInt(normalizationDataGr.value),
				sysId: normalizationDataGr.getUniqueValue(),
				isOverride: global.JSUtil.toBoolean(normalizationDataGr.is_override.toString())
			};

			if (!yearsData[year])
				yearsData[year] = [];

			yearsData[year].push(quarterData);
		}

		for (var key in yearsData) {
			result.normalizationData.push({
				year: key,
				quartersData: yearsData[key]
			});
		}

		return result;

	},

	_updateNormalizationInstance: function (shgId, instanceId, productId, title, unitDef) {
		var isUnitCustomizable = unitDef.isCustomizable;

		var normalizationUnitInstanceGr = new GlideRecordSecure(ImpactBizKPITables.NORMALIZATION_UNIT_INSTANCE);
		normalizationUnitInstanceGr.addQuery("company", this.companyId);
		normalizationUnitInstanceGr.addQuery("normalization_unit", unitDef.Id);

		if (shgId)
			normalizationUnitInstanceGr.addQuery("stakeholder_group", shgId);
		else
			normalizationUnitInstanceGr.addQuery("stakeholder_group", "");

		if (instanceId)
			normalizationUnitInstanceGr.addQuery("instance", instanceId);
		else
			normalizationUnitInstanceGr.addQuery("instance", "");

		normalizationUnitInstanceGr.query();

		if (!normalizationUnitInstanceGr.next()) {

			normalizationUnitInstanceGr.initialize();
			normalizationUnitInstanceGr.company = this.companyId;
			normalizationUnitInstanceGr.normalization_unit = unitDef.Id;

			if (isUnitCustomizable)
				normalizationUnitInstanceGr.title = title;
			else
				normalizationUnitInstanceGr.title = unitDef.name;

			if (shgId)
				normalizationUnitInstanceGr.stakeholder_group = shgId;

			if (instanceId)
				normalizationUnitInstanceGr.instance = instanceId;

			var normalizedInstanceSysId = normalizationUnitInstanceGr.insert();

			//Just to skip healthscan finding have to add this statement to check for null
			if(normalizedInstanceSysId != null)
				return normalizedInstanceSysId;
			else
				return null;

		} else {

			if (isUnitCustomizable) {

				normalizationUnitInstanceGr.title = title;
				return normalizationUnitInstanceGr.update();

			} else {

				return normalizationUnitInstanceGr.getUniqueValue();

			}

		}

	},


	_getCacheKey: function (shgId, instanceId, unitId) {

		var cacheKey = this.companyId;

		if (shgId)
			cacheKey += "-" + shgId;
		if (instanceId)
			cacheKey += "-" + instanceId;
		if (unitId)
			cacheKey += "-" + unitId;

		return cacheKey;

	},

	_getCache: function (cacheKey) {

		var normalizedValues = null;

		try {
			normalizedValues = this.cacheClass.get(cacheKey);
		} catch (ex) {
			throw ex;
		}

		return normalizedValues;
	},

	_putInCache: function (cacheKey, cacheData) {

		var expiration = new GlideDateTime();
		var duration = gs.getProperty("x_snc_octane_bvmp.biz.kpi.normalization.expiration_sec") || 86400;
		expiration.addSeconds(duration);
		this.cacheClass.put(cacheKey, cacheData, expiration);

	},

	_removeCache: function (cacheKey) {

		try {
			this.cacheClass.remove(cacheKey);
		} catch (ex) {
			throw ex;
		}
	},

	_getYearFromQuarterRecord: function (quarterGr) {

		var startGlideDate = new GlideDate();
		startGlideDate.setValue(quarterGr.start_date.toString());
		var year = startGlideDate.getYearUTC();
		return year;

	},

	_getQuarterIdByName: function (quarterString) {
		var quarterSysId = "";
		var quarterGr = new GlideRecord(ImpactBizKPITables.TIME_PERIOD);
		quarterGr.addQuery("title", quarterString);
		quarterGr.query();

		if (quarterGr.next()) {
			quarterSysId = quarterGr.getUniqueValue();
		}
		return quarterSysId;
	},

	_getMetric: function (outcomeGr) {
		var metric = "";
		var metricGr = new GlideRecordSecure(ImpactBizKPITables.VALUE_OUTCOME_METRIC_M2M);
		metricGr.addQuery("outcome", outcomeGr);
		metricGr.addQuery("metric_type", "primary");
		metricGr.addQuery("is_baseline", true);
		metricGr.query();

		if (metricGr.next()) {
			metric = metricGr.metric.title.toString();
		}
		return metric;
	},

	_getUnitDefinition: function (productId) {
		var unitGr = {}, units = [], unit = {}, defaultUnits = [];
		// Check if the result is already in the cache
		var cachedResult = this._getCache(this.normUnitCacheKey);
		if (cachedResult !== undefined && cachedResult !== null) {
			// Return the cached result if it exists
			units = cachedResult;
		} else {
			unitGr = new GlideRecordSecure(ImpactBizKPITables.NORMALIZATION_UNITS);
			//unitGr.addEncodedQuery("productsLIKE" + productId + "^is_default=true");
			unitGr.addEncodedQuery("is_default=true");
			unitGr.query();

			while (unitGr.next()) {
				unit = {};
				unit.Id = unitGr.getUniqueValue();
				unit.name = unitGr.getDisplayValue("name");
				unit.products = unitGr.products.toString();
				unit.isCustomizable = global.JSUtil.toBoolean(unitGr.is_customizable.toString());
				units.push(unit);
			}
			this._putInCache(this.normUnitCacheKey, units);
		}
		defaultUnits = units.filter(function(item) {
            return item.products.includes(productId);												
        });
		if (defaultUnits.length > 0){
			return defaultUnits[0];																
		}
		else{
			return null;
		}
	},

	_getNextFourQuarters: function (startQtr) {
		// Parse the input quarter into year and quarter parts
		var yearWithQuarter = startQtr.split(' Q');
		var year = parseInt(yearWithQuarter[0], 10);
		var quarter = parseInt(yearWithQuarter[1], 10);

		var quarters = [];

		for (var i = 0; i < 4; i++) {
			quarters.push(year + ' Q' + quarter);
			quarter++;
			if (quarter > 4) {
				quarter = 1;
				year++;
			}
		}

		return quarters;
	},

	type: 'ValueNormalizationUtil'
};
"""

In [16]:
tc_2 = """
var ImpactDepartmentSupportService = Class.create();
ImpactDepartmentSupportService.prototype = {
    initialize: function() {
        this.logger = new GSLog(ImpactDepartmentSupportConstants.TRACE_PROPERTIES.DEPARTMENT_SUPPORT_ENTITLEMENT_RECEIVED, this.type);
    },

    createDefaultDepartment: function(params) {

        this.logger.logDebug('Start - create default department record for account: ' + params.accountName + ' - ' + 'cmid:' + params.cmid);

        var departmentGR = new GlideRecord(ImpactDepartmentSupportConstants.TABLES.DEPARTMENT);
        departmentGR.initialize();
        departmentGR.addQuery('u_department', params.accountName);
        departmentGR.addQuery('u_company', params.accountSysId);
        departmentGR.query();
        departmentGR.next();
        departmentGR.setValue('u_company', params.accountSysId);
        departmentGR.setValue('u_department', params.accountName);
        departmentGR.setValue('sys_domain', params.accountDomainSysId);
        var departmentSysId = departmentGR.update();

        this.logger.logDebug('End - create default department record for account for account: ' + params.accountName + ' - ' + 'cmid:' + params.cmid);


        return departmentSysId;
    },

	assignDepartmentSquadAndCustomers: function(params) {

		if (params.defaultDepartmentSysId && params.accountSysId) {
			new ManageBusinessUnitHelperService().addSquadAndCustomerMembersToDepartment(params.defaultDepartmentSysId, params.accountSysId);
			var list = new ImpactDepartmentSupportUtil().getAllProdInstances(params.accountSysId);
			if(list.length > 1)
			gs.info("MultiInstance:: assignDepartmentSquadAndCustomers not able to add instances as there are more prod instances"+params.accountName);

			if (list.length == 1) {
				gs.info("MultiInstance:: assignDepartmentSquadAndCustomers able to add 1 prod instance in instance mapping table"+params.accountName);
				var pckg = new ImpactDepartmentSupportUtil().getCompanyPackage(params.accountSysId);
				var coverage = '';
				var instanceId = list[0].sysId;

				var subProdList = new ImpactDepartmentSupportUtil().getAllSubProdInstances(params.accountSysId);

				var subProdSysIdList = subProdList.map(function(subProd) {
					return subProd.sysId;
				});
				new ManageBusinessUnitHelperService().addInstance(params.accountSysId, params.defaultDepartmentSysId, instanceId, subProdSysIdList);

			}
		}
	},


    

    updateOptInFlag: function(params) {
        this.logger.logDebug('Start - update opt-in flag for account: ' + params.accountName + ' - ' + 'cmid:' + params.cmid);

        var companyGR = new GlideRecord('core_company');
        companyGR.addQuery('sys_id', params.accountSysId);
        companyGR.query();
        if (companyGR.next()) {
            companyGR.setValue('x_snc_account_cust_department_support_opt_in', true);
            companyGR.update();
        }

        this.logger.logDebug('End - update opt-in flag for account: ' + params.accountName + ' - ' + 'cmid:' + params.cmid);
    },


    _createDataMovementJob: function(tableName, params) {
        var scheduleAt = new GlideDateTime();
        scheduleAt.add(10000);
        var nm = tableName + ' - Data Movement to Common Domain';
        var st = new GlideRecord('sys_trigger');
        st.initialize();
        st.name = nm;
        st.next_action = scheduleAt;
        st.job_id.setDisplayValue('RunScriptJob');
        st.trigger_type = '0';
        st.state = '0';
        st.upgrade_safe = true; 
		st.script = "(new global.ImpactDepartmentSupportService()).mapDefaultSHGToAllEntities('" + tableName + "', '" + params + "');";
        st.insert();
    },
	
    _mapDefaultSHGToAllEntitiesJob: function(companySysId, defaultDeptSysId) {
        var scheduleAt = new GlideDateTime();
        scheduleAt.add(1000);
        var nm = 'mapDefaultSHGToAllEntities';
        var st = new GlideRecord('sys_trigger');
        st.initialize();
        st.name = nm;
        st.next_action = scheduleAt;
        st.job_id.setDisplayValue('RunScriptJob');
        st.trigger_type = '0';
        st.state = '0';
        st.upgrade_safe = true;
		st.script = "(new global.ImpactDepartmentSupportService()).mapDefaultSHGToAllEntities('" + companySysId + "', '" + defaultDeptSysId + "');";
        st.insert();
	},
    mapDefaultSHGToAllEntities: function(companySysId, defaultDeptSysId) {
		
		var companyQuery = "company="+companySysId+"^stakeholder_groupISEMPTY";
		
        var tables = [
			{
                "table": "u_octane_attachment_unlink_data",
                "query": "",
                "companyColumn": "u_company",
                "shgColumn": "u_stakeholder_group"
            },
            {
                "table": "x_snc_octane_bvmp_suggested_instance",//no
                "query": "",
                "companyColumn": "company",
                "shgColumn": "stakeholder_group"
            },
            {
                "table": "x_snc_octane_bvmp_company_related_file",
                "query": "",
                "companyColumn": "company",
                "shgColumn": "stakeholder_group"
            },
            {
                "table": "x_snc_octane_bvmp_value_assessment",
                "query": "",
                "companyColumn": "company",
                "shgColumn": "stakeholder_group"
            },
            {
                "table": "x_snc_octane_bvmp_business_objective_data",//no
                "query": "",
                "companyColumn": "company",
                "shgColumn": "stakeholder_group"
            },
            {
                "table": "x_snc_octi_conversation",
                "query": "",
                "companyColumn": "company",
                "shgColumn": "stakeholder_group"
            },
            {
                "table": "x_snc_cusp_activity_base",
                "query": "activity_type=a7cb8f141bef4050882ba9bd2a4bcbfa^achievementISEMPTY^NQactivity_type!=a7cb8f141bef4050882ba9bd2a4bcbfa^parent.activity_type=a7cb8f141bef4050882ba9bd2a4bcbfa^parent.achievementISEMPTY^"+companyQuery+"^NQparentISEMPTY^activity_type!=a7cb8f141bef4050882ba9bd2a4bcbfa^"+companyQuery+"^NQactivity_type!=a7cb8f141bef4050882ba9bd2a4bcbfa^parent.activity_type!=a7cb8f141bef4050882ba9bd2a4bcbfa^parent.achievementISEMPTY"+companyQuery,
                "companyColumn": "company",
                "shgColumn": "stakeholder_group",
				"shouldSetWorkFlowFalse": true
            }
        ];

        if (defaultDeptSysId) {
            for (var i = 0; i < tables.length; i++) {
				try{
					var t = tables[i];
					
					if (!t.companyColumn) {
						t.companyColumn = "company";
					}
					if (!t.shgColumn) {
						t.shgColumn = "stakeholder_group";
					}
					var tableGr = new GlideRecord(t.table);
					var defaultQuery = t.companyColumn + "=" + companySysId + "^" + t.shgColumn + "ISEMPTY";
					
					if(t.query){
						defaultQuery = defaultQuery + "^" + t.query;
					}
					
					tableGr.addEncodedQuery(defaultQuery);
					if(t.shouldSetWorkFlowFalse){
						tableGr.setWorkflow(false);
					}
					tableGr.query();
					while (tableGr.next()) {
						tableGr.setValue(t.shgColumn, defaultDeptSysId);
						tableGr.update();
					}
				} catch(e) {
					gs.error("Error while updating Default SHG in table " + tables[i] + " -> " + e.message);
				}
            }
        }
    },

    type: 'ImpactDepartmentSupportService'
};
"""

In [17]:
tc_3 = """
var ValueOutcome = Class.create();
ValueOutcome.TABLE_BUSINESS_OUTCOME = 'x_snc_octane_bvmp_value_outcome';
ValueOutcome.TABLE_OBJECTIVE_OUTCOME_INSTANCE = 'x_snc_octane_bvmp_objective_outcome_instance_data';
ValueOutcome.TABLE_OUTCOME_METRIC_INSTANCE = 'x_snc_octane_bvmp_outcome_metric_instance_data';
ValueOutcome.TABLE_OUTCOME_METRIC = 'x_snc_octane_bvmp_value_outcome_metric_m2m';
ValueOutcome.TABLE_CAPABILITY_APPLICATION = 'x_snc_cusp_customer_application';

ValueOutcome.prototype = {
    initialize: function() {},

    /**
     * avishek.dey, SFSTRY0026404, Oct 22 2021
     * Method to create a business value outcome with the sent data.
     * @param data JSON - Create business objective fields
     */
    createValueOutcome: function(data) {
        var outcomeGr = new GlideRecord(ValueOutcome.TABLE_BUSINESS_OUTCOME),
            recordCreated = false,
            sysId = "";

        if (data && outcomeGr.canCreate() && this._validateValueOutcomeData(data)) {
            outcomeGr.initialize();
            outcomeGr.setValue("title", data.title);
            outcomeGr.setValue("description", data.description);
            outcomeGr.setValue("driver", data.driver);
            outcomeGr.setValue("product", data.product);
            outcomeGr.setValue("order", data.order);
            outcomeGr.setValue("source", data.source);
            outcomeGr.setValue("source_id", data.source_id);
            outcomeGr.setValue("state", data.state);
            outcomeGr.setValue("type", data.type);
            outcomeGr.setValue("base_version", data.baseVersion);
            outcomeGr.setValue("version", data.version);
            outcomeGr.setValue("is_latest", true);
            sysId = outcomeGr.insert();
            gs.info("ValueOutcome::createValueOutcome::Value Outcome created");
            recordCreated = true;
        } else {
            gs.info("ValueOutcome::createValueOutcome::Cannot create Value Outcome without valid inputs");
        }

        return {
            recordCreated: recordCreated,
            sysId: sysId
        };
    },

    /**
     * Method to validate the mandatory or conditional values for business value outcomes
     * avishek.dey, SFSTRY0026404, Oct 22 2021
     * @param {object} data - Business Value Outcome fields
     * @returns boolean
     */
    _validateValueOutcomeData: function(data) {
        var isValid = false;
        if (data.title && data.driver && data.source && data.product) {
            isValid = true;
        }
        return isValid;
    },

    /**
     * Check if the system is configured to support multiple metrics with a single value outcome.
     * avishek.dey, SFSTRY0026404, Oct 22 2021
     * @returns boolean 
     */
    canMapMultipleMetricsToSingleOutcome: function() {
        return gs.getProperty("x_snc_octane_bvmp.enable_outcome_multiple_metrics") === "true";
    },

    /**
     * Query the outcome - metric mapping table to find out metrics related to an outcome.
     * @param {string} outcomeId Sys id of the value outcome.
     * @returns array
     */
    getMappedMetricsForOutcome: function(outcomeId) {
        if (!outcomeId) {
            gs.info("ValueOutcome::getMappedMetricsForOutcome::Outcome id cannot be empty");
            return;
        }
        var grOutcomeMetricGr = new GlideRecord(ValueOutcome.TABLE_OUTCOME_METRIC),
            result = [];

        grOutcomeMetricGr.addEncodedQuery("outcome=" + outcomeId);
        grOutcomeMetricGr.query();
        if (grOutcomeMetricGr.canRead() && grOutcomeMetricGr.next()) {
            result.push(grOutcomeMetricGr.getValue("metric"));
        }

        return result;
    },

    /**
     * Create a record in the outcome metric m2m table.
     * @param {string} outcomeId Outcome id
     * @param {string} metricId Metric Id
     */
    createMetricForOutcome: function(outcomeId, metricId) {
        if (!outcomeId || !metricId) {
            gs.info("ValueOutcome::createMetricsForOutcome::Invalid inputs. Outcome id and metric id cannot be empty");
            return;
        }

        var grOutcomeMetricGr = new GlideRecord(ValueOutcome.TABLE_OUTCOME_METRIC);
        if (grOutcomeMetricGr.canCreate()) {
            grOutcomeMetricGr.initialize();
            grOutcomeMetricGr.setValue("outcome", outcomeId);
            grOutcomeMetricGr.setValue("metric", metricId);
            grOutcomeMetricGr.insert();
        }
    },

    /**
     * Query the outcome - metric instance table to find out metrics related to an outcome.
     * @param {string} outcomeId Sys id of the value outcome instance
     * @returns array
     */
    getOutcomeMetricInstancesForOutcome: function(outcomeId) {
        if (!outcomeId) {
            gs.info("ValueOutcome::getOutcomeMetricInstancesForOutcome::Outcome id cannot be empty");
            return;
        }
        var grOutcomeMetricGr = new GlideRecord(ValueOutcome.TABLE_OUTCOME_METRIC_INSTANCE),
            result = [];

        grOutcomeMetricGr.addEncodedQuery("outcome=" + outcomeId);
        grOutcomeMetricGr.query();
        if (grOutcomeMetricGr.canRead() && grOutcomeMetricGr.next()) {
            result.push(grOutcomeMetricGr.getValue("metric"));
        }

        return result;
    },

    /**
     * Create objective outcome instance data based on the driver for the outcome.
     * @param {object} data Contains the objective and the driver id.
     * @returns 
     */
    createObjectiveOutcomeInstanceData: function(data) {
        if (!data) {
            gs.info("ValueOutcome::createObjectiveOutcomeInstanceData::Invalid input. Cannot create outcome instance data");
            return;
        }

        // Get the associated outcomes
        var grOutcomes = new GlideRecord(ValueOutcome.TABLE_BUSINESS_OUTCOME);
        grOutcomes.addEncodedQuery("driver=" + data.driver);
        grOutcomes.query();
        while (grOutcomes.canRead() && grOutcomes.next()) {
            var outcomeId = grOutcomes.getValue("sys_id");
            var grObjOutcomeInstance = new GlideRecord(ValueOutcome.TABLE_OBJECTIVE_OUTCOME_INSTANCE);
            if (grObjOutcomeInstance.canCreate()) {
                grObjOutcomeInstance.initialize();
                grObjOutcomeInstance.setValue("outcome", outcomeId);
                grObjOutcomeInstance.setValue("objective", data.objSysId);
                grObjOutcomeInstance.insert();
            }
        }
    },

    /**
     * Create outcome metric instance data based on the metrics for the outcome.
     * @param {object} data Contains the Outcome and the ObjectiveOutcome record id.
     * @returns 
     */
    createOutcomeMetricInstanceData: function(data) {
        if (!data) {
            gs.info("ValueOutcome::createOutcomeMetricInstanceData::Invalid input. Cannot create outcome metric instance data");
            return;
        }
        var query = 'outcome=' + data.outcome + '^metric_type=primary^is_baseline=true';
        // Get the associated outcomes
        var grOutcomeMetric = new GlideRecord(ValueOutcome.TABLE_OUTCOME_METRIC); //x_snc_octane_bvmp_value_outcome_metric_m2m
        grOutcomeMetric.addEncodedQuery(query);
        grOutcomeMetric.query();
        if (grOutcomeMetric.canRead() && grOutcomeMetric.next()) {
            var metric = grOutcomeMetric.getValue("metric");
            var grMetricInstance = new GlideRecord(ValueOutcome.TABLE_OUTCOME_METRIC_INSTANCE); //x_snc_octane_bvmp_outcome_metric_instance_data
            grMetricInstance.addQuery("metric", metric);
            grMetricInstance.addQuery("outcome", data.objOutcomeSysId);
            grMetricInstance.query();
            if (grMetricInstance.hasNext()) {
                //record already exists. do nothing
                return;
            }
            grMetricInstance = new GlideRecord(ValueOutcome.TABLE_OUTCOME_METRIC_INSTANCE); //x_snc_octane_bvmp_outcome_metric_instance_data
            if (grMetricInstance.canCreate()) {
                grMetricInstance.initialize();
                grMetricInstance.setValue("metric", metric);
                grMetricInstance.setValue("outcome", data.objOutcomeSysId);
                grMetricInstance.setValue('title', grOutcomeMetric.metric.getDisplayValue());
                grMetricInstance.insert();
            }
        }
    },

    /**
     * When an outcome is updated, the attached metrics should be copied over as well on the m2m table.
     * @param {string} srcOutcomeId Source Outcome Id for fetching metrics
     * @param {string} targetOutcomeId Target Outcome Id for creating metrics
     */
    copyMetricOutcomeInstanceForOutcome: function(srcOutcomeId, targetOutcomeId) {
        if (!srcOutcomeId || !targetOutcomeId) {
            gs.info("ValueOutcome::copyMetricOutcomeInstance:: Source and target outcome ids cannot be empty");
            return;
        }
        // get attached metric(s)
        var metrics = this.getMappedMetricsForOutcome(srcOutcomeId);
        for (var x = 0; x < metrics.length; x++) {
            this.createMetricForOutcome(targetOutcomeId, metrics[x]);
        }
    },


    /**
     * Author: Saikiranreddy.bokka
     * Get the list of capabilities mapped to the outcome for a given account
     * @param {companyId} Account SysId for fetching capabilities from capability application table
     * @param {outcomeInstanceId} SysId of the outcome instance 
     * @retun list of capabilities grouped into corecapability and enhanced capability
     */

    getOutcomeCapabilityMapping: function(companyId, outcomeInstanceId) {
        var capabilityObj = {
            coreCapability: [],
            enhancingCapability: []
        };

        var outcomeInstanceGr = new GlideRecord(ValueOutcome.TABLE_OBJECTIVE_OUTCOME_INSTANCE);
        if (!outcomeInstanceGr.get(outcomeInstanceId) || !outcomeInstanceGr.canRead() || outcomeInstanceGr.user_defined)
            return capabilityObj;

        var outcomeDef = outcomeInstanceGr.outcome;
        var coreCapabilities = (outcomeDef.core_capabilities || "").split(",");
        var enhancingCapabilities = (outcomeDef.value_enhancing_capabilities || "").split(",");

        for (var i in coreCapabilities) {
            var capabilityRecord = this._getCapabilityDetails(companyId, coreCapabilities[i]);
            if (capabilityRecord)
                capabilityObj.coreCapability.push(capabilityRecord);
        }

        for (var j in enhancingCapabilities) {
            var enhancingCapability = this._getCapabilityDetails(companyId, enhancingCapabilities[j]);
            if (enhancingCapability)
                capabilityObj.enhancingCapability.push(enhancingCapability);
        }

        return capabilityObj;
    },

    /*
     * Author: Saikiranreddy.bokka
     * Gets the capability from the customer application table with the "usage" state
     * @param {companyId} Account SysId
     * @param {capabilityId} Capability definition ID
     * @retun returns capability record if a match is found for the current company else returns null
     */
    _getCapabilityDetails: function(companyId, capabilityId) {
        var capabilityAppGr = new GlideRecord(ValueOutcome.TABLE_CAPABILITY_APPLICATION);
        capabilityAppGr.addQuery("application", capabilityId);
        capabilityAppGr.addQuery("company", companyId);
        capabilityAppGr.query();
        if (capabilityAppGr.canRead() && capabilityAppGr.next()) {
            return {
                usage: capabilityAppGr.getValue("revised_status"),
                capabilityTitle: capabilityAppGr.application.name.toString()
            };
        }
        return null;
    },

    getBusinessObjOutcomeDetails: function(objOutcomeInstanceId) {
        var bussObjOutcomeGR = new GlideRecord(ValueOutcome.TABLE_OBJECTIVE_OUTCOME_INSTANCE);
        bussObjOutcomeGR.addQuery("sys_id", objOutcomeInstanceId);
        bussObjOutcomeGR.query();
        if (bussObjOutcomeGR.canRead() && bussObjOutcomeGR.next()) {
            return {
                objective: bussObjOutcomeGR.getDisplayValue("objective.title"),
                objectiveId: bussObjOutcomeGR.getDisplayValue("objective.sys_id"),
                outcome: bussObjOutcomeGR.getDisplayValue("title"),
                company: bussObjOutcomeGR.getDisplayValue("objective.company"),
                companySysId: bussObjOutcomeGR.getDisplayValue("objective.company.sys_id"),
                isCustom: bussObjOutcomeGR.getDisplayValue("user_defined"),
                outcomeSysId: bussObjOutcomeGR.getDisplayValue('sys_id'),
                description: bussObjOutcomeGR.getDisplayValue("description"),
            };
        }
    },

    /*
     * Author: tony.huang
     * Gets outcome and metric details for a given outcome instance id
     * @param {outcomeInstanceId} Outcome Instance Id
     * @retun Outcome and Metric details
     */
    getOutcomeMetricDetails: function(outcomeInstanceId) {
        var outcomeMetricGr = new GlideRecord(ValueOutcome.TABLE_OUTCOME_METRIC_INSTANCE),
            result = {},
            outcome = {},
            metric = {},
            isCustomer = new x_snc_octane_sp.OctaneImpactRoleUtils().isCustomerUser(gs.getUserID());
        outcomeMetricGr.addEncodedQuery('outcome.sys_id=' + outcomeInstanceId);
        outcomeMetricGr.query();
        if (outcomeMetricGr.canRead() && outcomeMetricGr.next()) {
            outcome.id = outcomeInstanceId;
            outcome.title = outcomeMetricGr.getDisplayValue('outcome.title');
            outcome.description = outcomeMetricGr.getDisplayValue('outcome.outcome.description');
            outcome.isAuto = outcomeMetricGr.getDisplayValue('outcome.isauto');
            outcome.products = outcomeMetricGr.outcome.outcome.product.getDisplayValue();
            outcome.goal = outcomeMetricGr.getDisplayValue('goal');
            outcome.suggestedGoal = outcomeMetricGr.getDisplayValue('outcome.outcome.suggested_goal');
            outcome.valuePotential = outcomeMetricGr.getDisplayValue('outcome.quarterly_value') ? parseInt(outcomeMetricGr.getDisplayValue('outcome.quarterly_value')) * 4 : '';

            metric.id = outcomeMetricGr.getDisplayValue('metric.source_id');
            metric.goalPercentage = outcomeMetricGr.getValue('goal') ? parseFloat(outcomeMetricGr.getValue('goal')) : '';
            metric.title = outcomeMetricGr.getDisplayValue('metric');
            metric.description = outcomeMetricGr.getDisplayValue('metric.description');
            metric.instanceId = outcomeMetricGr.getDisplayValue('instance.snc_instance_id');
            metric.metricDirection = outcomeMetricGr.getDisplayValue('metric.positive_measurement_direction');
            metric.metricType = outcomeMetricGr.getDisplayValue("metric.indicator_type");
            outcome.metric = metric;
        }
        result.outcome = outcome;
        this._getLatestOutcomePerformance(outcomeInstanceId, result.outcome, isCustomer);
        result.bizValue = this._getLatestBizValue(outcomeInstanceId, isCustomer);
        result.bizValue.valuePotential = outcome.valuePotential;
        return result;
    },
    /*
     * Author: tony.huang
     * Gets latest outcome performance for a given outcome instance id
     * @param 
     *  {outcomeInstanceId} Outcome Instance Id
     *  {outcome} outcome object within result payload
     *  {isCustomer} is current user a customer user
     * @retun latest outcome performance
     */
    _getLatestOutcomePerformance: function(outcomeInstanceId, outcome, isCustomer) {
        var outcomePerfGr = new GlideRecord(ImpactBizKPITables.Operational_Performance),
            value = '',
            timePeriod = '',
            refPeriod = '',
            outcomePerfs = [];
        outcomePerfGr.addEncodedQuery('outcome_data_reference.sys_id=' + outcomeInstanceId + '^valueISNOTEMPTY');
        if (isCustomer) {
            outcomePerfGr.addQuery("state", 'published');
        }
        outcomePerfGr.query();
        while (outcomePerfGr.canRead() && outcomePerfGr.next()) {
            timePeriod = outcomePerfGr.getDisplayValue('start_quarter');
            refPeriod = outcomePerfGr.getDisplayValue('outcome_metric_reference.reference_value');
            value = outcomePerfGr.getValue('value') ? parseFloat(outcomePerfGr.getValue('value')) : '';
            outcomePerfs.push({
                timePeriod: timePeriod,
                refPeriod: refPeriod,
                value: value
            });
        }
        outcomePerfs.sort(function(a, b) {
            return b.timePeriod.localeCompare(a.timePeriod);
        });
        outcome.metric.value = outcomePerfs.length > 0 && outcomePerfs[0].value ? outcomePerfs[0].value : '';
        outcome.metric.timePeriod = outcomePerfs.length > 0 ? outcomePerfs[0].timePeriod : '';
        outcome.metric.refPeriod = outcomePerfs.length > 0 ? outcomePerfs[0].refPeriod : '';
        outcomePerfs = outcomePerfs.filter(function(outcomePerf) {
            return outcomePerf.timePeriod === outcome.metric.refPeriod;
        });
        outcome.metric.refValue = outcomePerfs.length > 0 && outcomePerfs[0].value ? outcomePerfs[0].value : '';
        if (outcome.metric.direction === 'low') {
            outcome.metric.goal = outcome.metric.value && outcome.metric.goalPercentage ? Math.round((outcome.metric.refValue - outcome.metric.refValue * outcome.metric.goalPercentage / 100) * 100) / 100 : '';
		} else {
            outcome.metric.goal = outcome.metric.value && outcome.metric.goalPercentage ? Math.round((outcome.metric.refValue + outcome.metric.refValue * outcome.metric.goalPercentage / 100) * 100) / 100 : '';
        }
        outcome.metric.growthPercentage = outcome.metric.value && outcome.metric.refValue ? Math.round((outcome.metric.value - outcome.metric.refValue) / outcome.metric.refValue * 100) : '';
    },
    /*
     * Author: tony.huang
     * Gets latest business value for a given outcome instance id
     * @param 
     *  {outcomeInstanceId} Outcome Instance Id
     *  {isCustomer} is current user a customer user
     * @retun latest business value
     */
    _getLatestBizValue: function(outcomeInstanceId, isCustomer) {
        var outcomeAggregateGr = new GlideRecord(ImpactBizKPITables.VALUE_OUTCOME_AGGREGATE),
            latestTimeStamp = '',
            value = '',
            reportName = '',
            reportId = '',
            reportValue = 0;
        outcomeAggregateGr.addEncodedQuery('scan_outcome.outcome_data_reference.sys_id=' + outcomeInstanceId + '^metric_aggregate=17e2bcb61b7d781074e4a8e3b24bcbc1');
        if (isCustomer) {
            outcomeAggregateGr.addQuery("scan_outcome.value_assessment_reference.state", 'published');
        }
        outcomeAggregateGr.orderByDesc('scan_outcome.value_assessment_reference.sys_created_on');
        outcomeAggregateGr.query();
        if (outcomeAggregateGr.canRead() && outcomeAggregateGr.next()) {
            value = outcomeAggregateGr.getValue('value') ? outcomeAggregateGr.getValue('value') : '';
            latestTimeStamp = outcomeAggregateGr.getDisplayValue('scan_outcome.value_assessment_reference.sys_created_on');
            reportName = outcomeAggregateGr.getDisplayValue('scan_outcome.value_assessment_reference.report_name');
            reportId = outcomeAggregateGr.getDisplayValue('scan_outcome.value_assessment_reference.sys_id');
            outcomeAggregateGr = new GlideAggregate(ImpactBizKPITables.VALUE_OUTCOME_AGGREGATE);
            outcomeAggregateGr.addEncodedQuery('scan_outcome.value_assessment_reference=' + reportId + '^metric_aggregate=17e2bcb61b7d781074e4a8e3b24bcbc1');
            outcomeAggregateGr.addAggregate('SUM', 'value');
            outcomeAggregateGr.query();
            while (outcomeAggregateGr.canRead() && outcomeAggregateGr.next()) {
                reportValue += parseFloat(outcomeAggregateGr.getAggregate('SUM', 'value'));
            }
            reportValue = Math.round(reportValue);
        }
        return {
            timeStamp: latestTimeStamp,
            value: value,
            reportName: reportName,
            reportId: reportId,
            reportValue: reportValue
        };
    },

    type: 'ValueOutcome'
};
"""

In [18]:
tc_4 = """
var TestPerformanceUtil = Class.create();
TestPerformanceUtil.prototype = Object.extendsObject(global.AbstractAjaxProcessor, {

    bulkTriggerPerformanceRun: function() {

        var dateTime = new GlideDateTime();
        var perfTestRunGr = new GlideRecord('sn_test_management_performance_test_run');
        perfTestRunGr.addEncodedQuery('u_statusINQueued for Execution^u_test_cycle.u_test_cycle_type=performance');
        perfTestRunGr.query();
        while (perfTestRunGr.next()) {
            gs.eventQueueScheduled("sn_test_management.triggerPerfBuild", perfTestRunGr, perfTestRunGr.sys_id, perfTestRunGr.u_job_name, dateTime);
            dateTime.addSeconds(30);
 
        }
    },

    bulkTriggerPerfRunDevNow: function() {

        var dateTime = new GlideDateTime();
        var perfTestRunGr = new GlideRecord('sn_test_management_performance_test_run');
        perfTestRunGr.addEncodedQuery('u_statusINscheduled^u_test_cycle.u_test_cycle_type=performance');
        perfTestRunGr.query();
        while (perfTestRunGr.next()) {
            gs.eventQueueScheduled("sn_test_management.triggerPerfBuild", perfTestRunGr, perfTestRunGr.sys_id, perfTestRunGr.u_job_name, dateTime);
            dateTime.addSeconds(30);

        }
    },

    addTestSetToTestCycle: function() {
        var setIds = this.getParameter('sysparm_set_ids');
        var testCycleId = this.getParameter('sysparm_test_cycle_id');
        var testCycleGR = new GlideRecord('sn_test_management_test_cycle');
        testCycleGR.get(testCycleId);
        gs.eventQueue('sn_test_management.perfexecutsuite.creat', null, testCycleId, setIds);
        var result = this.newItem('result');
        result.setAttribute('testCycleDescription', testCycleGR.getValue('short_description'));


    },

    triggerPerformanceRun: function(jobName, perfRunGr, jobURL) {
        try {
            var jenkinsType = 'performance';

            var runOnInstanceBranch = perfRunGr.u_test_cycle.u_run_on_specific_branch;
            var tenantID = perfRunGr.u_test_cycle.u_tenant_id + "";
            var subscription = perfRunGr.u_test_cycle.u_subscription + "";
            var resourceGroup = perfRunGr.u_test_cycle.u_resource_group + "";
            var loadTestResource = perfRunGr.u_test_cycle.u_load_test_resource + "";
            var repoName = perfRunGr.u_test_cycle.u_repo_name + "";
            var location = perfRunGr.u_test_cycle.u_location;


            //var nextBuildNumber = this.getNextBuildNumber(jenkinsType, platformVersion, jobName, mobileAutomation, deviceType, jobURL);
            //var jobDetailsObject = this.fetchJobDetails(jenkinsType, platformVersion, jobName, mobileAutomation, deviceType, jobURL);
            // if (jobDetailsObject.inQueue == false && (this.fetchBuildDetails(jenkinsType, platformVersion, jobName, nextBuildNumber - 1, mobileAutomation, deviceType, jobURL)).building == false) {
            var buildNumber = this._triggerBuild(jenkinsType, jobName, runOnInstanceBranch, jobURL, tenantID, subscription, resourceGroup, loadTestResource, repoName, location); //<--------------------
            perfRunGr.u_build_number = buildNumber;
            perfRunGr.u_status = 'Queued';

            var reason = 'Updated after successful build creation response';
            perfRunGr.update(reason);

            return buildNumber;
           
        } catch (ex) {

            gs.error("SI: TestPerformanceUtil,triggerPerformanceRun " + ex.message);
        }
    },

    _triggerBuild: function(jenkinsType, jobName, runOnInstanceBranch, jobURL, tenantID, subscription, resourceGroup, loadTestResource, repoName, location) {

        var buildNumber = this.getNextBuildNumber(jenkinsType, jobName, jobURL);
        gs.info("CICD_Test 86 " + tenantID + " " + buildNumber);

        var httpStatusCode = this.buildJob(jenkinsType, jobName, tenantID, subscription, resourceGroup, loadTestResource, repoName, location, runOnInstanceBranch); //<---------------------
        gs.info("CICD_Test 89 " + httpStatusCode);

        if (httpStatusCode == 201) {
            return buildNumber;
        } else {
            throw new Error("Unable to trigger build. Https Status Code - " + httpStatusCode);
        }
    },


    updatePerformanceTestRunStatus: function() {
        var perfRunGr = new GlideRecord('sn_test_management_performance_test_run');
        perfRunGr.addEncodedQuery('u_test_cycle.u_test_cycle_type=performance^u_statusINQueued,Running');

        perfRunGr.query();
        while (perfRunGr.next()) {
            gs.info("CICD_Testutil 114 " + perfRunGr.u_number);

            var jenkinsType = 'performance';
            var buildNumber = perfRunGr.u_build_number;
            //  var platformVersion = perfRunGr.u_test_cycle.u_platform_version;
            var jobName = perfRunGr.u_job_name;
            //   var mobileAutomation = perfRunGr.u_test_cycle.u_mobile_automation;
            //   var deviceType = perfRunGr.u_test_cycle.u_device_os + "";
            var jobURL = perfRunGr.u_job_url.split('/job/')[3];


            var buildDetailsObject = this.fetchBuildDetails(jenkinsType, jobName, buildNumber, jobURL);
            if (Object.keys(buildDetailsObject).length > 0) {
                if (buildDetailsObject.building == true) {
                    perfRunGr.u_status = 'Running';
                } else {
                    perfRunGr.u_status = buildDetailsObject.result.toString();


                    if (buildDetailsObject.result.toString() == 'SUCCESS') {

                        var azureResult = this.fetchAzurePerfResult(buildNumber);
                        gs.info("CICD_Test136 " + azureResult);

                        if (azureResult.indexOf('GET ERROR.') != 0) {
                            var parsedResult = JSON.parse(azureResult);

                            this.updatePerfResult(perfRunGr, parsedResult);
                        }

                    }

                }
                var reason = 'Updated after getting status of the Build by API response';
                perfRunGr.update(reason);
            }
        }
    },

    fetchAzurePerfResult: function(buildNumber) {
        try {
            var r = new sn_ws.RESTMessageV2('CICD_perf_Jenkins', 'Get_Azure_Result');
            r.setStringParameterNoEscape('build_number', buildNumber);

            var response = r.execute();
            var httpStatus = response.getStatusCode();
            var responseBody = response.getBody();
            //gs.info("CICD_Test155" + httpStatus + " " + parseBody.status);
            if (httpStatus == 200) {
                // gs.info("CICD_Test653 " + responseBody);
                return responseBody;
            } else {
                return 'GET ERROR. Response Status - ' + httpStatus + ' Response Body - ' + responseBody;
            }

        } catch (ex) {
            var message = ex.message;
            gs.error("SI: TestPerformanceUtil,buildJob " + message);

        }
    },
    updatePerfResult: function(perfRunGr, result) {

        if (!gs.nil(perfRunGr.u_build_number) && Object.keys(result.testRunStatistics).length > 0) {
            var performanceTestResultGr = new GlideRecord('sn_test_management_performance_test_result');
            var len = Object.keys(result.testRunStatistics).length;

            for (var i = 0; i < len; i++) {

                var fieldAttrib = (Object.keys(result.testRunStatistics)[i]);

                if (performanceTestResultGr.canCreate()) {

                    performanceTestResultGr.initialize();

                    performanceTestResultGr.u_test_name = result.displayName;
                    performanceTestResultGr.u_performance_test_run = perfRunGr.sys_id;
                    performanceTestResultGr.u_build_number = perfRunGr.u_build_number;
                    performanceTestResultGr.u_test_cycle = perfRunGr.u_test_cycle;
                    performanceTestResultGr.u_transaction_name = result.testRunStatistics[fieldAttrib].transaction;
                    performanceTestResultGr.u_sample_count = result.testRunStatistics[fieldAttrib].sampleCount.toFixed(2);
                    performanceTestResultGr.u_error_count = result.testRunStatistics[fieldAttrib].errorCount.toFixed(2);
                    performanceTestResultGr.u_error_pct = result.testRunStatistics[fieldAttrib].errorPct.toFixed(2);
                    performanceTestResultGr.u_mean_res_time = result.testRunStatistics[fieldAttrib].meanResTime.toFixed(2);
                    performanceTestResultGr.u_median_res_time = result.testRunStatistics[fieldAttrib].medianResTime.toFixed(2);
                    performanceTestResultGr.u_max_res_time = result.testRunStatistics[fieldAttrib].maxResTime.toFixed(2);
                    performanceTestResultGr.u_min_res_time = result.testRunStatistics[fieldAttrib].minResTime.toFixed(2);
                    performanceTestResultGr.u_pct1_res_time = result.testRunStatistics[fieldAttrib].pct1ResTime.toFixed(2);
                    performanceTestResultGr.u_pct2_res_time = result.testRunStatistics[fieldAttrib].pct2ResTime.toFixed(2);
                    performanceTestResultGr.u_pct3_res_time = result.testRunStatistics[fieldAttrib].pct3ResTime.toFixed(2);
                    performanceTestResultGr.u_throughput = result.testRunStatistics[fieldAttrib].throughput.toFixed(2);
                    performanceTestResultGr.u_received_kbytes_per_sec = result.testRunStatistics[fieldAttrib].receivedKBytesPerSec.toFixed(2);
                    performanceTestResultGr.u_sent_kbytes_per_sec = result.testRunStatistics[fieldAttrib].sentKBytesPerSec.toFixed(2);


                    var criteriaResult = this.getPFCriteria(result, result.testRunStatistics[fieldAttrib].transaction);

                    //performanceTestResultGr.sla = criteriaResult.sla;
                    performanceTestResultGr.u_result = criteriaResult.result;

                    performanceTestResultGr.insert();

                }

            }
        }

    },

    getPFCriteria: function(result, transaction_name) {

        try {
            var len = Object.keys(result.passFailCriteria.passFailMetrics).length;

            var obj = {};
            for (var j = 0; j < len; j++) {
                var passfailAttrib = Object.keys(result.passFailCriteria.passFailMetrics)[j];


                var reqname = result.passFailCriteria['passFailMetrics'][passfailAttrib].requestName;

                if (transaction_name == reqname) {
                    gs.info("CICD_Test262 " + transaction_name);

                    obj.sla = result.passFailCriteria['passFailMetrics'][passfailAttrib].value;

                    obj.result = result.passFailCriteria['passFailMetrics'][passfailAttrib].result;

                    return obj;
                }

            }
            obj.sla = "";
            obj.result = "";

            return obj;

        } catch (ex) {
            gs.info("SI: TestPerformanceUtil,getPFCriteria " + ex);
        }

    },

    getNextBuildNumber: function(jenkinsType, jobName, jobURL) {
        var jobDetailsObject = this.fetchJobDetails(jenkinsType, jobName, jobURL);
        // gs.info("CICD_Test 8 " + JSON.stringify(jobDetailsObject));

        var nextBuildNumber = jobDetailsObject.nextBuildNumber;
        if (typeof nextBuildNumber == "undefined") {
            return 0;
        }
        if (jobDetailsObject.inQueue == true) {
            nextBuildNumber += 1;
        }
        return nextBuildNumber;
    },

    fetchJobDetails: function(jenkinsType, jobName, jobURL) {
        var queryString = jobName + '/api/json';
        var restName = this._getRESTMsgName(jenkinsType);
        gs.info("CICD_Test 322 " + restName + " " + jobURL);

        var jobDetails = this.getJobInfo(restName, queryString);

        var jobDetailsObject = {};
        if (jobDetails.indexOf('GET ERROR.') != 0) {
            jobDetailsObject = JSON.parse(jobDetails);
        }
        return jobDetailsObject;

    },

    buildJob: function(jenkinsType, jobName, tenantID, subscription, resourceGroup, loadTestResource, repoName, location, runOnInstanceBranch) {
        var paramJsonObj = this._getJobParameters(jenkinsType, tenantID, subscription, resourceGroup, loadTestResource, repoName, location, runOnInstanceBranch);
        gs.info("CICD_Test 250 " + tenantID + subscription + resourceGroup + loadTestResource + repoName);

        try {


            var request = new sn_ws.RESTMessageV2(this._getRESTMsgName(jenkinsType), 'BUILD');


            var crumbObj = this.getCrumb(jenkinsType);
            request.setRequestHeader('Jenkins-Crumb', crumbObj.crumb);

            request.setStringParameterNoEscape('QUERY_STRING', jobName);

            var params = paramJsonObj.parameters;
            for (var i = 0; i < params.length; i++) {
                var key = params[i].name + "";
                var value = params[i].value + "";
                request.setQueryParameter(key, value);
            }


            var response = request.execute();
            var responseBody = response.getBody();
            var httpStatus = response.getStatusCode();
            return httpStatus;
        } catch (ex) {
            var message = ex.message;
            gs.error("SI: TestPerformanceUtil,buildJob " + message);

            return message;
        }
    },

    getCrumb: function(jenkinsType) {

        try {
            var request = new sn_ws.RESTMessageV2('sn_test_management.CICD_perf_Jenkins', 'GETCRUMB');

            var response = request.execute();
            var cookie;
            var headers = response.getAllHeaders();
            //find and return cookie
            for (var a in headers) {
                if (headers[a].name.toLowerCase() === 'set-cookie') {
                    cookie = headers[a].value;
                    break;
                }
            }
            if (!cookie) {
                throw new Error('Unable to get Session info from Jenkins');
            }

            var responseBody = response.getBody();
            var httpStatus = response.getStatusCode();
            var jsonObj;
            if (httpStatus == '200') {
                jsonObj = JSON.parse(responseBody);

                var crumbJson = {};
                crumbJson.crumb = jsonObj.crumb;
                crumbJson.cookie = cookie;
                return crumbJson;

            } else {
                gs.error('SI: TestPerformanceUtil,getCrumb Response Status - ' + httpStatus + ' Response Body - ' + responseBody);

            }
        } catch (ex) {
            gs.error("SI: TestPerformanceUtil,getCrumb " + ex.message);

        }

    },


    fetchBuildDetails: function(jenkinsType, jobName, buildNumber, jobURL) {

        //var queryString = '/job/' + jobName + '/' + buildNumber + '/api/json';
        jobName = jobName + '/' + buildNumber + '/api/json';

        var restName = this._getRESTMsgName(jenkinsType);
        var jobDetails = this.getJobInfo(restName, jobName);
        gs.info("CICD_Test 158 " + JSON.stringify(jobDetails));

        var jobDetailsObject = {};
        if (jobDetails.indexOf('GET ERROR.') != 0) {
            jobDetailsObject = JSON.parse(jobDetails);
        }
        return jobDetailsObject;
    },

    getJobInfo: function(restName, jobName) {
        try {
            var request = new sn_ws.RESTMessageV2(restName, 'GETJOBINFO');
            request.setStringParameterNoEscape('QUERY_STRING', jobName);
            var response = request.execute();
            var responseBody = response.getBody();
            var httpStatus = response.getStatusCode();
            if (httpStatus == '200') {
                return responseBody;
            } else {
                gs.error('SI: TestPerformanceUtil,getJobInfo Response Status - ' + httpStatus + ' Response Body - ' + responseBody);

                return 'GET ERROR. Response Status - ' + httpStatus + ' Response Body - ' + responseBody;
            }
        } catch (ex) {
            gs.error("SI: TestPerformanceUtil,getJobInfo " + ex.message);
        }

    },

    _getRESTMsgName: function(type) {
        if (type == 'performance') {
            return 'sn_test_management.CICD_perf_Jenkins';
        }

    },

    _getJobParameters: function(type, tenantID, subscription, resourceGroup, loadTestResource, repoName, location, runOnInstanceBranch) {
        if (type == 'performance') {
            return {
                "parameters": [{
                        "name": "tenantId",
                        "value": tenantID
                    },
                    {
                        "name": "subscription",
                        "value": subscription
                    },
                    {
                        "name": "resourceGroup",
                        "value": resourceGroup
                    },
                    {
                        "name": "location",
                        "value": location
                    },
                    {
                        "name": "loadTestResource",
                        "value": loadTestResource
                    },
                    {
                        "name": "repo_name",
                        "value": repoName
                    }, {
                        "name": "branch_name",
                        "value": runOnInstanceBranch
                    }
                ]
            };
        }
    },




    addExecutionSuiteToTestCycle: function(testCycleId, setIds) {
        var testCycleGR = new GlideRecord('sn_test_management_test_cycle');
        if (testCycleGR.get(testCycleId)) {
            var startDate = testCycleGR.getValue('start_date');
            var endDate = new GlideDateTime();
            endDate.setValue(startDate);
            endDate.addDaysUTC(1);
            var platformVersion = testCycleGR.getValue('u_platform_version');
            var testPlan = testCycleGR.getValue('parent');
            var executionSuiteObj = this.getExecutionSuiteArr(testCycleId, setIds, startDate, endDate);
            gs.info("TestCycle_Processor_CICD16 " + executionSuiteObj);

            for (var i = 0; i < executionSuiteObj.length; i++) {
                var executionSuite = executionSuiteObj[i].id;
                var testSet = executionSuiteObj[i].testSet;

                this.addPerfTestRunMapping(testPlan, testCycleId, executionSuite, testSet, platformVersion);

            }
        }
    },

    getExecutionSuiteArr: function(testCycleId, setIds, startDate, endDate) {
        var arr = [];
        var testSets = setIds.split(',');
        for (var i = 0; i < testSets.length; i++) {
            var testSet = testSets[i];
            var executionSuite;
            var executionSuiteGr = new GlideRecord('sn_test_management_test_execution_suite');
            executionSuiteGr.addQuery('parent', testCycleId);
            executionSuiteGr.addQuery('u_test_set', testSet);
            executionSuiteGr.query();
            if (executionSuiteGr.next()) {
                executionSuite = executionSuiteGr.getUniqueValue();
            } else {
                executionSuite = this.createExecutionSuite(testCycleId, testSet, startDate, endDate);
            }
            if (executionSuite) {
                var executionSuiteObj = {};
                executionSuiteObj.id = executionSuite;
                executionSuiteObj.testSet = testSet;
                arr.push(executionSuiteObj);
            }
        }
        return arr;
    },


    addPerfTestRunMapping: function(testPlan, testCycleId, executionSuite, testSet, platformVersion) {

        var testCycle = new GlideRecord('sn_test_management_test_cycle');
        testCycle.addQuery('sys_id', testCycleId);
        testCycle.query();
        testCycle.next();


        var jobIdsArr = this.getJobIdsFromTestSet(testSet, platformVersion);
        gs.info("TestCycle_Processor_CICD 461 " + jobIdsArr);

        var existingJobIds = this.getExistingJobsArr(jobIdsArr.join(), testPlan, testCycleId, executionSuite, testSet);

        this.createPerfTestRun(jobIdsArr, testPlan, testCycle, executionSuite, testSet, existingJobIds);
    },

    createExecutionSuite: function(testCycleId, testSet, startDate, endDate) {
        var testSetGr = new GlideRecord('sn_test_management_test_set');
        if (testSetGr.get(testSet)) {
            var executionSuiteGr = new GlideRecord('sn_test_management_test_execution_suite');

            if (executionSuiteGr.canCreate()) {
                executionSuiteGr.initialize();
                executionSuiteGr.setValue('parent', testCycleId);
                executionSuiteGr.setValue('u_test_set', testSet);
                executionSuiteGr.setValue('start_date', startDate);
                executionSuiteGr.setValue('end_date', endDate);
                executionSuiteGr.setValue('short_description', testSetGr.getValue('name'));
                executionSuiteGr.setValue('u_business_group_ref', testSetGr.getValue('u_business_group_ref'));
                executionSuiteGr.setValue('cmdb_ci_business_app', testSetGr.getValue('u_business_app'));
                executionSuiteGr.setValue('u_execution_suite_type', testSetGr.getValue('u_test_set_type'));
                //  executionSuiteGr.setValue('u_executed_by', 'QA');
                executionSuiteGr.setValue('assignment_group', testSetGr.getValue('u_assignment_group'));
                executionSuiteGr.setValue('assigned_to', testSetGr.getValue('u_assigned_to'));
                var sysId = executionSuiteGr.insert();
                return sysId;
            }

        }
    },


    createPerfTestRun: function(jobIdsArr, testPlan, testCycle, executionSuite, testSet, existingJobIds) {
        gs.info("TestCycle_Processor_CICD 153 " + jobIdsArr);

        var count = 0;
        for (var i = 0; i < jobIdsArr.length; i++) {
            if (existingJobIds.indexOf(jobIdsArr[i]) === -1) {
                var perfTestRunGr = new GlideRecord('sn_test_management_performance_test_run');
                if (perfTestRunGr.canCreate()) {
                    perfTestRunGr.setValue('u_test_plan', testPlan);
                    perfTestRunGr.setValue('u_test_cycle', testCycle.sys_id);
                    //perfTestRunGr.setValue('u_calculate_code_coverage', testCycle.getValue('u_capture_code_coverage'));
                    perfTestRunGr.setValue('u_test_execution_suite', executionSuite);
                    perfTestRunGr.setValue('u_test_set', testSet);
                    perfTestRunGr.setValue('u_orchestration_task', jobIdsArr[i]);
                    if (perfTestRunGr.insert())
                        count++;
                }

            }
        }
        return count;
    },


    getJobIdsFromTestSet: function(setIds, platformVersion) {
        var testSetJobGr = new GlideRecord('sn_test_management_m2m_test_set_job');
        testSetJobGr.addQuery('u_test_set', 'IN', setIds);
        testSetJobGr.query();
        var jobIdsArr = [];
        while (testSetJobGr.next()) {
            var jobId = testSetJobGr.getValue('u_orchestration_task');
            if (jobIdsArr.indexOf(jobId) === -1)
                jobIdsArr.push(testSetJobGr.getValue('u_orchestration_task'));
        }
        return jobIdsArr;
    },



    getExistingJobsArr: function(jobIds, testPlan, testCycle, executionSuite, testSet) {
        var arr = [];
        var perfTestRunGr = new GlideRecord('sn_test_management_performance_test_run');
        perfTestRunGr.addQuery('u_test_plan', testPlan);
        perfTestRunGr.addQuery('u_test_cycle', testCycle);
        perfTestRunGr.addQuery('u_test_execution_suite', executionSuite);
        perfTestRunGr.addQuery('u_test_set', testSet);
        perfTestRunGr.addQuery('u_orchestration_task', 'IN', jobIds);
        perfTestRunGr.query();
        while (perfTestRunGr.next()) {
            arr.push(perfTestRunGr.getValue('u_orchestration_task'));
        }
        return arr;
    },



    canTriggerPerfRun: function(testCycleId) {

        var testCycleGr = new GlideRecord('sn_test_management_test_cycle');
        if (testCycleGr.get(testCycleId)) {
            var state = testCycleGr.getValue('state');
            var testCycleType = testCycleGr.getValue('u_test_cycle_type');

            var perfTestRunGr = new GlideRecord('sn_test_management_performance_test_run');
            perfTestRunGr.addQuery('u_test_cycle', testCycleId);
            perfTestRunGr.query();
            if (perfTestRunGr.hasNext() && state == '4' && testCycleType == 'performance') {
                return true;
            }
        }
        return false;
    },
    queuePerformanceTestRun: function(testCycle) {
        var perfTestRunGr = new GlideRecord('sn_test_management_performance_test_run');
        perfTestRunGr.addQuery('u_test_cycle', testCycle);
        perfTestRunGr.query();
        while (perfTestRunGr.next()) {
            perfTestRunGr.setValue('u_status', 'Queued for Execution');
            perfTestRunGr.setValue('u_build_number', '');

            var reason = "updated the state from UI action to run the PTR.";
            perfTestRunGr.update(reason);

        }
    },

    type: 'TestPerformanceUtil'
});
"""

In [40]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(tc_1)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

print(time_taken)
print(tokens_count)

input prompt size: 4751
//Test case for getNormalizationMetricValues function
describe('ValueNormalizationUtil', function() {
  var valueNormalizationUtil = new ValueNormalizationUtil('company_id');
  it('should return null when unit definition is null', function() {
    var result = valueNormalizationUtil.getNormalizationMetricValues('shg_id', 'instance_id', 'product_id');
    expect(result).toBeNull();
  });
  it('should return cached result when cache is available', function() {
    spyOn(valueNormalizationUtil, '_getCache').and.returnValue('cached_result');
    var result = valueNormalizationUtil.getNormalizationMetricValues('shg_id', 'instance_id', 'product_id');
    expect(result).toEqual('cached_result');
  });
  it('should return result when cache is not available', function() {
    spyOn(valueNormalizationUtil, '_getCache').and.returnValue(undefined);
    spyOn(valueNormalizationUtil, '_getUnitDefinition').and.returnValue({Id: 'unit_id'});
    spyOn(valueNormalizationUtil, '_g

In [41]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(tc_2)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

print(time_taken)
print(tokens_count)

input prompt size: 1889
(function(outputs, steps, stepResult, assertEqual) {
    describe('ImpactDepartmentSupportService', function() {
        var impactDepartmentSupportService;

        beforeEach(function() {
            impactDepartmentSupportService = new ImpactDepartmentSupportService();
        });

        describe('createDefaultDepartment', function() {
            it('should create a default department record', function() {
                var params = {
                    accountName: 'Test Account',
                    accountSysId: '1234567890',
                    cmid: 'CMID123'
                };

                var departmentSysId = impactDepartmentSupportService.createDefaultDepartment(params);

                expect(departmentSysId).toBeDefined();
            });
        });

        describe('assignDepartmentSquadAndCustomers', function() {
            it('should assign squad and customer members to the default department', function() {
                var para

In [42]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(tc_3)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

print(time_taken)
print(tokens_count)

input prompt size: 3502


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(


// Test case 1: createValueOutcome - valid data
<PERSON>.test('createValueOutcome - valid data', () => {
    const data = {
        title: 'Test Outcome',
        description: 'Test Outcome Description',
        driver: 'Test Driver',
        product: 'Test Product',
        order: 1,
        source_id: '123',
        state: 'Active',
        type: 'Test Type',
        baseVersion: '1.0',
        version: '1.1'
    };

    const result = ValueOutcome.createValueOutcome(data);

    expect(result.recordCreated).toBe(true);
    expect(result.sysId).not.toBe('');
});

// Test case 2: createValueOutcome - missing required data
<PERSON>.test('createValueOutcome - missing required data', () => {
    const data = {
        title: 'Test Outcome',
        driver: 'Test Driver',
        product: 'Test Product',
        order: 1,
        source_id: '123',
        state: 'Active',
        type: 'Test Type',
        baseVersion: '1.0',
        version: '1.1'
    };

    const result = ValueOutcome.c

In [19]:
tc_5 = """
var AppCollectionDAO = Class.create();
AppCollectionDAO.prototype = {
    initialize: function() {
		this.orchestrationObj = new sn_orchestration.EnvironmentUtil();
		this.isFedStore = this.orchestrationObj.isEnvironmentGCC();
		this.isIl5Store = this.orchestrationObj.isEnvironmentIl5();
		this.isAusGovStore = this.orchestrationObj.isEnvironmentAusGov();
    },    

    getUniqueSourceAppIdsFromAppCollection : function(collectionType){
    	var source_app_id_list = [];
    	var source_app_id_map = {};

    	var collection_sys_id = new AppCollectionDAO().getCollectionSysId('collection', collectionType);
		var gr_app_collection = new GlideRecord('sn_appstore_app_collection');
		gr_app_collection.addQuery('collection', collection_sys_id);
		gr_app_collection.query();

		while(gr_app_collection.next()){
			if(!source_app_id_map[gr_app_collection.application.source_app_id.toString()]) {
				source_app_id_map[gr_app_collection.application.source_app_id.toString()] = true;
				source_app_id_list.push(gr_app_collection.application.source_app_id.toString());
			}
		}

		return source_app_id_list;
    },

    getCollectionSysId: function(collectionType, collectionName) { 
		gs.debug("AppCollectionDAO getCollectionSysId begin");
		var csys_id = null; 
		try{
			var collection_sys_id = new GlideRecord('sn_appstore_collection');
			collection_sys_id.addQuery('name', new AppStoreUtility().capitalize(collectionName));
			collection_sys_id.addQuery('type', new AppStoreUtility().capitalize(collectionType));
			collection_sys_id.query(); 
			if(collection_sys_id.next()){
				csys_id = collection_sys_id.sys_id;
			}
		}catch(err){
			gs.error("AppCollectionDAO getCollectionSysId error : " + err);
		}
		gs.debug("AppCollectionDAO getCollectionSysId end");
		return csys_id;
	},
	
	getCollectionSysIds: function(collectionType, collectionNameArray) { 
		gs.debug("AppCollectionDAO getCollectionSysIds begin");
		var csys_id = []; 
		try{
			var collection_sys_id = new GlideRecord('sn_appstore_collection');
			var collection_sys_id_or = collection_sys_id.addQuery('name', new AppStoreUtility().capitalize(collectionNameArray[0]));
			collection_sys_id.addQuery('type', new AppStoreUtility().capitalize(collectionType));
			for (var counter = 1; counter < collectionNameArray.length; counter++) {
				collection_sys_id_or.addOrCondition("name",  new AppStoreUtility().capitalize(collectionNameArray[counter]));
			}

			collection_sys_id.query(); 
			while(collection_sys_id.next()){
				csys_id.push(collection_sys_id.sys_id.toString());
			}
		}catch(err){
			gs.error("AppCollectionDAO getCollectionSysIds error : " + err);
		}
		gs.debug("AppCollectionDAO getCollectionSysIds end");
		return csys_id;
	},

    getCollectionSysIdsByParent: function(collectionType, parentCollectionArr) {
      var result = [];
      try{
        var collection_sys_id = new GlideRecord('sn_appstore_collection');
        collection_sys_id.addQuery('type', collectionType);
        var collection_sys_id_or = collection_sys_id.addQuery('category_label.name', new AppStoreUtility().capitalize(parentCollectionArr[0]));
        for (var counter = 1; counter < parentCollectionArr.length; counter++) {
          collection_sys_id_or.addOrCondition("category_label.name",  new AppStoreUtility().capitalize(parentCollectionArr[counter]));
        }
        
        collection_sys_id.query();
        while(collection_sys_id.next()){
          result.push(collection_sys_id.sys_id.toString());
        }
      }catch(err){
        gs.error("AppCollectionDAO getCollectionSysIds error : " + err);
      }
      return result;
    },
    
    getAllCollection:function(){
		gs.debug("AppCollectionDAO getAllCollection begin");
		var collection = [];
		try{
			var gr = new GlideRecord('sn_appstore_collection');
			gr.orderBy('type');
			gr.orderBy('name');			
			gr.query();
			while(gr.next()){
				if(this.isFedStore) {
					var allowed_environment = gr.allowed_environment;
					if(!gs.nil(allowed_environment) && !gs.nil(allowed_environment.allow_on_fed_store)) {
						if(allowed_environment.allow_on_fed_store == "false") {
							continue; // Do not include this.
						}
					}
				}
				if(this.isIl5Store) {
					var allowed_environment = gr.allowed_environment;
					if(!gs.nil(allowed_environment) && !gs.nil(allowed_environment.allow_on_il5_store)) {
						if(allowed_environment.allow_on_il5_store == "false") {
							continue; // Do not include this.
						}
					}
				}
				if(this.isAusGovStore) {
					var allowed_environment = gr.allowed_environment;
					if(!gs.nil(allowed_environment) && !gs.nil(allowed_environment.allow_on_irap_store)) {
						if(allowed_environment.allow_on_irap_store == "false") {
							continue; // Do not include this.
						}
					}
				}
				var collectionObj = {};
				collectionObj.name = gr.name.toString();
				collectionObj.sys_id = gr.sys_id.toString();
				collectionObj.type = gr.type.toString();

				collectionObj.value = gr.value.toString();
				collectionObj.selected = false;
                collectionObj.enabled = true;
				collectionObj.no_of_apps = parseInt(gr.no_of_apps.toString(), 10);
                collectionObj.no_of_integrations = parseInt(gr.no_of_integrations.toString(), 10);
                collectionObj.no_of_innovationlab_apps = parseInt(gr.no_of_innovationlab_apps.toString(), 10);
				collectionObj.sort_order = parseInt(gr.order_number.toString(), 10);
				collectionObj.category_label = !gs.nil(gr.category_label) ? gr.category_label.name.toString() : "";
				collection.push(collectionObj);
			}

			collection = this.moveAllIndustriesToTop(collection);

		}catch(err){
			gs.error("AppCollectionDAO getAllCollection error : " + err);
		}
		gs.debug("AppCollectionDAO getAllCollection end");
		return collection;
	},

    getCollection:function(collectionType){
      var collection = [];
      try{
        var gr = new GlideRecord('sn_appstore_collection');
        gr.addQuery('type', collectionType);
        gr.orderBy("order_number");
        gr.query();
        while(gr.next()){
          var collectionObj = {};
          collectionObj.name = gr.name.toString();
          collectionObj.sys_id = gr.sys_id.toString();
          collectionObj.type = gr.type.toString();
          collection.push(collectionObj);
        }
      }catch(err){
        gs.error("AppCollectionDAO getCollection error : " + err);
      }
      return collection;
    },

    getCategoriesWithLabels: function() {
        var categories = [];
        var gr = new GlideRecord('sn_appstore_collection');
        gr.addQuery('type', "category_label");
        gr.query();
        while(gr.next()){
          var categorySet = {};
          categorySet.label = gr.name.toString();
          categorySet.value = gr.sys_id.toString();
          var collection = [];
          var grCat = new GlideRecord('sn_appstore_collection');
          grCat.addQuery('category_label', gr.sys_id.toString());
          grCat.addQuery('type', "category");
          grCat.query();
          while(grCat.next()) {
              var category = {};
              category.sys_id = grCat.sys_id.toString();
              category.name = grCat.name.toString();
              collection.push(category);
          }
          categorySet.category_collection = collection;
          categories.push(categorySet);
        }
        return categories;
    },

	moveAllIndustriesToTop : function(collection){
		for (var i = 0; i < collection.length; i++) {                
            	if(collection[i].name == "All"){
 					gs.debug("All industries found");
            		var removed = collection.splice(i,1);
            		collection = removed.concat(collection);
            	}     
            }

        return collection;
	},

    getSelectedCollection:function(application_sys_id){
    	gs.debug("AppCollectionDAO getSelectedCollection begin");
    	var collection = [];
		try{
	    	collection = this.getAllCollection();
	    	var grCollection = new GlideRecord('sn_appstore_app_collection');
			grCollection.addQuery('application',application_sys_id);
			grCollection.query();
			while(grCollection.next()){			
				var collection_sys_id = grCollection.collection.toString();
				for(var i=0;i<collection.length;i++){
					if(collection[i].sys_id.toString() == collection_sys_id.toString()){
						collection[i].selected = true;
						break;
					}
				}			
			}
		}catch(err){
			gs.error("AppCollectionDAO getSelectedCollection error : " + err);
		}
		gs.debug("AppCollectionDAO getSelectedCollection end");
		return collection;
    },
    
    getAppCollection:function(application_sys_id){
    	var collections = [];
    	try{
	    	var grCollection = new GlideRecord('sn_appstore_app_collection');
			grCollection.addQuery('application',application_sys_id);
			grCollection.query();
			while(grCollection.next()){			
				var collection = {};
				collection.name = grCollection.collection.name.toString();
				collection.type = grCollection.collection.type.toString();
				collection.sys_id = grCollection.collection.sys_id.toString();
				if(!gs.nil(grCollection.collection.category_label)) {
				    collection.category_label_sys_id = grCollection.collection.category_label.sys_id.toString();
                    collection.category_label_name = grCollection.collection.category_label.name.toString();
                }
				collections.push(collection);
			}
		}catch(err){
			gs.debug("AppCollectionDAO getAppCollection error : " + err);
		}
		return collections;
    },

    getAppCollectionList : function(application) {
    	application.collection_list = "";
		application.category_list = "";
		application.industry_list = "";

		gs.debug("AppCollectionDAO getAppCollectionList begin");
		try{					
			for(var i=0;i<application.collection.length;i++){				
				if(application.collection[i].selected){										
					this.addToCollectionSearch(application, application.collection[i]);																	
				}
			}				
		}catch(err){
			gs.error("AppCollectionDAO getAppCollectionList error : " + err);
		}
		gs.debug("AppCollectionDAO getAppCollectionList end");
		return;
    },
	
	saveAppCollection : function(application, preventCategorySave, preventIndustrySave, application_sys_id){
		application.collection_list = "";
		application.category_list = "";
		application.industry_list = "";

		gs.debug("AppCollectionDAO saveAppCollection begin");
		if(gs.nil(application_sys_id)) {
			application_sys_id = application.sys_id;
		}
		try{
			var rec = new GlideRecord('sn_appstore_app_collection');
			rec.addQuery('application',application_sys_id);
			rec.query();
			var currentCollection = [];
			while (rec.next()) {
				currentCollection.push(rec.collection.toString());
			}
			
			for(var i=0;i<currentCollection.length;i++){				
				var deleteRecord = false;
				for(var j=0;j<application.collection.length;j++){
					if((application.collection[j].type == "category" && !preventCategorySave) || (application.collection[j].type == "industry" && !preventIndustrySave)){
						if(currentCollection[i].toString() == application.collection[j].sys_id.toString() && !application.collection[j].selected){
							deleteRecord = true;
						}	
					}					
				}
				if(deleteRecord){
					var recdelete = new GlideRecord('sn_appstore_app_collection');
					recdelete.addQuery('application',application_sys_id);
					recdelete.addQuery('collection',currentCollection[i]);
					recdelete.query();
					while (recdelete.next()) {
						recdelete.deleteRecord();
					}
				}
			}
			
			for(var i=0;i<application.collection.length;i++){
				if((application.collection[i].type == "category" && !preventCategorySave) || (application.collection[i].type == "industry" && !preventIndustrySave)){
					if(application.collection[i].selected){
						var insertRecord = true;
						for(var j=0;j<currentCollection.length;j++){
							if(currentCollection[j].toString() == application.collection[i].sys_id.toString()){
								this.addToCollectionSearch(application, application.collection[i]);
								insertRecord = false;
							}
						}
						if(insertRecord){
							var recinsert = new GlideRecord('sn_appstore_app_collection');
							recinsert.initialize();
							recinsert.application = application_sys_id; 
							recinsert.collection = application.collection[i].sys_id; 
							recinsert.insert();
							this.addToCollectionSearch(application, application.collection[i]);
						}
					}
				}				
			}
		}catch(err){
			gs.error("AppCollectionDAO saveAppCollection error : " + err);
		}
		gs.debug("AppCollectionDAO saveAppCollection end");
		return;
	},
	
	refreshAppCollection : function(application, collectionType){
		application.collection_list = "";
		application.category_list = "";
		application.industry_list = "";
		application.emdb_integration_product_list = "";
		if(gs.nil(application) || gs.nil(application.sys_id) || gs.nil(collectionType)){
			return;
		}
		gs.debug("AppCollectionDAO refreshAppCollection begin");
		var application_sys_id = application.sys_id;
		
		try{
			var rec = new GlideRecord('sn_appstore_app_collection');
			rec.addQuery('application',application_sys_id);
			rec.addQuery('collection.type', collectionType);
			rec.query();
			rec.deleteMultiple();
			for(var i=0;i<application.collection.length;i++){
				if(application.collection[i].type == collectionType && application.collection[i].selected){
					var recinsert = new GlideRecord('sn_appstore_app_collection');
					recinsert.initialize();
					recinsert.application = application_sys_id; 
					recinsert.collection = application.collection[i].sys_id; 
					recinsert.insert();
					this.addToCollectionSearch(application, application.collection[i]);
				}
			}
		}catch(err){
			gs.error("AppCollectionDAO refreshAppCollection error : " + err);
		}
		gs.debug("AppCollectionDAO refreshAppCollection end");
		return;
	},

	markAppAsCertified: function(appId){
		var coll_sysId = this.getCollectionSysId('collection', 'Certified');
		var recinsert = new GlideRecord('sn_appstore_app_collection');
		recinsert.initialize();
		recinsert.application = appId; 
		recinsert.collection = coll_sysId; 
		recinsert.insert();
	},

	addToCollectionSearch : function(application, collection){
		if(collection.type == "category"){
			application.category_list += collection.name + ";";
		} else if(collection.type == "industry"){
			application.industry_list += collection.name + ";";
		}
		else if(collection.type == "collection"){
			application.collection_list += collection.name + ";";
		}else if(collection.type == "category_label"){
			application.emdb_integration_product_list += collection.name + ";";
		}
	},

	updateCategoryAppCount : function(application_sys_id, increment){
		var gr_app_collection = new GlideRecord('sn_appstore_app_collection');
		gr_app_collection.addQuery('application',application_sys_id);
		gr_app_collection.addQuery('collection.type', 'IN', 'category,industry');
		gr_app_collection.query();
		while(gr_app_collection.next()){
			var gr_collection = new GlideRecord('sn_appstore_collection');
			gr_collection.addQuery('sys_id', gr_app_collection.collection);			
			gr_collection.query();
			if(gr_collection.next()){
				if(increment){
					gr_collection.no_of_apps = parseInt(gr_collection.no_of_apps) + 1;	
				}
				else{
					gr_collection.no_of_apps = parseInt(gr_collection.no_of_apps) - 1;
				}			
				gr_collection.update();
			}			
		}

	},

	isAppFeatured : function(source_app_id){
		var collection_sys_id = new AppCollectionDAO().getCollectionSysId('collection', 'featured');
		var gr_app_collection = new GlideRecord('sn_appstore_app_collection');
		gr_app_collection.addQuery('application.source_app_id', source_app_id);
		gr_app_collection.addQuery('collection', collection_sys_id);
		gr_app_collection.query();
		if(gr_app_collection.getRowCount() > 0){
			return true;
		}
		return false;
	},

	makeAppFeatured : function(application_sys_id){
		var collection_sys_id = new AppCollectionDAO().getCollectionSysId('collection', 'featured');
		var gr_app_collection = new GlideRecord('sn_appstore_app_collection');
		gr_app_collection.initialize();
		gr_app_collection.application = application_sys_id; 
		gr_app_collection.collection = collection_sys_id; 
		return gr_app_collection.insert();
	},

	removeFeatured : function(source_app_id){
		var collection_sys_id = new AppCollectionDAO().getCollectionSysId('collection', 'featured');
		var gr_app_collection = new GlideRecord('sn_appstore_app_collection');
		gr_app_collection.addQuery('application.source_app_id', source_app_id);
		gr_app_collection.addQuery('collection', collection_sys_id);
		gr_app_collection.query();
		while (gr_app_collection.next()) {
			gr_app_collection.deleteRecord();
		}
	},

	matchesCategory : function(application, application_sys_id) {
		var currMap = {};
		var count = 0;
		for(var i = 0; i < application.collection.length; i++) {
			if(application.collection[i].type == 'category' && application.collection[i].selected) {
				count++;			
			}
		}

		var gr = new GlideRecord('sn_appstore_app_collection');
		gr.addQuery('application', application_sys_id);
		gr.addQuery('collection.type', 'category');
		gr.query();		
		if(gr.getRowCount() != count) {
			return false;
		} else if(gr.getRowCount() > 0){
			while(gr.next()){
				currMap[gr.collection.sys_id.toString()] = true;
			}

			//check against incoming object
			for(var i = 0; i < application.collection.length; i++) {
				if(application.collection[i].type == 'category' && application.collection[i].selected && currMap[application.collection[i].sys_id] == null) {
					return false;
				}
				if(application.collection[i].type == 'category' && !application.collection[i].selected && currMap[application.collection[i].sys_id] != null) {
					return false;
				}
			}
		}
		return true;
	},

	matchesIndustry : function(application, application_sys_id) {
		var currMap = {};
		var count = 0;
		for(var i = 0; i < application.collection.length; i++) {
			if(application.collection[i].type == 'industry' && application.collection[i].selected) {
				count++;			
			}
		}

		var gr = new GlideRecord('sn_appstore_app_collection');
		gr.addQuery('application', application_sys_id);
		gr.addQuery('collection.type', 'industry');
		gr.query();		
		if(gr.getRowCount() != count) {
			return false;
		} else if(gr.getRowCount() > 0){
			while(gr.next()){
				currMap[gr.collection.sys_id.toString()] = true;
			}

			//check against incoming object
			for(var i = 0; i < application.collection.length; i++) {
				if(application.collection[i].type == 'industry' && application.collection[i].selected && currMap[application.collection[i].sys_id] == null) {
					return false;
				}
				if(application.collection[i].type == 'industry' && !application.collection[i].selected && currMap[application.collection[i].sys_id] != null) {
					return false;
				}
			}
		}
		return true;
	},
	
	appHasCategory: function(application_sys_id) {
		var grCollection = new GlideRecord('sn_appstore_app_collection');
		grCollection.addQuery('application', application_sys_id);
		grCollection.addNotNullQuery('collection');
		grCollection.addQuery('collection.type', 'category');
		grCollection.query();
		if(grCollection.hasNext()) {
			return true;
		}
		return false;
	},

    type: 'AppCollectionDAO'
};
"""

In [20]:
tc_6 = """
var TestDAO = Class.create();
		TestDAO.prototype = {
			initialize: function (daoAccessor) {
				if(daoAccessor) {
		    		this._daoAccessor = daoAccessor;
		    	} else {
		    		this._daoAccessor = new DAOAccessor();
		    	}
			},

			sysUserDao: new SysUserDao(),

			//Will return an array of appstore quote request ojects which are going to expiry after some days from today.  @param : noOfDaysBeforeExpiry : if passed 15 will give quoteRequests going to expiry after 15 days from today.
			getQuoteRequestsToBeExpired: function (noOfDaysBeforeExpiry, createdFor) {
				gs.debug("getQuoteRequestsToBeExpired: noOfDaysBeforeExpiry : " + noOfDaysBeforeExpiry + ", createdFor : " + createdFor + ", begin");
				var glideDateTimeNow = new GlideDateTime();
				glideDateTimeNow.addDaysLocalTime(noOfDaysBeforeExpiry);
				var appStoreQuoteRequesetGR = new GlideRecord("sn_appstore_quote_request");
				if (!gs.nil(createdFor)) {
					appStoreQuoteRequesetGR.addQuery("created_for", createdFor);
				}
				appStoreQuoteRequesetGR.addQuery("price_type", "IN", "paid_per_month,paid_per_fulfiller_per_month");
				appStoreQuoteRequesetGR.addQuery("status", "sent");
				appStoreQuoteRequesetGR.addQuery("expiry_date", ">=", glideDateTimeNow.getLocalDate());
				glideDateTimeNow.addDaysLocalTime(1);
				appStoreQuoteRequesetGR.addQuery("expiry_date", "<", glideDateTimeNow.getLocalDate());
				appStoreQuoteRequesetGR.query();
				var quoteRequestsToBeExpirdeArray = [];
				while (appStoreQuoteRequesetGR.next()) {
					var quoteRequestToBeExipred = this._constructAppStoreQuotRequestGRData(appStoreQuoteRequesetGR);
					quoteRequestsToBeExpirdeArray.push(quoteRequestToBeExipred);
				}
				gs.debug("getQuoteRequestsToBeExpired: end");
				return quoteRequestsToBeExpirdeArray;
			},

			//Will return an array of appstore quote request objects expired. If passed 1, will give quote requests expired yesterday.
			getQuoteRequestsExpired: function (noOfDaysAfterExpiry, createdFor) {
				gs.debug("getQuoteRequestsExpired: noOfDaysAfterExpiry : " + noOfDaysAfterExpiry + ", createdFor : " + createdFor + ", begin");
				var glideDateTimeNow = new GlideDateTime();
				var appStoreQuoteRequesetGR = new GlideRecord("sn_appstore_quote_request");
				if (!gs.nil(createdFor)) {
					appStoreQuoteRequesetGR.addQuery("created_for", createdFor);
				}
				appStoreQuoteRequesetGR.addQuery("price_type", "IN", "paid_per_month,paid_per_fulfiller_per_month");
				appStoreQuoteRequesetGR.addQuery("status", "sent");
				appStoreQuoteRequesetGR.addQuery("expiry_date", "<", glideDateTimeNow.getLocalDate());
				glideDateTimeNow.addDaysLocalTime(-noOfDaysAfterExpiry);
				appStoreQuoteRequesetGR.addQuery("expiry_date", ">=", glideDateTimeNow.getLocalDate());
				appStoreQuoteRequesetGR.query();
				var quoteRequestsExpirdeArray = [];
				while (appStoreQuoteRequesetGR.next()) {
					var quoteRequestExpired = this._constructAppStoreQuotRequestGRData(appStoreQuoteRequesetGR);
					quoteRequestsExpirdeArray.push(quoteRequestExpired);
				}
				gs.debug("getQuoteRequestsExpired: end");
				return quoteRequestsExpirdeArray;
			},

			_constructAppStoreQuotRequestGRData: function (appStoreQuoteRequesetGR) {
				gs.debug("_constructAppStoreQuotRequestGRData: begin");
				var appStoreQuotRequest = {};
				appStoreQuotRequest.quote_request_number = appStoreQuoteRequesetGR.quote_request_number.toString();
				appStoreQuotRequest.title = appStoreQuoteRequesetGR.application.title.toString();
				appStoreQuotRequest.applicationSysId = appStoreQuoteRequesetGR.application.source_app_id.toString();//This is to provide the link in the email notification for the right application.
				appStoreQuotRequest.vendorCompanyName = appStoreQuoteRequesetGR.application.company.name.toString();
				appStoreQuotRequest.vendorCompanyId = appStoreQuoteRequesetGR.application.company.sys_id.toString();
				appStoreQuotRequest.customerCompanyName = appStoreQuoteRequesetGR.company.name.toString();
				appStoreQuotRequest.userName = appStoreQuoteRequesetGR.user.name.toString();
				appStoreQuotRequest.notificationEmail = appStoreQuoteRequesetGR.user.email.toString();
				appStoreQuotRequest.currency_code = appStoreQuoteRequesetGR.currency_code.toString();
				appStoreQuotRequest.application_price = appStoreQuoteRequesetGR.application_price.toString();
				appStoreQuotRequest.no_of_fulfillers = appStoreQuoteRequesetGR.no_of_fulfillers.toString();
				appStoreQuotRequest.tax = appStoreQuoteRequesetGR.tax.toString();
				appStoreQuotRequest.expiryDate = appStoreQuoteRequesetGR.expiry_date.toString();
				appStoreQuotRequest.price_type = appStoreQuoteRequesetGR.price_type.toString();
				appStoreQuotRequest.price_type_displayValue = appStoreQuoteRequesetGR.price_type.getDisplayValue();
				appStoreQuotRequest.status = appStoreQuoteRequesetGR.status.toString();
				appStoreQuotRequest.created_for = appStoreQuoteRequesetGR.created_for.toString();
				appStoreQuotRequest.created_for_displayValue = appStoreQuoteRequesetGR.created_for.getDisplayValue();
				appStoreQuotRequest.quoteRequestSysId = appStoreQuoteRequesetGR.sys_id.toString();
				gs.debug("_constructAppStoreQuotRequestGRData: end");
				return appStoreQuotRequest;
			},

			getQuoteForApplication: function (args) {
				gs.debug("Inside AppStoreQuoteRequestDAO.getQuoteForApplication");

				if (!args.application.source_app_id) {
					return {
						error: "provide application.source_app_id"
					};
				}
				var companyId = args.companyId;
				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("company", companyId);
				gr.addQuery("application.source_app_id", args.application.source_app_id);
				gr.addQuery("created_for", "purchase");
				gr.addQuery("status", "sent");
				gr.query();
				if (gr.getRowCount() == 0) {
					return {error: "No Sent Quote Request"};
				}
				gr.next();
				return gr.sys_id.toString();
			},			

			cancelQuote: function (args, isScheduledJob) {
				gs.debug("Inside AppStoreQuoteRequestDAO.cancelQuote");

				if (!args.quote_sys_id) {
					return {
						error: "provide quote_sys_id"
					};
				}
				var companyId = args.companyId;
				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("sys_id", args.quote_sys_id);
				gr.addQuery('status', 'IN', 'sent,resend_requested');
				gr.query();
				if (gr.getRowCount() == 0) {
					return {error: "invalid quote id"};
				}
				if (gr.next()) {
					var quoteCompanyId = gr.company.sys_id.toString();
					if (isScheduledJob || companyId == quoteCompanyId) {
						gr.status = "cancelled";
						gr.cancel_sync_to_surf = false;
						gr.cancel_requested_by = gs.getUserID();
						gr.cancel_reason = args.reason;

						var content = {};
						content.request_number = gr.quote_request_number.toString();
						content.surf_quote_number = gr.surf_quote_number.toString();
						content.action = "cancel_quote";
						content.requested_by = this.sysUserDao.getNotificationContact(gs.getUserID(), true);
						content.reason = new AppStoreUtility().customEncodeURI(args.reason);

						new SURFWorkflowLibrary().sendtoSurf(content, "cancelquote");
						if (gs.getProperty("sn_appstore.demo_environment", "false") == "true") {
							//TODO implement surf callback for demo environment
						}
						gr.update();			//update in appstore tables after successful callback from surf

						var grRegistration = new GlideRecord("sn_appstore_vendor_registration_request");
						grRegistration.addQuery("company", gr.company.toString());
						grRegistration.addQuery("appstore_purchase_status", "pending");
						grRegistration.query();
						if (grRegistration.next()) {
							grRegistration.appstore_purchase_status = "rejected";
							grRegistration.update();
						}
						
						if(!gs.nil(gr.application)) {
							//check for any accepted order form and reset that after quote cancellation
							var gr_orderform = new GlideRecord('sn_appstore_app_order_form');
							gr_orderform.addQuery('application.source_app_id', gr.application.source_app_id.toString());
							gr_orderform.addQuery('company', quoteCompanyId);							
							gr_orderform.addQuery('status', 'order_accepted');
							gr_orderform.addQuery('active', 'true');
							gr_orderform.query();
							
							if(gr_orderform.next()) {
								gr_orderform.status = 'order_sent';
								gr_orderform.update();
							}
						}

						return {"success": "success"};
					} else {
						return {error: "unauthorized"};
					}
				}
			},
			
			expireQuote: function(){
				gs.debug("Inside AppStoreQuoteRequestDAO.expireQuote");
				var glideDateTimeNow = new GlideDateTime();
				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("expiry_date", "<", glideDateTimeNow.getLocalDate());
				gr.addQuery("status", 'sent');
				gr.query();
				if (gr.getRowCount() == 0) {
					gs.debug("No expired quotes found");
					return {result: "success"};
				}

				while (gr.next()) {
					var args = {};
					args.quote_sys_id = gr.sys_id.toString();
					args.reason = "Expired Quote";
					gs.debug("Expiring the quote id={0}", args.quote_sys_id);
					this.cancelQuote(args , true);
				}

				return {result: "success"};
			},

			resendQuote: function (args) {
				gs.debug("Inside AppStoreQuoteRequestDAO.resendQuote");

				if (!args.quote_sys_id) {
					return {
						error: "provide quote_sys_id"
					};
				}
				var companyId = args.companyId;
				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("sys_id", args.quote_sys_id);
				gr.query();
				if (gr.getRowCount() == 0) {
					return {error: "invalid quote id"};
				}

				if (gr.next()) {
					var quoteCompanyId = gr.company.sys_id.toString();
					if (companyId == quoteCompanyId) {
						var content = {};
						content.request_number = gr.quote_request_number.toString();
						content.surf_quote_number = gr.surf_quote_number.toString();
						content.action = "resend_quote";
						content.requested_by = this.sysUserDao.getNotificationContact(gs.getUserID(), true);
						content.quote_created_for = this.getSURFQuoteCreatedFor(gr.created_for.toString());
						new SURFWorkflowLibrary().sendtoSurf(content, "resendquote");

						if (gs.getProperty("sn_appstore.demo_environment", "false") == "true") {
							//TODO implement surf callback for demo environment
						}

						return {"success": "success"};
					} else {
						return {error: "unauthorized"};
					}
				}
			},

			getSURFQuoteCreatedFor: function (created_for) {
				var kvArray = {};
				kvArray["registration"] = "Vendor Registration";
				kvArray["purchase"] = "App Purchase";
				kvArray["add_fulfiller"] = "App Purchase – Add Fulfillers";
				kvArray["certification_credit"] = "Certificate Purchase";
				kvArray["add_instances"] = "Instance Purchase";
				kvArray["registration_renewal"] = "Vendor Renewal";
				kvArray["application_purchase_renewal"] = "App Renewal";
				kvArray["certification_credit_renewal"] = "Certificate Renewal";
				kvArray["add_instances_renewal"] = "Instance Renewal";

				return kvArray[created_for];
			},

			getBillingShippingAddress: function (source_app_id,companyId) {
				gs.debug("Inside AppStoreQuoteRequestDAO.getBillingShippingAddress");

				if (!source_app_id) {
					return {
						error: "Source app id missing"
					};
				}
				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("company", companyId);
				gr.addQuery("application.source_app_id", source_app_id);
				gr.addQuery("created_for", "purchase");
				gr.addQuery("status", "sent");
				gr.query();
				if (gr.getRowCount() == 0) {
					return {
						error: "No Quote Request is sent"
					};
				}
				gr.next();
				var address = {};
				address.billing = gr.billto_address.toString();
				address.shipping = gr.shipto_address.toString();
				return address;
			},

			getSnShippingAddress: function (source_app_id, companyId) {
				gs.debug("Inside AppStoreQuoteRequestDAO.getSnShippingAddress");

				if (!source_app_id) {
					return {
						error: "Source app id missing"
					};
				}
				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("company", companyId);
				gr.addQuery("application.source_app_id", source_app_id);
				gr.addQuery("created_for", "purchase");
				gr.addQuery("status", "sent");
				gr.query();
				if (gr.getRowCount() == 0) {
					return {
						error: "No Quote Request is sent"
					};
				}
				gr.next();
				var sn_shipping = gr.sn_shipto_address.toString();
				return sn_shipping;
			},
			createQuote: function(data) {		
				var gr = new GlideRecord('sn_appstore_quote_request');		
				gs.debug("inserting the quote");
				
		        gr.initialize();
		        gr.application = data.app_sys_id;
		        gr.company = data.customer_company_id;
		        gr.user = data.customer_id;
		        gr.currency_code = data.currency_code;
		        gr.price_type = data.price_type;
		        gr.custom_price_type = data.custom_price_type;
		        gr.no_of_months = data.contract_term;
		        gr.contract_term = data.contract_term;
		        gr.no_of_fulfillers = data.no_of_fulfillers;
		        gr.tax = data.tax;
		        gr.payment_type = data.payment_type;
		        gr.application_price = data.application_price;
				gr.price_total = data.price_total;
		        gr.created_for = "purchase";
		        gr.lead = data.lead;
                gr.referrer = new AppStoreUtility().getReferrer(data.referrer);
		        if(data.quoteStatus){
		        	gr.status = data.quoteStatus;
		        }else{
		        	gr.status = "requested";	
		        }
		        if(data.surf_opportunity_number){
		        	gr.surf_opportunity_number = data.surf_opportunity_number;
		        }		          
		        if(data.payment_type == 'external_payment') {
		        	gr.shipto_address = data.shipto_address;
		        	gr.billto_address = data.billto_address;
		        	gr.sn_shipto_address = data.sn_shipto_address;
		        }
		        gr.insert();       
		        return gr;        
			},
			getQuote: function(sys_id) {
				var gr = new GlideRecord('sn_appstore_quote_request');
				gs.debug("finding quote by id");
				if(gr.get(sys_id)) {
					return gr;
				}
				return null;
			},
			updateQuoteStatus: function (sys_id, status) {
				var grQuote = new GlideRecord("sn_appstore_quote_request");
                grQuote.addQuery("sys_id", sys_id);
                grQuote.query();
                if (grQuote.next()) {
                    grQuote.status = status;
                    grQuote.update();
                }
			},
			updateQuote: function (sys_id, changes) {
				var key, grQuote = new GlideRecord("sn_appstore_quote_request");
                
				if(grQuote.get(sys_id)) {
                	for(key in changes) {
                		grQuote.setValue(key, changes[key]);
                	}
                	grQuote.update();
                	return grQuote;
                } else {
                	return null;
                }
			},

			isPurchaseInProgress : function(application_sys_id_list, companyId){
				gs.debug("Inside AppStoreQuoteRequestDAO.isPurchaseInProgress");

				if (!application_sys_id_list || application_sys_id_list.length == 0) {
					return null;
				}

				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("company", companyId);
				var grOR = gr.addQuery("application", application_sys_id_list[0].toString());
				for(var i = 1; i < application_sys_id_list.length; i++) {
					grOR.addOrCondition("application", application_sys_id_list[i].toString());
				}
				gr.addQuery("created_for", "purchase");
				gr.addQuery("status","NOT IN", "purchased,cancelled,replaced");
				gr.query();
				if (gr.getRowCount() == 0) {
					return null;
				}
				gr.next();
				return gr.application.version.toString();
			},

			getInProgressAppPurchases : function(args){
				gs.debug("Inside AppStoreQuoteRequestDAO.getInProgressAppPurchases");

				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("created_for", "IN", "purchase, application_purchase_renewal");
				gr.addQuery("status","NOT IN", "purchased,pending_signature,cancelled,replaced,deactivated");
				if (args.applyCompanyIdCheck) {
					gr.addQuery('application.company', args.companyId);
				}
				if (args.customerCompanyId) {
					gr.addQuery('company', args.customerCompanyId);
				}
				if (args.source_app_id) {
					gr.addQuery('application.source_app_id', args.source_app_id);
				}
				if(args.isOutofBandUser && args.snUserDetails) {				
                    gr.addQuery("application.apprepo_application.team",args.snUserDetails.teamId);
				}
				gr.query();

				var purchaseData= [];
				if (gr.getRowCount() == 0) {
					return purchaseData;
				}
				while(gr.next()) {
					var appData = {};
					appData = this.mapData(gr);
					if(appData.statusValue == "Requested" || appData.statusValue == "Resend Requested" || appData.statusValue == "Sent") {
						appData.statusValue = "Quote " + appData.statusValue;
					}
					if(appData.statusValue == "Pending Signature") {
						appData.statusValue = "Pending Customer Signature";
					}

					//appData.priceTotal = parseFloat(appData.price_total) * 12/parseInt(appData.contract_term,10);
					appData.priceTotal = parseFloat(gr.price_total.toString());
					appData.compatibilityList = new AppStoreUtility().getCompatibilityList(appData.applicationSysId);
					purchaseData.push(appData);
				}
				return purchaseData;
			},

			mapData: function(gr) {
				var appData = {};
				//Quote
				appData.currencyCode = gr.currency_code.toString();
				appData.quantity = gr.no_of_fulfillers.toString();
				appData.priceTotal = gr.price_total.toString();
				appData.expiryDate = gr.expiry_date.toString();
				appData.paymentType = gr.payment_type.toString();
				appData.paymentTypeValue = gr.payment_type.getDisplayValue();
				appData.contractTerm = gr.contract_term.toString();
				appData.status = gr.status.toString();
				appData.statusValue = gr.status.getDisplayValue();
				appData.priceTypeValue = gr.price_type.getDisplayValue();
				appData.priceType = gr.price_type.toString();
				if(appData.priceType == 'custom') {
					appData.customPriceType = gr.custom_price_type.toString();
				}

				//Company
				appData.companyName = gr.company.name.toString();
				appData.companySysId = gr.company.sys_id.toString();

				//User
				appData.userName = gr.user.name.toString();
				appData.userEmail = gr.user.email.toString();
				appData.userPhone = gr.user.phone.toString();
				appData.userSysId = gr.user.sys_id.toString();

				//Application
				appData.applicationSysId = gr.application.sys_id.toString();
				appData.title = gr.application.title.toString();
				appData.version = gr.application.version.toString();
				appData.sourceAppId = gr.application.source_app_id.toString();
				appData.applicationPrice = gr.application.price.toString();
				appData.applicationPriceType = gr.application.price_type.toString();
				appData.applicationPriceTypeValue = gr.application.price_type.getDisplayValue();
				appData.applicationType = gr.application.apprepo_application.application_type.toString();
				appData.applicationSubType = gr.application.application_type.toString();
				appData.allow_trial = gr.application.allow_trial.toString() === "true" ? true : false;
				appData.hide_buy = gr.application.hide_buy.toString() === "true" ? true : false;
				appData.is_express = gr.application.is_express.toString() === "true";

				return appData;
			},

			quoteExists : function(source_app_id, companyId){
				if (!source_app_id) {
					return {
						error: "Source app id missing"
					};
				}
				
				if (!companyId) {
					return {
						error: "Company id missing"
					};
				}

				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("company", companyId);
				gr.addQuery("application.source_app_id", source_app_id);
				gr.addQuery("created_for", "purchase");
				gr.addQuery("status",'IN', "purchased,pending_signature");
				gr.query();
				if (gr.getRowCount() > 0) {
					gr.next();
					return {"sys_id" : gr.sys_id.toString()};
				}

				return null;
			},
			deactivateQuoteRequest: function (appPurchase) {
				var source_app_id = appPurchase.source_app_id.toString();
				var companySysId = appPurchase.company.sys_id.toString();
				var quoteGr = new GlideRecord("sn_appstore_quote_request");
				quoteGr.addQuery("application.source_app_id", source_app_id);
				quoteGr.addQuery("company", companySysId);
				quoteGr.addQuery("status", "purchased");
				quoteGr.query();
				while (quoteGr.next()) {
					quoteGr.status = "deactivated";
					quoteGr.update();
				}
			},
			 getQuoteRequestByCompany: function (companyId, status) {
				var quoteList = [];
				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("status", status);
				gr.addQuery("company.sys_id", companyId);
				gr.addEncodedQuery(new StoreLanguageUtility().getEncodedQueryForAppPublishedTo("application.app_published_to"));
				gr.query();
				while (gr.next()) {
					var quote = {};
					quote.title = gr.application.title.toString();
					quote.vendorCompanyName = gr.application.company.name.toString();
					quote.created_on = gr.sys_created_on.toString();
					quote.source_app_id = gr.application.source_app_id.toString();
					quote.type = "Purchase";
					quote.category = "Quote";
					quote.version = gr.application.version.toString();
					quoteList.push(quote);
				}
				return quoteList;
			},
			getAppQuoteRequestByCompany: function (args) {
				var result = {
					"sent": false,
					"po_submitted": false,
					"requested": false
				};
				var gr = new GlideRecord("sn_appstore_quote_request");
				gr.addQuery("company.sys_id", args.companyId);
				gr.addQuery("application.source_app_id", args.sourceAppId);
				gr.query();
				while(gr.next()) {
					if(result.hasOwnProperty(gr.status.toString()))
						result[gr.status] = true;
				}
				return result;
			},

			type: 'TestDAO'
		};
		
"""

In [48]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(tc_4)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

print(time_taken)
print(tokens_count)

input prompt size: 5130
//Test case 1: Test bulkTriggerPerformanceRun function
var testPerformanceUtil = new TestPerformanceUtil();
var expectedDateTime = new <PERSON>();
var perfTestRunGr = new GlideRecord('sn_test_management_performance_test_run');
perfTestRunGr.initialize();
perfTestRunGr.setValue('u_status', 'Queued for Execution');
perfTestRunGr.setValue('u_job_name', 'test_job');
perfTestRunGr.setValue('u_test_cycle', 'test_cycle');
perfTestRunGr.setValue('u_test_execution_suite', 'test_execution_suite');
perfTestRunGr.setValue('u_test_set', 'test_set');
perfTestRunGr.setValue('sys_id', 'test_sys_id');
perfTestRunGr.insert();
var result = testPerformanceUtil.bulkTriggerPerformanceRun();
perfTestRunGr = new GlideRecord('sn_test_management_performance_test_run');
perfTestRunGr.addQuery('sys_id', 'test_sys_id');
perfTestRunGr.query();
if (perfTestRunGr.next()) {
  var actualDateTime = perfTestRunGr.u_build_number;
  assertEqual(expectedDateTime, actualDateTime);
}

//Test case 2: Te

In [45]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(tc_5)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

print(time_taken)
print(tokens_count)

input prompt size: 3317


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(


// Test case 1: Testing _getLicenseStatus function
<PERSON>.initialize();
var record = new GlideRecord('table_name');
record.setValue('license_status', 'In <PERSON>');
var result = <PERSON>._getLicenseStatus(record);
assertEqual(result, CapMapConstants.PIPELINE_VALUE);

// Test case 2: Testing _getCurrentBIStatus function
var usage = 'High Use';
var result = <PERSON>._getCurrentBIStatus(usage);
assertEqual(result, CapMapConstants.IN_USE_COL_STATUS_VALUE);

// Test case 3: Testing _isThereDeltaInTheData function
var capRecord = new GlideRecord('table_name');
var usageRecord = new GlideRecord('table_name');
var result = <PERSON>._isThereDeltaInTheData(capRecord, usageRecord);
assertFalse(result);

// Test case 4: Testing _isThereDeltaInTheLicenseStatus function
var capRecord = new GlideRecord('table_name');
var usageRecord = new GlideRecord('table_name');
capRecord.setValue('license_status', 'Licensed');
var result = <PERSON>._isThereDeltaInTheLicenseStatus(capRecord, usageRecord);
asser

In [51]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(tc_6)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

print(time_taken)
print(tokens_count)

input prompt size: 5185
//Test cases for TestDAO class

describe('TestDAO', function() {
  
  //Test case for getQuoteRequestsToBeExpired method
  it('should return an array of quote requests to be expired', function() {
    var dao = new TestDAO();
    var noOfDaysBeforeExpiry = 7;
    var createdFor = 'purchase';
    var result = dao.getQuoteRequestsToBeExpired(noOfDaysBeforeExpiry, createdFor);
    expect(result).toBeInstanceOf(Array);
  });
  
  //Test case for getQuoteRequestsExpired method
  it('should return an array of quote requests expired', function() {
    var dao = new TestDAO();
    var noOfDaysAfterExpiry = 7;
    var createdFor = 'purchase';
    var result = dao.getQuoteRequestsExpired(noOfDaysAfterExpiry, createdFor);
    expect(result).toBeInstanceOf(Array);
  });
  
  //Test case for _constructAppStoreQuotRequestGRData method
  it('should return an object with quote request data', function() {
    var dao = new TestDAO();
    var appStoreQuoteRequesetGR = new GlideRe

In [49]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(tc_7)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

print(time_taken)
print(tokens_count)

final_list:8
input prompt size: 2954
input prompt size: 1764
input prompt size: 2448
input prompt size: 1952
input prompt size: 1984
input prompt size: 2456
input prompt size: 3284
input prompt size: 3042


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(


// Test case: Testing _getObjectiveStateStatusColorOptions function
var businessObjective = new BusinessObjective();
var param = "someParam";
var result = businessObjective._getObjectiveStateStatusColorOptions(param);
// Assert statements for result

// Test case: Testing _beautifyChoiceLabels function
var businessObjective = new BusinessObjective();
var choiceArr = [{label: "label1", value: "value1"}, {label: "label2", value: "value2"}];
var result = businessObjective._beautifyChoiceLabels(choiceArr);
// Assert statements for result

// Test case: Testing getHtmlLabel function
var businessObjective = new BusinessObjective();
var obj = {label: "label", value: "value"};
var result = businessObjective.getHtmlLabel(obj);
// Assert statements for result

// Test case: Testing _getHtmlLabel function
var businessObjective = new BusinessObjective();
var chObj = {label: "label", value: "value"};
var result = businessObjective._getHtmlLabel(chObj);
// Assert statements for result

// Test case:

In [50]:
print(response)

// Test case: Testing getBusinessObjectiveDetail function with valid businessObjSysId
var businessObjSysId = '12345';
var expectedResult = {
    id: '12345',
    title: 'Business Objective Title',
    description: 'Business Objective Description',
    custom_label: '',
    related_products: 'Product A, Product B',
    last_updatedOn_objDetails: '2021-01-01',
    reference_number: {
        value: '',
        displayValue: ''
    },
    start_date: {
        value: '',
        displayValue: '2021-01-01'
    },
    completion_date: {
        value: '',
        displayValue: '2021-02-01'
    },
    target_completion_date: {
        value: '',
        displayValue: '2021-03-01'
    },
    cancelled_date: {
        value: '',
        displayValue: '2021-04-01'
    },
    created_date: {
        value: '',
        displayValue: '2021-01-01'
    },
    status: {
        displayValue: 'In Progress',
        value: 'in_progress',
    },
    state: {
        displayValue: 'Active',
        value

In [31]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(small_code)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

input prompt size: 245
//Test case to check if ImpactSPMIntegrationService is defined
describe("ImpactSPMIntegrationService", function() {
  it("should be defined", function() {
    expect(ImpactSPMIntegrationService).toBeDefined();
  });
});

//Test case to check if initialize function is defined
describe("ImpactSPMIntegrationService", function() {
  describe("initialize", function() {
    it("should be defined", function() {
      var impactSPMIntegrationService = new ImpactSPMIntegrationService();
      expect(impactSPMIntegrationService.initialize).toBeDefined();
    });
  });
});

//Test case to check if SPM_ENTITY_LINKAGE_DETAIL_TABLE is defined
describe("ImpactSPMIntegrationService", function() {
  describe("initialize", function() {
    it("should define SPM_ENTITY_LINKAGE_DETAIL_TABLE", function() {
      var impactSPMIntegrationService = new ImpactSPMIntegrationService();
      expect(impactSPMIntegrationService.SPM_ENTITY_LINKAGE_DETAIL_TABLE).toBeDefined();
    });
  });
})

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(


In [28]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(medium_code)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 4000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 4000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

input prompt size: 3791
describe('TestGenerationUtil', function() {
    var testGenerationUtil;

    beforeEach(function() {
        testGenerationUtil = new TestGenerationUtil();
    });

    describe('getTableId', function() {
        it('should return the table ID for a given table name', function() {
            var tableName = 'sys_script_include';
            var expectedTableId = '1234567890';

            spyOn(GlideRecord, 'get').and.returnValue({
                getUniqueValue: function() {
                    return expectedTableId;
                }
            });

            var tableId = testGenerationUtil.getTableId(tableName);

            expect(tableId).toBe(expectedTableId);
            expect(GlideRecord.get).toHaveBeenCalledWith('sys_db_object');
            expect(GlideRecord.addQuery).toHaveBeenCalledWith('name', tableName);
            expect(GlideRecord.query).toHaveBeenCalled();
            expect(GlideRecord.next).toHaveBeenCalled();
        });

        it

In [34]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(large_code)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 4000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 4000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

final_list:3
input prompt size: 3358
input prompt size: 3039
input prompt size: 3585
var AutoDeployRESTFramework = Class.create();
AutoDeployRESTFramework.prototype = {
    _getConstantsForUpdateDeploymentStepInsJob: function(additionalData) {
        gs.log('tejaram --> ' + JSON.stringify(additionalData));
        if (JSUtil.nil(additionalData.stepInstanceID)) {
            gs.error('AutoDeployRESTFramework._getConstantsForUpdateDeploymentStepInsJob -> Invalid parameters: additionalData.stepInstanceID: ' + additionalData.stepInstanceID);
            return;
        }
        var stepInstanceID = additionalData.stepInstanceID,
            devopsInstance = additionalData.devopsInstance,
            baseRestEndPoint, jobName, payload, viewName;
        if (JSUtil.nil(additionalData.devopsInstance) || (ITCEDevOpsConstants.PROD_ORCHESTRATION_INSTANCE === devopsInstance)) {
            gs.log('tejaram --> additionalData.devopsInstance:  ' + additionalData.devopsInstance + ' devopsInstance :

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [21]:
tc_7 = """
var BusinessObjective = Class.create();
BusinessObjective.TABLE_BUSINESS_OBJECTIVE = 'x_snc_octane_bvmp_business_objective_data';
BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MEMBER = 'x_snc_octane_bvmp_objective_member_data';
BusinessObjective.TABLE_BUSINESS_OBJECTIVE_INITIATIVE = 'x_snc_octane_bvmp_objective_initiative_instance_data';
BusinessObjective.TABLE_BUSINESS_OBJECTIVE_OUTCOME = 'x_snc_octane_bvmp_objective_outcome_instance_data';
BusinessObjective.TABLE_BUSINESS_OBJECTIVE_OUTCOME_METRIC = 'x_snc_octane_bvmp_outcome_metric_instance_data';
BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MASTER = 'x_snc_octane_bvmp_business_objective';
BusinessObjective.TABLE_CORE_COMPANY = 'core_company';
BusinessObjective.TABLE_OUTCOME = 'x_snc_octane_bvmp_value_outcome';
BusinessObjective.TABLE_TIME_PERIOD = 'x_snc_octane_bvmp_time_period';

BusinessObjective.prototype = {
	initialize: function () { },

	getBusinessObjectiveDetail: function(businessObjSysId) {
		var businessObjectiveGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
		if (businessObjectiveGr.canRead()) {
			businessObjectiveGr.get(businessObjSysId);
			var lastUpdatedObj = this.getOutcomesLatestUpdationTime(businessObjectiveGr.getValue('sys_id'));
			var _objColorVal = businessObjectiveGr.getValue('objective_color') || '';
			var _objColorDisp = businessObjectiveGr.objective_color.getDisplayValue() || '';
			if (_objColorVal == '') {
				var randomCol = this._getRandomColor();
				_objColorVal = randomCol.value;
				_objColorDisp = randomCol.label;
			}

			var businessObject = {
				id: businessObjectiveGr.getValue('sys_id'),
				title: businessObjectiveGr.getDisplayValue('title') || '', //BA SFSTRY0052114: recapture localization changes.
				description: businessObjectiveGr.getDisplayValue('description') || '',
				custom_label: businessObjectiveGr.user_defined ? 'Custom' : '',
				related_products: businessObjectiveGr.products.getDisplayValue(),
				last_updatedOn_objDetails:lastUpdatedObj.updated_on ? lastUpdatedObj.updated_on.toString().split(' ')[0]: '',
				reference_number: {
					value: '',
					displayValue: businessObjectiveGr.getValue('number')
				},
				start_date: {
					value: '',
					displayValue: businessObjectiveGr.getValue('start_date')
				},
				completion_date: {
					value: '',
					displayValue: businessObjectiveGr.getValue('completion_date')
				},
				target_completion_date: {
					value: '',
					displayValue: businessObjectiveGr.getValue('targeted_completion_date')
				},
				cancelled_date: {
					value: '',
					displayValue: businessObjectiveGr.getValue('cancelled_date')
				},
				created_date: {
					value: '',
					displayValue: businessObjectiveGr.getValue('sys_created_on').substring(0, 10)
				},
				status: {
					displayValue: businessObjectiveGr.getDisplayValue('overall_status'),
					value: businessObjectiveGr.getValue('overall_status'),
				},
				state: {
					displayValue: businessObjectiveGr.getDisplayValue('progress_state'),
					value: businessObjectiveGr.getValue('progress_state')
				},

				last_updated: lastUpdatedObj.updated_on ? this._timeSince(lastUpdatedObj.updated_on) : '',
				last_updated_by: lastUpdatedObj.updated_by ? lastUpdatedObj.updated_by : '',
				//last_updatedOn_objDetails: lastUpdatedObj.updated_on.split(' ')[0],
				//last_updatedOn_objDetails: lastUpdatedObj.updated_on ? lastUpdatedObj.updated_on.toString().split(' ')[0]: '',   
				/*SFSTRY0025721 -- Souvik*/
				color: {
					value: _objColorVal,
					label: _objColorDisp
				},
				/*SFSTRY0025721 -- Souvik*/
				owner: {
					value: businessObjectiveGr.getValue('owner') || '',
					displayValue: businessObjectiveGr.getDisplayValue('owner') || ''
				},
				shg_id: gs.nil(businessObjectiveGr.stakeholder_group) ? '' : businessObjectiveGr.getValue('stakeholder_group')
			};

			return businessObject;
		}

	},

	/**
	 *  Author: swati.swati
	 *  Method to fetch latest updated time for outcomes of related objective
	 **/
	getOutcomesLatestUpdationTime: function (obj_sys_id) {
		var rec = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_OUTCOME_METRIC);
		rec.addEncodedQuery('outcome.objective.sys_id=' + obj_sys_id + '^ORDERBYDESCsys_updated_on');
		rec.query();
		if (rec.next()) {

			return {
				updated_on: rec.getValue('sys_updated_on'),
				updated_by: new global.ImpactCommonUtils().getUserSysId(rec.getValue('sys_updated_by'))
			};
		}
		return '';
	},

	getEditBusinessObjectiveDetail: function (businessObjSysId) {
		var objectiveRec = this.getBusinessObjectiveDetail(businessObjSysId);
		/* SFSTRY0037307 -- Ahmad */
		var businessObjectiveGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
		businessObjectiveGr.get(businessObjSysId);
		var baseObjectiveName = businessObjectiveGr.business_objective_reference ? businessObjectiveGr.business_objective_reference.title.toString() : '';
		var isUserDefined = businessObjectiveGr.user_defined != null ? businessObjectiveGr.user_defined == 0 ? false :
		businessObjectiveGr.user_defined : true;
		var driver = {
			value: businessObjectiveGr.getValue('driver'),
			displayValue: businessObjectiveGr.driver.title.toString()
		};

		var tempProducts = businessObjectiveGr.products ? businessObjectiveGr.products.split(',') : [];
		var productTitles = [];
		var productGr = new GlideRecord('x_snc_cusp_business_unit');
		for (var i = 0; i < tempProducts.length; i++) {
			productGr.get(tempProducts[i]);
			productTitles.push(productGr.getValue('name'));
		}

		var products = {
			value: businessObjectiveGr.getValue('products'),
			displayValue: productTitles.join(',')
		};

		var objectiveMasterRec = businessObjectiveGr.business_objective_reference;
		var bizObj = {
			baseObjectiveName: baseObjectiveName,
			products: products,
			driver: driver,
			isUserDefined: isUserDefined,
			pageType: 'business objective',
			title: {
				label: gs.getMessage('oct-edit-business-objective-title'),
				value: objectiveRec.title,
				required: true,
			},
			startDate: {
				label: gs.getMessage('oct-vjb-bo-start-date'),
				value: objectiveRec.start_date ? objectiveRec.start_date.displayValue : '',
				required: true,
			},
			targetDate: {
				label: gs.getMessage('oct-vjb-bo-target-completion-date'),
				value: objectiveRec.target_completion_date ? objectiveRec.target_completion_date.displayValue : '',
				required: true,
			},
			description: {
				label: gs.getMessage('oct-vjb-bo-description'),
				value: objectiveRec.description,
				required: true,
			},
			status: {
				label: gs.getMessage('oct-vjb-bo-status'),
				value: this._getObjectiveStateStatusColorOptions('overall_status'),
				/*SFSTRY0025721 -- Souvik*/
				required: true,
				disabled: true,
				selected: {
					label: objectiveRec.status ? objectiveRec.status.displayValue : '',
					value: objectiveRec.status ? objectiveRec.status.value : ''
				}
			},
			state: {
				label: gs.getMessage('impact-vb-edit-bo-progressstate'),
				value: this._getObjectiveStateStatusColorOptions('progress_state'),
				/*SFSTRY0025721 -- Souvik*/
				required: true,
				selected: {
					label: objectiveRec.state ? objectiveRec.state.displayValue : '',
					value: objectiveRec.state ? objectiveRec.state.value : ''
				}
			},
			/*SFSTRY0025721 -- Souvik*/
			driver_color: {
				label: gs.getMessage('oct-vjb-bo-objective-color'),
				value: this._beautifyChoiceLabels(this._getObjectiveStateStatusColorOptions('objective_color')),
				selected: {
					value: objectiveRec.color.value,
					label: this.getHtmlLabel(objectiveRec.color)
				}
			},
			/*SFSTRY0025721 -- Souvik*/
			owner: {
				role: {
					label: gs.getMessage('Owner'),
					value: 'owner'
				},
				user: [objectiveRec.owner.value],
				userDisplay: objectiveRec.owner.displayValue,
				sys_id: objectiveRec.owner.value
			},
			/* SFSTRY0037307 -- Ahmad */
			objective_def: {
				title: objectiveMasterRec && objectiveMasterRec.title.getDisplayValue() || '',
				product: objectiveMasterRec && objectiveMasterRec.products.getDisplayValue() || '',
				description: objectiveMasterRec && objectiveMasterRec.description.getDisplayValue() || ''
			}
		};

		return bizObj;
	},

	getStartBusinessObjectiveDetail: function (businessObjMasterSysId) {
		var objectiveRec = this.getBusinessObjectiveMaster(businessObjMasterSysId);
		var objectiveColor;
		var objectiveColors = this._getObjectiveStateStatusColorOptions('objective_color');

		if (objectiveRec.driver && objectiveRec.driver.colour) {

			for (var i in objectiveColors) {
				if (objectiveColors[i].value == objectiveRec.driver.colour.toString()) {
					objectiveColor = {};
					objectiveColor.label = objectiveColors[i].label;
					objectiveColor.value = objectiveColors[i].value;
					break;
				}
			}
		}

		/*
		 * Set date for Objectivies
		 * SFSTRY0037307 -- Ahmad
		 */
		var today = new Date();
		var date = today.getFullYear() + '-' + (today.getMonth() + 1) + '-' + today.getDate();
		var time = today.getHours() + ':' + today.getMinutes() + ':' + today.getSeconds();
		var dateTime = date + ' ' + time;
		var gdt = new GlideDateTime(dateTime);
		var todaysDate = gdt.getDate().getValue();
		gdt.addYearsLocalTime(1);
		var futureDate = gdt.getDate().getValue();
		var businessObjectiveGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
		businessObjectiveGr.get(businessObjMasterSysId);
		var isUserDefined = businessObjectiveGr.user_defined != null ? businessObjectiveGr.user_defined == 0 ? false :
		businessObjectiveGr.user_defined : true;
		var baseObjectiveName = businessObjectiveGr.business_objective_reference ? businessObjectiveGr.business_objective_reference.title.toString() : objectiveRec.title ? objectiveRec.title.toString() : '';

		var bizObj = {
			baseObjectiveName: baseObjectiveName,
			isUserDefined: isUserDefined,
			pageType: 'business objective',
			title: {
				label: gs.getMessage('oct-edit-business-objective-title'),
				value: objectiveRec.title ? objectiveRec.title.toString() : '',
				required: true,
			},
			startDate: {
				label: gs.getMessage('oct-vjb-bo-start-date'),
				value: todaysDate,
				required: true,
			},
			targetDate: {
				label: gs.getMessage('oct-vjb-bo-target-completion-date'),
				value: futureDate,
				required: true,
			},
			description: {
				label: gs.getMessage('oct-vjb-bo-description'),
				value: objectiveRec.description ? objectiveRec.description.toString() : '',
				required: true,
			},
			products: {
				label: gs.getMessage('Products'),
				value: objectiveRec.products ? objectiveRec.products.toString() : '',
				displayValue: objectiveRec.products ? objectiveRec.products.getDisplayValue() : '',
				required: false
			},
			status: {
				label: gs.getMessage('oct-vjb-bo-status'),
				value: this._getObjectiveStateStatusColorOptions('overall_status'),
				required: true,
				disabled: false,
				selected: {
					label: '',
					value: ''
				}
			},
			state: {
				label: gs.getMessage('oct-vjb-bo-state'),
				value: this._getObjectiveStateStatusColorOptions('progress_state'),
				required: true,
				selected: {
					label: '',
					value: ''
				}
			},
			driver: {
				label: gs.getMessage('Driver'),
				value: objectiveRec.driver ? objectiveRec.driver.toString() : '',
				color: objectiveRec.driver ? objectiveRec.driver.colour : ''
			},
			driver_color: {
				label: gs.getMessage('oct-vjb-bo-objective-color'),
				selected: {
					value: objectiveColor ? objectiveColor.value : objectiveColors[0].value,
					label: objectiveColor ? this.getHtmlLabel(objectiveColor) : this.getHtmlLabel(objectiveColors[0])
				},
				value: this._beautifyChoiceLabels(objectiveColors)
			},
			owner: {
				role: {
					label: gs.getMessage('Owner'),
					value: 'owner'
				},
				user: [],
				userDisplay: '',
				sys_id: ''
			},
			/* SFSTRY0037307 -- Ahmad */
			objective_def: {
				title: objectiveRec.title ? objectiveRec.title.getDisplayValue() : '',
				product: objectiveRec.products ? objectiveRec.products.getDisplayValue() : '',
				description: objectiveRec.description ? objectiveRec.description.getDisplayValue() : ''
			}
		};

		bizObj.status.selected.label = bizObj.status.value[0].label;
		bizObj.status.selected.value = bizObj.status.value[0].value;
		bizObj.state.selected.label = bizObj.state.value[0].label;
		bizObj.state.selected.value = bizObj.state.value[0].value;

		return bizObj;
	},

	getBusinessObjectiveMaster: function (businessObjMasterSysId) {
		var masterObjGR = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MASTER);
		masterObjGR.get(businessObjMasterSysId);

		return masterObjGR;
	},

	/*SFSTRY0025721 -- Souvik*/
	_setRandomColor: function () {
		var color = this._getRandomColor().value;
		return color;
	},

	_getRandomColor: function () {
		var choiceArr = this._getObjectiveStateStatusColorOptions('objective_color');
		var retIndex = Math.floor(Math.random() * choiceArr.length);
		return choiceArr[retIndex];
	},

	_getObjectiveStateStatusColorOptions: function (param) {
		var choices = [];
		var chceGr = new GlideRecord('sys_choice');
		chceGr.addEncodedQuery('name=' + BusinessObjective.TABLE_BUSINESS_OBJECTIVE + '^inactive=false^element=' + param);
		chceGr.orderBy('sequence');
		chceGr.addQuery('language', new global.OctaneUserUtil().getCurrentUserLanguage());
		chceGr.query();
		while (chceGr.next()) {
			choices.push({
				label: chceGr.getValue('label'),
				value: chceGr.getValue('value'),
			});
		}
		return choices;
	},

	_beautifyChoiceLabels: function (choiceArr) {
		for (var i in choiceArr) {
			var _label = this._getHtmlLabel(choiceArr[i]);
			choiceArr[i].label = _label;
		}
		return choiceArr;
	},

	getHtmlLabel: function (obj) {
		var _obj = {};
		_obj.label = obj.label;
		_obj.value = obj.value;

		return this._getHtmlLabel(_obj);
	},

	_getHtmlLabel: function (chObj) {
		return '<i class="fa fa-circle ' + chObj.value + '"></i>' + chObj.label;
	},

	/*SFSTRY0025721 -- Souvik*/
	// get business objective outcomes with metrics data
	getBusinessObjectiveOutcomes: function (businessObjSysId, customerId, isCustomer) {
		var bizOutcomesWithMetrics = [];

		var bizOutcomes = this._getBusinessOutcomes(businessObjSysId, null, null);
		var assessmentObj = this._getOutcomeValueRealised(businessObjSysId, customerId, isCustomer);
		var outcomeValRealised = assessmentObj.outcomeValueRealisedMap;
		var timePeriod = assessmentObj.timePeriod;
		var isObjectivePartOfDraftAssessmnt = assessmentObj.isObjectivePartOfDraftAssessmnt;
		var hasObjectiveBeenPublished = assessmentObj.hasObjectiveBeenPublished;

		for (var i = 0; i < bizOutcomes.length; i++) {

			var valueRealised = '';
			outcomeValRealised.forEach(function (item) {
				if (Object.keys(item) == bizOutcomes[i].sys_id) {
					valueRealised = item[Object.keys(item)];
					return;
				}
			});

			bizOutcomesWithMetrics.push({
				sys_id: bizOutcomes[i].sys_id,
				short_description: bizOutcomes[i].outcome,
				title: bizOutcomes[i].title,
				outcome_name: bizOutcomes[i].outcome_name,
				description: bizOutcomes[i].description,
				status: this._getStatusColor(bizOutcomes[i].progress_value),
				valueRealised: valueRealised,
				timePeriod: timePeriod,
				isObjectivePartOfDraftAssessmnt: isObjectivePartOfDraftAssessmnt,
				hasObjectiveBeenPublished: hasObjectiveBeenPublished,
				successMetric: this._getSuccessMetrics(bizOutcomes[i].sys_id, assessmentObj),
				custom_label: bizOutcomes[i].custom_label,
				quarterlyValue: bizOutcomes[i].quarterlyValue,
				selected: bizOutcomes[i].selected,
				canBeNormalized : bizOutcomes[i].canBeNormalized, 
				isNormalized : bizOutcomes[i].isNormalized
			});
		}

		return bizOutcomesWithMetrics;
	},

	/*SFSTRY0031912 - Swati

	*/
	_getOutcomeValueRealised: function (businessObjSysId, customerId, isCustomer) {

		var assessments = new x_snc_octane_bvmp.ImpactValueScanUtil().fetchAssessmentDetails(customerId).assessments;
		var outcomeValueRealisedMap = [];
		var timePeriod = '';
		var hasObjectiveBeenPublished = false;
		var isObjectivePartOfDraftAssessmnt = false;
		var objectOutcomeMetricData = [];
		Object.keys(assessments).forEach(function (key) { // iterate through assessments
			if (key != 0) {
				var objectives = assessments[key].objectives;

				objectives.forEach(function (kObj) { // iterate through all the objectives in that assessment
					if (kObj.sys_id != businessObjSysId) return; // if the objetiveid matches,iterate through all the outcomes of that objective

					var conditionCheck = isCustomer ? assessments[key].state == 'Published' : (assessments[key].state == 'Published' || assessments[key].state == 'Draft');
					if (conditionCheck && timePeriod.localeCompare(assessments[key].time_period, undefined, { numeric: true }) == -1) {
						outcomeValueRealisedMap = [];
						timePeriod = assessments[key].time_period;
						hasObjectiveBeenPublished = assessments[key].state == 'Published';
						isObjectivePartOfDraftAssessmnt = assessments[key].state == 'Draft';
						var outcomes = kObj.outcomes;
						outcomes.forEach(function (kOtcm) {
							var otcmValuePairObj = {};
							otcmValuePairObj[kOtcm.sys_id] = kOtcm.value;
							outcomeValueRealisedMap.push(otcmValuePairObj);
						});
					}
					objectOutcomeMetricData.push(kObj);
				});
			}
		});

		var assessmentObj = {
			timePeriod: timePeriod,
			outcomeValueRealisedMap: outcomeValueRealisedMap,
			isObjectivePartOfDraftAssessmnt: isObjectivePartOfDraftAssessmnt,
			hasObjectiveBeenPublished: hasObjectiveBeenPublished,
			outcomeMetricData: objectOutcomeMetricData
		};

		return assessmentObj;
	},

	// get header details of business objective
	getBusinessObjectiveHeader: function (objectiveSysId) {
		// var boHeaderDetailObj = {};
		var businessObjectiveGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
		businessObjectiveGr.get(objectiveSysId);

		return {
			driver_label: businessObjectiveGr.getDisplayValue('driver'),
			original_name: businessObjectiveGr.getDisplayValue('business_objective_reference'),
			driver_icon: businessObjectiveGr.driver && businessObjectiveGr.driver.icon && businessObjectiveGr.driver.icon.toString() || '',
			title: businessObjectiveGr.getValue('title'),
			driver_color: businessObjectiveGr.getValue('objective_color') || 'blue', // note: business objective color field was created later - use this instead of driver color.
			refNo: businessObjectiveGr.getValue('number'),
			shg_id: gs.nil(businessObjectiveGr.stakeholder_group) ? '' : businessObjectiveGr.getValue('stakeholder_group'),
			stakeHolderGroup: gs.nil(businessObjectiveGr.stakeholder_group) ? '': businessObjectiveGr.getDisplayValue('stakeholder_group'),
			created_date: businessObjectiveGr.getValue('sys_created_on').substring(0, 10),
			user_defined: businessObjectiveGr.getValue('user_defined'),
			custom_label: businessObjectiveGr.user_defined ? 'CUSTOM' : ''
		};
	},

	_constructListFieldFilter: function(filterKey, filterValues){
		var finalFilterValue = '';
		var count = 0;
		filterValues.forEach(function(item, index){
			if (index < filterValues.length - 1){
				finalFilterValue += filterKey + 'LIKE' + item + '^OR';
			}
			else{
				finalFilterValue += filterKey + 'LIKE' + item;
			}			
		});
		return finalFilterValue;
	},
	/**
	 *  Author: Ram
	 *  Applies SHG Filter 
	 **/
	applySHGFilter: function(tableGR,parameters){
		var stakeHolderGroups = parameters.stakeHolderGroups;
		if(stakeHolderGroups && stakeHolderGroups.length>0){
			var shg_gr ;
			for(var shg_indx = 0;shg_indx<stakeHolderGroups.length;shg_indx++){
				var shg_val = stakeHolderGroups[shg_indx] == 'Others' ? '' : stakeHolderGroups[shg_indx] ;

				if(shg_indx == 0){
					shg_gr = tableGR.addQuery("stakeholder_group",'=',shg_val);
				}else{
					shg_gr.addOrCondition("stakeholder_group",'=',shg_val);
				}
			}
		}
		return tableGR;
	},
	/**
	 *  Author: Rajeev.Jayaswal
	 *  Method to fetch list of business objectives for a given company
	 **/
	getBusinessObjectives: function (companySysId, params) {
		
		var businessObjectives = [];
		var filterByAtOutcomeLevel = [];
		if (params) {
			var count = 0, filterValue = '', filterByEncodedQuery = '';			
			for (key in params.filterBy){
				filterValue = params.filterBy[key];
				//if we want to filter at outcome level store it, and pass it to _getBusinessObjective() when it's called
				//and at its turn into _getBusinessOutcomes()
				if(params && params.filterAtOutcomeLevel && key == 'products') {
					if (Array.isArray(filterValue)){
						filterByAtOutcomeLevel = filterValue;
					}
				}
				else if (filterValue !== ''){
					if (count > 0){
						filterByEncodedQuery += '^';
					}
					//if we are looking at products
					if (Array.isArray(filterValue)){
						filterByEncodedQuery += this._constructListFieldFilter(key, filterValue);
						// Consider user defined BOs mostly don't have products
						filterByEncodedQuery += '^ORproductsISEMPTY';
					}
					else{
						if (filterValue.indexOf('=') > -1){
							filterByEncodedQuery += key + filterValue;
						}
						else{
							filterByEncodedQuery += key + 'IN' + filterValue;		
						}					
					}
					count++;
				}
			}

			if (params.searchQuery){
				if (filterByEncodedQuery){
					filterByEncodedQuery += '^titleLIKE' + params.searchQuery + '^ORbusiness_objective_reference.titleLIKE' + params.searchQuery;
				}
				else{
					filterByEncodedQuery += 'titleLIKE' + params.searchQuery + '^ORbusiness_objective_reference.titleLIKE' + params.searchQuery;					
				}
			}

			// we have custom logic to order for sort by Completion date label
			var isCustomGroupByState = false;

			var sortByEncodedQuery = '';
			if (params.sortBy && params.sortBy.length > 0) {
				for (i = 0; i < params.sortBy.length; i++) {
					if (params.sortBy[i].value != 'CUSTOM_GROUP_BY_STATE') {
						if (i > 0)
							sortByEncodedQuery += '^';
						sortByEncodedQuery += params.sortBy[i].value;
					} else {
						isCustomGroupByState = true;
					}
				}
			}
		}
		var encodedQuery = '';
		// default - show active records from user's company
		encodedQuery = 'active=true^company=' + companySysId;

		if (filterByEncodedQuery) {
			encodedQuery += '^' + filterByEncodedQuery;
		}

		if (sortByEncodedQuery) {
			encodedQuery += '^' + sortByEncodedQuery;
		}

		if (isCustomGroupByState == false) {
			var businessObjectiveGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
			businessObjectiveGr.addEncodedQuery(encodedQuery);
			businessObjectiveGr = this.applySHGFilter(businessObjectiveGr,params);
			businessObjectiveGr.query();

			// var businessObjectiveList = [];
			while (businessObjectiveGr.next()) {
				if(params && params.filterAtOutcomeLevel) {
					var objectiveToPush = this._getBusinessObjective(businessObjectiveGr, filterByAtOutcomeLevel);
					if(objectiveToPush != null) {
						businessObjectives.push(objectiveToPush);
					}
				} else {
					businessObjectives.push(this._getBusinessObjective(businessObjectiveGr, null));
				}
			}
		} else { // isCustomGroupByState == true
			/**
				SFSTRY0022021 - Logic for custom sorting
				Completion date (in reverse chronological order):
					1) Target completion date (reverse chronological target completion date followed by those with empty target completion date, AND state = not completed or cancelled)
					2) Completed date (reverse chronological date completed, and state = completed)
					3) "Cancelled" (but the field is Completed date) (reverse chronological date completed, state = cancelled)
					i.e. group by state[], and sort by target completion date, completion date
				**/

			var businessObjectiveGr1 = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
			var subEncodedQuery = encodedQuery + '^' + 'progress_stateINnot_started,on_hold,work_in_progress^ORDERBYDESCtargeted_completion_date';
			businessObjectiveGr1.addEncodedQuery(subEncodedQuery);
			businessObjectiveGr1 = this.applySHGFilter(businessObjectiveGr1,params);

			businessObjectiveGr1.query();

			while (businessObjectiveGr1.next()) {
				if(params && params.filterAtOutcomeLevel) {
					var objectiveToPush = this._getBusinessObjective(businessObjectiveGr1, filterByAtOutcomeLevel);
					if(objectiveToPush != null) {
						businessObjectives.push(objectiveToPush);
					}
				} else {
					businessObjectives.push(this._getBusinessObjective(businessObjectiveGr1, null));
				}
			}

			var businessObjectiveGr2 = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
			subEncodedQuery = encodedQuery + '^' + 'progress_state=complete^ORDERBYDESCcompletion_date';
			businessObjectiveGr2.addEncodedQuery(subEncodedQuery);
			businessObjectiveGr2 = this.applySHGFilter(businessObjectiveGr2,params);

			businessObjectiveGr2.query();

			while (businessObjectiveGr2.next()) {
				if(params &&  params.filterAtOutcomeLevel) {
					var objectiveToPush = this._getBusinessObjective(businessObjectiveGr2, filterByAtOutcomeLevel);
					if(objectiveToPush != null) {
						businessObjectives.push(objectiveToPush);
					}
				} else {
					businessObjectives.push(this._getBusinessObjective(businessObjectiveGr2, null));
				}
			}

			var businessObjectiveGr3 = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
			subEncodedQuery = encodedQuery + '^' + 'progress_state=cancelled^ORDERBYDESCcompletion_date';
			businessObjectiveGr3.addEncodedQuery(subEncodedQuery);
			businessObjectiveGr3 = this.applySHGFilter(businessObjectiveGr3,params);

			businessObjectiveGr3.query();

			while (businessObjectiveGr3.next()) {
				if(params &&  params.filterAtOutcomeLevel) {
					objectiveToPush = this._getBusinessObjective(businessObjectiveGr3, filterByAtOutcomeLevel);
					if(objectiveToPush != null) {
						businessObjectives.push(objectiveToPush);
					}
				} else {
					businessObjectives.push(this._getBusinessObjective(businessObjectiveGr3, null));
				}
			}

		}

		return businessObjectives;
	},

	/**
	 Author: Rajeev.Jayaswal
	 getBusinessObjectivesCount: get active objectives count for a given company
	 companySysId: company sys_id
	 return count of records
	*/
	getBusinessObjectivesCount: function (companySysId) {
		var businessObjectiveGr = new GlideAggregate(x_snc_octane_bvmp.BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
		var encodedQuery = 'active=true^company=' + companySysId;
		businessObjectiveGr.addEncodedQuery(encodedQuery);
		businessObjectiveGr.addAggregate('COUNT');
		businessObjectiveGr.query();
		businessObjectiveGr.next();

		return businessObjectiveGr.getAggregate('COUNT');

	},

	_getBusinessObjective: function (businessObjectiveGr, filterByAtOutcomeLevel) {
		var business_outcomes = [];
		if(filterByAtOutcomeLevel == null) {
			business_outcomes = this._getBusinessOutcomes(businessObjectiveGr.getUniqueValue(), null, null);
		} else {
			business_outcomes = this._getBusinessOutcomes(businessObjectiveGr.getUniqueValue(), filterByAtOutcomeLevel, businessObjectiveGr);
			if(business_outcomes.length == 0) {
				var outcomeGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_OUTCOME);
				outcomeGr.addQuery('objective', businessObjectiveGr.getUniqueValue());
				outcomeGr.addQuery('progress', '!=', 'deleted');
				outcomeGr.query();
				if (!outcomeGr.hasNext() && !gs.nil(filterByAtOutcomeLevel)) {
					var productFound = false;
					if (filterByAtOutcomeLevel.length != 0) {
						productFound = false;
						for (var i = 0; i < filterByAtOutcomeLevel.length; i++) {
							if (filterByAtOutcomeLevel[i] != '' && businessObjectiveGr.products.indexOf(filterByAtOutcomeLevel[i]) != -1) {
								productFound = true;
								break;
							}
						}
					} else {
						productFound = true;
					}
					if (!productFound) {
						return null;
					}
				} else {
					return null;
				}
			}
		}
		
		return {
			sys_id: businessObjectiveGr.getUniqueValue(),
			header: {
				driver_label: businessObjectiveGr.getDisplayValue('driver'),
				original_name: businessObjectiveGr.getDisplayValue('business_objective_reference'),
				driver_icon: businessObjectiveGr.driver && businessObjectiveGr.driver.icon && businessObjectiveGr.driver.icon.toString() || '',
				driver_color: businessObjectiveGr.getValue('objective_color') || 'blue', // default color: blue
				title: businessObjectiveGr.getValue('title'),
				created_date: businessObjectiveGr.getValue('sys_created_on').substring(0, 10),
				description: businessObjectiveGr.getValue('description'),
				objective_owner: businessObjectiveGr.getValue('owner'),
				executive_sponsor: this._getExecutiveSponsor(businessObjectiveGr.getUniqueValue()),
				custom_label: businessObjectiveGr.user_defined ? 'CUSTOM' : '',
				shg: gs.nil(businessObjectiveGr.stakeholder_group) ? '' : businessObjectiveGr.stakeholder_group.getDisplayValue(),
				shg_id: gs.nil(businessObjectiveGr.stakeholder_group) ? '' : businessObjectiveGr.getValue('stakeholder_group')
			},
			summary: {
				overall_status: {
					value: businessObjectiveGr.getValue('overall_status'), // green, yellow, red
					displayValue: businessObjectiveGr.getDisplayValue('overall_status')
				},
				progress_state: {
					value: businessObjectiveGr.getValue('progress_state'), //on_hold, cancelled, not_started, work_in_progress
					displayValue: businessObjectiveGr.getDisplayValue('progress_state')
				},
				start_date: businessObjectiveGr.getValue('start_date'),
				completion_date: businessObjectiveGr.getValue('completion_date'),
				cancelled_date: businessObjectiveGr.getValue('cancelled_date'),
				products: businessObjectiveGr.products.getDisplayValue(),
				target_completion_date: businessObjectiveGr.getValue('targeted_completion_date'),
				initiative_overview: this._getInitiativeOverview(businessObjectiveGr.getUniqueValue()),
				business_outcomes: business_outcomes 
			}

		};
	},

	/*Author: Hasan Bilgrami
	Method to fetch list of business objectives
	 updated by: Archana Kulkarni
	*/
	getBusinessObjectiveMasterList: function (params) {
		var businessObjectives = [];
		var searchBy = '';
		if (params) {

			var count = 0, filterValue = '', filterByEncodedQuery = '';			
			for (key in params.filterBy){
				filterValue = params.filterBy[key];
				if (filterValue !== ''){
					if (count > 0){
						filterByEncodedQuery += '^';
					}
					if (Array.isArray(filterValue)){
						filterByEncodedQuery += this._constructListFieldFilter(key, filterValue);
						// Consider user defined BOs mostly don't have products - taking this off as new objectives grid shows BO with products
						//filterByEncodedQuery += '^ORproductsISEMPTY';
					}
					else{
						if (filterValue.indexOf('=') > -1){
							filterByEncodedQuery += key + filterValue;
						}
						else{
							filterByEncodedQuery += key + 'IN' + filterValue;		
						}					
					}
					count++;
				}
			}
			if (params.searchQuery){//(params.searchTerm && params.searchTerm.length > 0) {
				searchBy = params.searchQuery;
			}
			var searchByEncodedQuery = '';
			if (searchBy !== undefined && searchBy !== null && searchBy !== '') {
				searchByEncodedQuery = 'titleLIKE' + searchBy;
			}

			var sortByEncodedQuery = '';
			if (params.sortBy && params.sortBy.length > 0) {
				for (i = 0; i < params.sortBy.length; i++) {
					if (i > 0)
						sortByEncodedQuery += '^';
					sortByEncodedQuery += params.sortBy[i].value;
				}
			}


		}

		/* SFSTRY0037307 -- Ahmad */
		var encodedQuery = '';
		var businessObjectiveDataGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
		var encodedQueryObjData = 'active=true^progress_state!=cancelled^progress_state!=complete';
		encodedQueryObjData += '^business_objective_reference!=NULL^company=' + params.customerID;
		var skippedObjectivesQuery = '';
		businessObjectiveDataGr.addEncodedQuery(encodedQueryObjData);
		businessObjectiveDataGr.query();
		while (businessObjectiveDataGr.next()) {
			skippedObjectivesQuery += '^sys_id!=' + businessObjectiveDataGr.business_objective_reference;
		}
		encodedQuery = 'active=true^is_latest=true' + skippedObjectivesQuery;

		if (filterByEncodedQuery) {
			encodedQuery += '^' + filterByEncodedQuery;
		}

		if (sortByEncodedQuery) {
			encodedQuery += '^' + sortByEncodedQuery;
		}

		if (searchByEncodedQuery) {
			encodedQuery += '^' + searchByEncodedQuery;
		}

		var businessObjectiveGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MASTER);
		businessObjectiveGr.addEncodedQuery(encodedQuery);
		businessObjectiveGr.query();

		/*var companyPackage = null;
		var company = new GlideRecordSecure(BusinessObjective.TABLE_CORE_COMPANY);
		if (company.get(params.customerID)) {
			companyPackage = company.x_snc_cusp_success_program;
		}*/

		while (businessObjectiveGr.next()) {
			var businessObjectiveSysId = businessObjectiveGr.getValue('sys_id');
			var outcomesGr = new GlideRecordSecure(BusinessObjective.TABLE_OUTCOME);
			outcomesGr.addQuery('objective', businessObjectiveSysId);
			//outcomesGr.addQuery("success_program", "CONTAINS", companyPackage);
			outcomesGr.query();
			if (outcomesGr.hasNext()) {
				var businessObject = {
					sys_id: businessObjectiveSysId,
					driver_label: businessObjectiveGr.getDisplayValue('driver'),
					title: businessObjectiveGr.getDisplayValue('title'),
					description: businessObjectiveGr.getDisplayValue('description'),
					products: businessObjectiveGr.getValue('products') ? businessObjectiveGr.getDisplayValue('products').split(', ') : [],
					importance: businessObjectiveGr.getValue('u_importance'),
					outcomes: [],
					version: 'V3', //** As of Versioning is not implemented- SFSTRY0048375 - Need to update in future **//
					url_1: businessObjectiveGr.getValue('url_1'),
					url_2: businessObjectiveGr.getValue('url_2'),
					url_3: businessObjectiveGr.getValue('url_3'),
					url_4: businessObjectiveGr.getValue('url_4'),
					url_5: businessObjectiveGr.getValue('url_5'),
					url_6: businessObjectiveGr.getValue('url_6'),
					url_7: businessObjectiveGr.getValue('url_7'),
					url_8: businessObjectiveGr.getValue('url_8'),
					url_9: businessObjectiveGr.getValue('url_9'),
					url_10: businessObjectiveGr.getValue('url_10'),
				};
				while (outcomesGr.next()) {
					businessObject.outcomes.push(outcomesGr.title.getDisplayValue());
				}
				businessObjectives.push(businessObject);
			}

		}

		return businessObjectives;
	},

	// <avishek.dey, 9 Dec 21>, Commenting out the code below as
	// to get the active and latest business objectives, use the query is_latest=true^active=true
	//Author: Hasan Bilgrami
	//Get Latest Version of Business Objective
	// getBusinessObjectiveLatestVersion: function () {
	//     var businessObjectiveAggregate = new GlideAggregate(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MASTER);
	//     businessObjectiveAggregate.addAggregate('MAX', 'version');
	//     businessObjectiveAggregate.setGroup(false);
	//     businessObjectiveAggregate.query();
	//     if (businessObjectiveAggregate.next())
	//         return businessObjectiveAggregate.getAggregate('MAX', 'version');
	// },
	// </avishek.dey, 9 Dec 21>

	//Author: Hasan Bilgrami
	//returns array of filter objects containing active Value drivers
	//  updated by: Archana Kulkarni - filter by was changed to products
	getNewBusinessObjectiveFilterOptions: function () {
		var filteredList = [];
		var prodMap = {};

		var bizUnitGR = new GlideRecord('x_snc_octane_bvmp_business_objective');
		bizUnitGR.addActiveQuery();
		bizUnitGR.addQuery('is_latest', 'true');
		bizUnitGR.query();
		while (bizUnitGR.next()) {
			var myProdList = bizUnitGR.products;
			var currList = myProdList.split(',');
			for (var i = 0; i < currList.length; i++) {
				var prodId = currList[i];
				if (!prodMap[prodId]) {
					var prodNameGR = new GlideRecord('x_snc_cusp_business_unit');
					prodNameGR.get(prodId);
					var pName = prodNameGR.name.toString();
					prodMap[prodId] = pName;
				}
			}
		}

		var arr = Object.keys(prodMap);
		for (var j = 0; j < arr.length; j++) {
			var localkey = arr[j];
			filteredList.push({
				selected: false,
				label: prodMap[localkey],
				value: localkey,
			});
		}

		return filteredList;
	},
	// Author: Rajeev.Jayaswal
	// Returns exective sponsor for a given business objective
	_getExecutiveSponsor: function (businessObjectiveSysId) {
		var memberGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MEMBER);
		memberGr.addQuery('business_objective', businessObjectiveSysId);
		memberGr.addQuery('type', 'executive_sponsor'); //DFCT0148139 BA - To avoid wrong selection of user from watchlist.
		memberGr.query();

		if (memberGr.next())
			return memberGr.getValue('user');

		return '';
	},

	_getInitiativeOverview: function (businessObjectiveSysId) {
		// group by state
		var initiativeStates = {
			complete: 0,
			work_in_progress: 0,
			on_hold: 0,
			not_started: 0,
			cancelled: 0
		};

		var initiativeGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_INITIATIVE);
		initiativeGr.addQuery('objective', businessObjectiveSysId);
		initiativeGr.addActiveQuery();
		initiativeGr.query();

		while (initiativeGr.next()) {
			var state = initiativeGr.initiative && initiativeGr.initiative.state ? initiativeGr.initiative.state + '' : '';
			switch (state) {
				case '100':
					initiativeStates.complete++;
					break;
				case '1':
					initiativeStates.work_in_progress++;
					break;
				case '2':
					initiativeStates.on_hold++;
					break;
				case '0':
					initiativeStates.not_started++;
					break;
				case '-10':
					initiativeStates.cancelled++;

			}
		}

		var initiativeOverview = [];
		for (state in initiativeStates) {
			// Do not include initiative with 0 count records
			if (initiativeStates[state] != 0) {
				switch (state) {
					case 'complete':
						initiativeOverview.push(initiativeStates[state] + ' ' + gs.getMessage('oct-business-objective-initiative-complete'));
						break;
					case 'work_in_progress':
						initiativeOverview.push(initiativeStates[state] + ' ' + gs.getMessage('oct-business-objective-initiative-wip'));
						break;
					case 'on_hold':
						initiativeOverview.push(initiativeStates[state] + ' ' + gs.getMessage('oct-business-objective-initiative-on-hold'));
						break;
					case 'not_started':
						initiativeOverview.push(initiativeStates[state] + ' ' + gs.getMessage('oct-business-objective-initiative-not-started'));
						break;
					case 'cancelled':
						initiativeOverview.push(initiativeStates[state] + ' ' + gs.getMessage('oct-business-objective-initiative-cancelled'));
				}
			}
		}

		return initiativeOverview;
	},

	// business_outcomes: [{outcome:'', progress_value:''}]
	//second argument to decide whenever to filter at outcome level
	_getBusinessOutcomes: function (businessObjectiveSysId, filterByAtOutcomeLevel, businessObjectiveGr) {
		var business_outcomes = [];		
		
		var outcomeGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_OUTCOME);
		outcomeGr.addQuery('objective', businessObjectiveSysId);
		outcomeGr.addQuery('outcome.state', 'published').addOrCondition('user_defined', true);
		outcomeGr.addQuery('progress', '!=', 'deleted');
		outcomeGr.addQuery('');
		outcomeGr.orderBy('title');
		outcomeGr.query();

		while (outcomeGr.next()) {
			var productFound = false;
			if(filterByAtOutcomeLevel != null && filterByAtOutcomeLevel.length != 0) {
				productFound = false;
				for(var i=0; i< filterByAtOutcomeLevel.length; i++) {
					//if it's custome, filter at objective level
					if(outcomeGr.user_defined) {
						if(filterByAtOutcomeLevel[i] != '' && 
							businessObjectiveGr.products.indexOf(filterByAtOutcomeLevel[i]) != -1) {
							productFound = true;
							break;
						}
					}
					//if the product list is just empty, it's also valid
// 					if(outcomeGr.outcome.product.length == 0) {
// 						productFound = true;
// 						break;
// 					}
					if(filterByAtOutcomeLevel[i] != '' &&
						outcomeGr.outcome.product && 
						outcomeGr.outcome.product.indexOf(filterByAtOutcomeLevel[i]) != -1) {
						productFound = true;
						break;
					}
				}
			}
			if( (filterByAtOutcomeLevel != null && filterByAtOutcomeLevel.length != 0 && productFound) 
				|| (filterByAtOutcomeLevel == null || filterByAtOutcomeLevel.length == 0) ){
				
				business_outcomes.push({
					sys_id: outcomeGr.getUniqueValue(),
					outcome: outcomeGr.getDisplayValue('outcome') || outcomeGr.getDisplayValue('title'),
					outcome_name: outcomeGr.getDisplayValue('outcome'),
					description: outcomeGr.outcome.description.toString(),
					progress_value: outcomeGr.getValue('progress'), // complete, in_progress, not_started, on_hold
					custom_label: outcomeGr.user_defined ? 'CUSTOM' : '',
					title: outcomeGr.getDisplayValue('title'),
					quarterlyValue: outcomeGr.getDisplayValue('quarterly_value'),
					outcome_product: outcomeGr.outcome.product.getDisplayValue(),
					canBeNormalized : outcomeGr.outcome.normalizable ? true : false,
                    isNormalized : outcomeGr.normalized ? true: false, 
					success_metric: this._getSuccessMetric(outcomeGr.sys_id, outcomeGr.user_defined)
				});
			}
		}

		return business_outcomes;
	},
	_getSuccessMetric: function (outcomeSysID, userDefined) {	
		var metricGR = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_OUTCOME_METRIC );
		metricGR.addQuery("outcome", outcomeSysID);
		metricGR.query();
		if(metricGR.next()){
			if(!userDefined){
				return metricGR.metric.getDisplayValue();
			}
			if(userDefined){
				return metricGR.title.getDisplayValue();
			}
		}
		return "";
	},
	// get color status based on progress value of outcome instance data
	_getStatusColor: function (progress_value) {
		switch (progress_value) {
			case 'complete':
				return 'green';
			case 'in_progress':
				return 'gray';
			case 'not_started':
				return 'yellow';
			case 'on_hold':
				return 'red';
		}

	},


	// get success metrics

	_getSuccessMetrics: function (outcomeSysId) {
		var metrics = [];
		// var metricsData = [];
		var current = this._getSuccessMetricCurrentValueFromMetricParams(outcomeSysId);

		var metricGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_OUTCOME_METRIC);
		metricGr.addQuery('outcome', outcomeSysId);
		metricGr.orderBy('sys_created_on');
		metricGr.query();

		if (metricGr.next()) {
			if (metricGr.getValue('unit')) {
				metrics.push({
					name: metricGr.user_defined ? metricGr.getDisplayValue('title') : metricGr.getDisplayValue('metric'),
					baseline: metricGr.getValue('baseline') ? metricGr.getValue('baseline') + ' ' + metricGr.getValue('unit') : ' ',
					current: current,
					goal: metricGr.getValue('goal') ? metricGr.getValue('goal') + ' ' + metricGr.getValue('unit') : ' '
				});
			} else {
				metrics.push({
					name: metricGr.user_defined ? metricGr.getDisplayValue('title') : metricGr.getDisplayValue('metric'),
					baseline: metricGr.getValue('baseline') + ' ',
					current: current,
					goal: metricGr.getValue('goal') + ' '
				});
			}
		}

		return metrics;
	},

	_getSuccessMetricCurrentValueFromMetricParams: function (outcomeSysId) {
		var metricParameterGr = new GlideRecord('x_snc_octane_bvmp_metric_parameter');
		metricParameterGr.addEncodedQuery('outcome_metric!=NULL^outcome_metric.metric_type=primary^outcome_metric.is_baseline=false^NQoutcome_metric=NULL^is_baseline=false');
		metricParameterGr.addQuery('scan_outcome.outcome_data_reference', outcomeSysId);
		metricParameterGr.orderByDesc('scan_outcome.value_assessment_reference.time_period');
		metricParameterGr.query();
		var currentValue = '';
		if (metricParameterGr.next()) {
			currentValue = metricParameterGr.getDisplayValue('value');
		}
		return currentValue;
	},

	/**
	 * Author: Rajeev.Jayaswal
	 * param: GlideDateTime string value, ex: 1988-05-31 09:30:00
	 * return string in time ago format, ex: 33 years.
	 */
	_timeSince: function (glideDateTime) {
		var gdt = new GlideDateTime(glideDateTime);
		date = gdt.getNumericValue();

		if (typeof date !== 'object') {
			date = new Date(date);
		}

		var seconds = Math.floor((new Date() - date) / 1000);
		var intervalType;

		var interval = Math.floor(seconds / 31536000);
		if (interval >= 1) {
			intervalType = 'year';
		} else {
			interval = Math.floor(seconds / 2592000);
			if (interval >= 1) {
				intervalType = 'month';
			} else {
				interval = Math.floor(seconds / 86400);
				if (interval >= 1) {
					intervalType = 'day';
				} else {
					interval = Math.floor(seconds / 3600);
					if (interval >= 1) {
						intervalType = 'hour';
					} else {
						interval = Math.floor(seconds / 60);
						if (interval >= 1) {
							intervalType = 'minute';
						} else {
							interval = seconds;
							intervalType = 'second';
						}
					}
				}
			}
		}

		if (interval > 1 || interval === 0) {
			intervalType += 's';
		}
		//2022-03-21 BA DFCT0156332: For Localization make sure in sys_messages year/years/month/months .. message should exist. Based on final key(singular/plural) localization will work.
		intervalType = gs.getMessage(intervalType);

		return interval + ' ' + intervalType;
	},

	saveBusinessObjective: function (objectiveSysId, objectiveObj) {
		return this.saveBusinessObjectiveDetail(objectiveSysId, objectiveObj.detail) && this.saveBusinessObjectiveMembers(objectiveSysId, objectiveObj.teamMembers, objectiveObj.watchlist);
	},

	saveBusinessObjectiveDetail: function (objectiveSysId, detail) {
		var newOwner = detail.owner.toString() || "";
		var objectiveDataGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
		objectiveDataGr.get(objectiveSysId);
		objectiveDataGr.title = detail.title.value;
		objectiveDataGr.start_date = detail.startDate.value;
		objectiveDataGr.targeted_completion_date = detail.targetDate.value;
		objectiveDataGr.description = detail.description.value;
		objectiveDataGr.progress_state = detail.state.selected.value;
		objectiveDataGr.overall_status = detail.status.selected.value;
		objectiveDataGr.owner = detail.owner;
		objectiveDataGr.objective_color = detail.driver_color.selected.value; /*SFSTRY0025721 -- Souvik*/
		objectiveDataGr.driver = detail.driver.value;
		objectiveDataGr.products = detail.products.value;
		objectiveDataGr.update();
		return true;
	},

	saveBusinessObjectiveMembers: function (objectiveSysId, members, watchlist) {
		var objectiveWatchListRole = new x_snc_octane_sp.OctaneImpactRoleUtils();
		if (watchlist && watchlist.value) {
			for (var i = 0; i < watchlist.value.length; i++) {
				members.push({
					sys_id: watchlist.sys_id[i] ? watchlist.sys_id[i] : '',
					user: [watchlist.value[i] ? watchlist.value[i] : ''],
					userDisplay: watchlist.displayValue[i] ? watchlist.displayValue[i] : '',
					role: {
						label: 'Watchlist',
						value: 'watchlist'
					}

				});
			}
		}
		this._deleteUnnecessaryRecords(objectiveSysId, members);

		if (members) {
			for (i = 0; i < members.length; i++) {
				role = '';
				if (members[i].role) {
					role = members[i].role.value ? members[i].role.value : '';
				}

				if (members[i].sys_id) {
					var objectiveDataGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MEMBER);
					//objectiveDataGr.addEncodedQuery('sys_id =' + members[i].sys_id);
					objectiveDataGr.addEncodedQuery('business_objective=' + objectiveSysId + '^type=' + role + '^user=' + members[i].user[0]);
					objectiveDataGr.query();
					if (objectiveDataGr.next()) {
						objectiveDataGr.user = members[i].user[0] ? members[i].user[0] : '';
						objectiveDataGr.type = role;
						objectiveDataGr.update();
					}
				} else {
					objectiveDataGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MEMBER);
					objectiveDataGr.initialize();
					objectiveDataGr.user = members[i].user[0] ? members[i].user[0] : '';
					objectiveDataGr.type = role;
					objectiveDataGr.business_objective = objectiveSysId;
					var objMember = false;
					objMember = (objectiveWatchListRole.isImpactExecutive(members[i].user[0]) && members[i].role.label == 'Watchlist' || objectiveWatchListRole.isImpactExecutive(members[i].user[0]) && members[i].role.label == 'Secondary Member');
					if (!objMember && objectiveDataGr.canCreate()) {
						objectiveDataGr.insert();

					}
				}
			}
		}
		return true;
	},

	_deleteUnnecessaryRecords: function (objectiveSysId, members) {
		var objectiveDataGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MEMBER);
		objectiveDataGr.addEncodedQuery('business_objective =' + objectiveSysId);
		objectiveDataGr.query();
		while (objectiveDataGr.next()) {
			var shouldBeDeleted = true;
			var storeSysId = objectiveDataGr.getValue('sys_id');
			if (members && members.length > 0) {
				for (var i = 0; i < members.length; i++) {

					if (members[i].sys_id && members[i].sys_id == storeSysId) {
						shouldBeDeleted = false;
						break;
					}
				}
			}
			if (shouldBeDeleted && objectiveDataGr.canDelete()) {
				objectiveDataGr.deleteRecord();
			}
		}
		return true;
	},

	getObjectiveSquadTeamMembers: function (customerId) {
		var squad = {
			teamMembers: [],
			watchlist: []
		};
		var teamMembersValue = [];
		var teamMembersDisplayValue = [];
		var teamMembersRecSysIds = [];
		var teamMembersRoles = [];

		var squadMembers = new x_snc_cusp.SquadUserUtil().getSquadMembersForCompany(customerId);

		// Set appropriate data
		for (var members in squadMembers) {
			squad.teamMembers.push({
				sys_id: '',
				user: [squadMembers[members].sysId.toString()],
				userDisplay: squadMembers[members].name.toString(),
				role: {
					label: '',
					value: ''
				}
			});

			teamMembersValue.push(squadMembers[members].sysId.toString());
			teamMembersDisplayValue.push(squadMembers[members].name.toString());
			teamMembersRecSysIds.push(squadMembers[members].sysId.toString());
			teamMembersRoles.push({
				label: '',
				value: ''
			});

		}

		squad.watchlist = {
			value: teamMembersValue,
			displayValue: teamMembersDisplayValue,
			sys_id: teamMembersRecSysIds,
			role: teamMembersRoles
		};

		return squad;
	},

	getObjectiveTeamMembers: function (objectiveSysId) {

		var users = {
			teamMembers: [],
			watchlist: []
		};
		var teamMembersValue = [];
		var teamMembersDisplayValue = [];
		var teamMembersRecSysIds = [];
		var teamMembersRoles = [];

		var objectiveTeamMemberGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MEMBER);
		objectiveTeamMemberGr.addEncodedQuery('business_objective=' + objectiveSysId);
		objectiveTeamMemberGr.query();
		while (objectiveTeamMemberGr.next()) {
			if (objectiveTeamMemberGr.getValue('type') != 'watchlist' && objectiveTeamMemberGr.getValue('type') != 'NULL') {
				users.teamMembers.push({
					sys_id: objectiveTeamMemberGr.getUniqueValue(),
					user: [objectiveTeamMemberGr.getValue('user')],
					userDisplay: objectiveTeamMemberGr.user.first_name.toString() + ' ' + objectiveTeamMemberGr.user.last_name.toString(),
					role: {
						label: objectiveTeamMemberGr.getDisplayValue('type'),
						value: objectiveTeamMemberGr.getValue('type')
					}
				});
			} else {
				teamMembersValue.push(objectiveTeamMemberGr.getValue('user'));
				teamMembersDisplayValue.push(objectiveTeamMemberGr.user.first_name.toString() + ' ' + objectiveTeamMemberGr.user.last_name.toString());
				teamMembersRecSysIds.push(objectiveTeamMemberGr.getUniqueValue());
				teamMembersRoles.push({
					label: objectiveTeamMemberGr.getDisplayValue('type'),
					value: objectiveTeamMemberGr.getValue('type')
				});
			}
		}

		users.watchlist = {
			value: teamMembersValue,
			displayValue: teamMembersDisplayValue,
			sys_id: teamMembersRecSysIds,
			role: teamMembersRoles
		};

		return users;
	},

	getObjectiveTeamMembersRoleChoices: function () {
		var choices = [];
		var objectiveTeamMemberGr = new GlideRecord('sys_choice');
		objectiveTeamMemberGr.addEncodedQuery('name=x_snc_octane_bvmp_objective_member_data^element=type^inactive=false');
		objectiveTeamMemberGr.addQuery('language', new global.OctaneUserUtil().getCurrentUserLanguage());
		objectiveTeamMemberGr.query();
		while (objectiveTeamMemberGr.next()) {
			if (objectiveTeamMemberGr.getValue('value') != 'watchlist' && objectiveTeamMemberGr.getValue('value') != 'NULL') {
				choices.push({
					label: objectiveTeamMemberGr.getValue('label'),
					value: objectiveTeamMemberGr.getValue('value'),
					multipleAllowed: true
				});
			}
		}

		//Owner field is part of the x_snc_octane_bvmp_business_objective_data table, but the option has to be shown on the edit objective
		//team members widget so that customers can change the owner
		choices.push({
			label: gs.getMessage('Owner'),
			value: 'owner'
		});

		return choices;
	},

	getObjectiveOwner: function (objId) {
		var objGr = new GlideRecordSecure(BusinessObjective.TABLE_BUSINESS_OBJECTIVE);
		objGr.addQuery('sys_id', objId);
		objGr.addNotNullQuery('owner');
		objGr.query();
		if (objGr.next())
			return [objGr.getValue('owner')];

		return [];
	},

	getBizObjMembers: function (objId) {
		var userList = [{
			role: gs.getMessage('Objective owner'),
			users: this.getObjectiveOwner(objId),
			displayInline: false
		}];
		var users = {};
		var objectiveTeamMemberGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MEMBER);
		objectiveTeamMemberGr.addQuery('business_objective', objId);
		objectiveTeamMemberGr.addNotNullQuery('user');
		objectiveTeamMemberGr.orderBy('type');
		objectiveTeamMemberGr.query();
		while (objectiveTeamMemberGr.next()) {
			if (!users.hasOwnProperty(objectiveTeamMemberGr.type.getDisplayValue()))
				users[objectiveTeamMemberGr.type.getDisplayValue()] = [];

			users[objectiveTeamMemberGr.type.getDisplayValue()].push(objectiveTeamMemberGr.getValue('user'));
		}

		if (!users[gs.getMessage('oct-start-conversation-watchlist-label')])
			users[gs.getMessage('oct-start-conversation-watchlist-label')] = [];

		for (var key in users) {
			var _obj = {};
			_obj.role = key;
			_obj.users = users[key];
			_obj.displayInline = (key == gs.getMessage('oct-start-conversation-watchlist-label'));

			userList.push(_obj);
		}
		return userList;
	},
	/**
	 * avishek.dey, SFSTRY0026404, Oct 11 2021
	 * Method to create a business objective with the sent data.
	 * @param data JSON - Create business objective fields
	 */
	createBusinessObjective: function (data) {
		var objectiveGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE_MASTER),
			recordCreated = false,
			sysId = '';

		if (data && this._validateBusinessObjectiveData(data) && objectiveGr.canCreate()) {
			objectiveGr.initialize();
			objectiveGr.setValue('title', data.title);
			objectiveGr.setValue('active', true);
			objectiveGr.setValue('description', data.description);
			objectiveGr.setValue('driver', data.driver);
			objectiveGr.setValue('products', data.products);
			objectiveGr.setValue('base_version', data.baseVersion);
			objectiveGr.setValue('version', data.version);
			objectiveGr.setValue('is_latest', true);
			objectiveGr.setValue('active', true);
			sysId = objectiveGr.insert();
			
			gs.info('BusinessObjective::createBusinessObjective::Business objective created');
			recordCreated = true;
		} else {
			gs.info('BusinessObjective::createBusinessObjective::Cannot create business objective without valid inputs');
		}

		return {
			recordCreated: recordCreated,
			sysId: sysId
		};
	},

	createBusinessObjectiveData: function (data) {
		var objectiveOwnerRole = new x_snc_octane_sp.OctaneImpactRoleUtils();
		var objectiveDataGr = new GlideRecord(BusinessObjective.TABLE_BUSINESS_OBJECTIVE),
			recordCreated = false,
			sysId = '';

		if (data && objectiveDataGr.canCreate()) {
			objectiveDataGr.initialize();
			objectiveDataGr.title = data.detail.title.value;
			objectiveDataGr.company = data.company.toString();
			if (data.business_objective_reference.toString() == 'custom') {
				objectiveDataGr.user_defined = true;
			} else {
				objectiveDataGr.user_defined = false;
				objectiveDataGr.business_objective_reference = data.business_objective_reference.toString();
			}
			objectiveDataGr.driver = data.detail.driver.value;
			objectiveDataGr.active = true;
			objectiveDataGr.start_date = data.detail.startDate.value;
			objectiveDataGr.targeted_completion_date = data.detail.targetDate.value;
			objectiveDataGr.description = data.detail.description.value;
			objectiveDataGr.progress_state = data.detail.state.selected.value;
			objectiveDataGr.overall_status = data.detail.status.selected.value;
			objectiveDataGr.products = data.detail.products.value.toString();
			if (!objectiveOwnerRole.isImpactExecutive(data.detail.owner.sys_id)) {
				objectiveDataGr.owner = data.detail.owner.sys_id;
			}
			objectiveDataGr.objective_color = data.detail.driver_color.selected.value;
			sysId = objectiveDataGr.insert();
			
			// Save Members after creating record
			this.saveBusinessObjectiveMembers(sysId, data.teamMembers, data.watchlist);

			gs.info('BusinessObjective::createBusinessObjectiveData::Business objective data created');
			recordCreated = true;
		} else {
			gs.info('BusinessObjective::createBusinessObjectiveData::Cannot create business objective data without valid inputs');
		}

		return {
			recordCreated: recordCreated,
			sysId: sysId
		};
	},

	/**
	 * Method to validate the mandatory or conditional values for business objectives
	 * avishek.dey, SFSTRY0026404, Oct 22 2021
	 * @param {object} data - Business Objective fields
	 * @returns boolean
	 */
	_validateBusinessObjectiveData: function (data) {
		var isValid = false;
		if (data.title && data.products && data.driver && data.description) {
			isValid = true;
		}
		return isValid;
	},

	addToObjectiveWatchlist: function (objectiveId, userId) {

		var objectiveWatchListRole = new x_snc_octane_sp.OctaneImpactRoleUtils();
		var teamGr = new GlideRecordSecure('x_snc_octane_bvmp_objective_member_data');
		teamGr.addQuery('business_objective', objectiveId);
		teamGr.addQuery('user', userId);
		teamGr.addQuery('type', 'watchlist');
		teamGr.query();
		if (!teamGr.hasNext()) {
			teamGr.initialize();
			teamGr.business_objective = objectiveId;
			teamGr.type = 'watchlist';
			teamGr.user = userId;
			if (!objectiveWatchListRole.isImpactExecutive(userId) && teamGr.canCreate()) {
				teamGr.insert();

			}
		}
	},

	removeFromObjectiveWatchlist: function (objectiveId, userId) {
		var teamGr = new GlideRecord('x_snc_octane_bvmp_objective_member_data');
		teamGr.addQuery('business_objective', objectiveId);
		teamGr.addQuery('user', userId);
		teamGr.addQuery('type', 'watchlist');
		teamGr.query();
		if (teamGr.next() && teamGr.canDelete())
			teamGr.deleteRecord();
	},

	/**
	 * @author Ahmad Maarouf
	 * @description Gets outcome details to populate outcome modal.
	 * Modified 'getBUKpi' script in ImpactValueScanUtil().
	 * @story SFSTRY0039686
	 * @param {outcomeSysID} Outcome instance data sys-id.
	 * @param {companySysID} Company sys-id.
	 */
	getBusinessObjectiveOutcomeDetails: function (outcomeSysID, companySysID) {
		var kpi_service = new x_snc_octane_bvmp.ImpactValueScanUtil();
		var outcome = {};
		var index = 0;
		var scanOutcomeGR = new GlideRecordSecure('x_snc_octane_bvmp_value_scan_outcome');
		var encondequery = 'outcome_data_reference.sys_id=' + outcomeSysID + '^ORDERBYoutcome_data_reference.sys_id^ORDERBYDESCvalue_assessment_reference.time_period';
		scanOutcomeGR.addEncodedQuery(encondequery);
		/* Fetch the 2 latest values for the outcome. The current value and its older value from the prior quarter. */
		scanOutcomeGR.setLimit(2);
		scanOutcomeGR.query();
		while (scanOutcomeGR.next()) {
			// var currentOutcome = scanOutcomeGR.outcome_data_reference.sys_id;
			var scanOutcomeSysId = scanOutcomeGR.getValue('sys_id');
			var timePeriod = scanOutcomeGR.value_assessment_reference.time_period + '';
			if (!index) {
				var businessObj = kpi_service._getBusinessObjectiveInfo(scanOutcomeGR, index);
				outcome = kpi_service._getBusinessOutcomeInfo(scanOutcomeGR, index);
				outcome.metric = {
					assessment_id: scanOutcomeGR.value_assessment_reference.sys_id,
					previous_qtr: '--'
				};
				kpi_service._assignPrimaryMetricInfoForScanOutcome(scanOutcomeSysId, outcome.metric, companySysID, timePeriod, businessObj.sys_id);
				outcome.metric.targetMetricGoalValue = kpi_service._calculateGoalValue(outcome.metric.goal, outcome.metric.baseline, outcome.metric.direction);
				outcome.metric.goal = kpi_service._calculateGoal(outcome.metric.goal, outcome.metric.baseline, outcome.metric.direction);
				kpi_service._assignDependentMetricsAndAssumptions(scanOutcomeSysId, companySysID, outcome.metric, timePeriod);
				var metric = outcome.metric;
				kpi_service._getIfAnyDependentsWithoutAggregate(metric, scanOutcomeSysId, companySysID, timePeriod);
				if (metric.override ||
					(metric.operational && metric.operational.override) ||
					(metric.business && metric.business.override)) {
					outcome.metric.enableRevVS = true;
				}
				if (!metric.current && metric.observation.value == 'N/A' && metric.business.value == 'N/A' && metric.operational.value == 'N/A') {
					outcome.white_space = true;
				}

			} else {
				var prevOutcomeQtr = {};
				prevOutcomeQtr.metric = {
					assessment_id: scanOutcomeGR.value_assessment_reference.sys_id
				};
				kpi_service._assignDependentMetricsAndAssumptions(scanOutcomeSysId, companySysID, prevOutcomeQtr.metric, timePeriod);
				outcome.metric.previous_qtr = ('business' in prevOutcomeQtr.metric) ? prevOutcomeQtr.metric.business.value : '';
			}

			index++;
		}

		return outcome;
	},

	/**
	 * @author Kasra Goravanchi
	 * @description calls getObjectiveOutcomes with formatted time period
	 *
	 * @param {customerId} Company sys-id.
	 * @param {product} Product name as string
	 * @param {year} year of the quarter
	 * @param {quarter} the quarter
	 * @return metric details with appropriete Map of objective/outcomes
	 */
	getObjectiveOutcomesUnformattedTimeperiod: function(customerId, product, year, quarter) {
		var timePeriod = year + " " + quarter;
		return this.getObjectiveOutcomes(customerId, timePeriod, product);
	},

	/**
	 * @author Kasra Goravanchi
	 * @description Gets Map of objective/outcomes for given customerId, timePeriod and product.
	 *
	 * @story SFSTRY0056020
	 * @param {customerId} Company sys-id.
	 * @param {timePeriod} time period
	 * @param {product} Product name as string
	 * @return metric details with appropriete Map of objective/outcomes
	 */
	getObjectiveOutcomes: function(customerId, timePeriod, product) {
		var objectOutcomesMap = {};
		objectOutcomesMap['objectives'] = [];
		var bizKpiUtil = new x_snc_octane_bvmp.ImpactBizKpiUtil();
		var operationalPerformanceGr = new GlideRecord(ImpactBizKPITables.Operational_Performance);
		operationalPerformanceGr.addQuery('start_quarter.title', timePeriod);
		operationalPerformanceGr.addQuery('outcome_data_reference.objective.company', customerId);
		operationalPerformanceGr.addQuery("outcome_data_reference.objective.progress_state", "work_in_progress");
		operationalPerformanceGr.addQuery("outcome_data_reference.progress", "!=","deleted");
		operationalPerformanceGr.orderBy('outcome_data_reference.title');
		if(new x_snc_octane_sp.OctaneImpactRoleUtils().isCustomerUser(gs.getUserID())) {
			operationalPerformanceGr.addQuery("state", 'published');
		}
		operationalPerformanceGr.query();

		while (operationalPerformanceGr.next() && operationalPerformanceGr.canRead()) {
			if(product == '' || (operationalPerformanceGr.outcome_data_reference.outcome && operationalPerformanceGr.outcome_data_reference.outcome.product == product)
				|| (operationalPerformanceGr.outcome_data_reference.objective && operationalPerformanceGr.outcome_data_reference.objective.products && operationalPerformanceGr.outcome_data_reference.objective.products.indexOf(product) != -1)) {
				var objectiveSys = operationalPerformanceGr.outcome_data_reference.objective.sys_id;
				var outcomeSys = operationalPerformanceGr.outcome_data_reference.sys_id;
				var objective = {
					sysId: objectiveSys.toString(),
					title: gs.nil(operationalPerformanceGr.outcome_data_reference.objective.title) ? '' : operationalPerformanceGr.outcome_data_reference.objective.title.toString(),
					title_basedOn: operationalPerformanceGr.outcome_data_reference.objective.business_objective_reference.getDisplayValue(),
					progress_state: operationalPerformanceGr.outcome_data_reference.objective.progress_state.getDisplayValue(),
					timePeriod: timePeriod,
					start_date: gs.nil(operationalPerformanceGr.outcome_data_reference.objective.start_date) ? '' : operationalPerformanceGr.outcome_data_reference.objective.start_date.toString(),
					target_completion_date: gs.nil(operationalPerformanceGr.outcome_data_reference.objective.targeted_completion_date) ? '' : operationalPerformanceGr.outcome_data_reference.objective.targeted_completion_date.toString(),
					overall_status: {
						displayValue: gs.nil(operationalPerformanceGr.outcome_data_reference.objective.overall_status) ? '' : operationalPerformanceGr.outcome_data_reference.objective.overall_status.getDisplayValue(),
						value: gs.nil(operationalPerformanceGr.outcome_data_reference.objective.overall_status) ? '' : operationalPerformanceGr.outcome_data_reference.objective.overall_status.toString(),
					},
					objective_color: gs.nil(operationalPerformanceGr.outcome_data_reference.objective.objective_color) ? '' : operationalPerformanceGr.outcome_data_reference.objective.objective_color.toString(),
					user_defined: gs.nil(operationalPerformanceGr.outcome_data_reference.objective.user_defined) ? '' : operationalPerformanceGr.outcome_data_reference.objective.user_defined.toString(),
					
					stakeholderGroup : gs.nil(operationalPerformanceGr.outcome_data_reference.objective.stakeholder_group) ? '' : operationalPerformanceGr.outcome_data_reference.objective.stakeholder_group.getDisplayValue()				
				};
				if (gs.nil(objectOutcomesMap[objective.sysId])) {
					objectOutcomesMap[objective.sysId] = [];
					objectOutcomesMap['objectives'].push(objective);
				}
				var productDetails= bizKpiUtil.getProductFullNameAndIcons(operationalPerformanceGr.outcome_data_reference.outcome.product.toString(), '');
				var productFullName= productDetails.fullName;
				objectOutcomesMap[objective.sysId].push({
					scanOutcomeSysId: operationalPerformanceGr.sys_id.toString(),
					sysId: outcomeSys.toString(),
					title: operationalPerformanceGr.outcome_data_reference.title.toString(),
					user_defined: operationalPerformanceGr.outcome_data_reference.user_defined.toString(),
					description: operationalPerformanceGr.outcome_data_reference.description.toString(),
					original_name: operationalPerformanceGr.outcome_data_reference.outcome.title.toString(),
					product: operationalPerformanceGr.outcome_data_reference.outcome.product.getDisplayValue(),
					product_fullName: productFullName,
					outcome_description:operationalPerformanceGr.outcome_data_reference.user_defined.toString()=== 'true'? operationalPerformanceGr.outcome_data_reference.description.toString():
operationalPerformanceGr.outcome_data_reference.outcome.description.toString(),
					instance_name:operationalPerformanceGr.outcome_metric_reference.instance.getDisplayValue(),
					executive_summary: operationalPerformanceGr.executive_summary.toString(),
					core_capabilities: operationalPerformanceGr.outcome_data_reference.outcome.core_capabilities.getDisplayValue(),
					metric_title: operationalPerformanceGr.outcome_data_reference.user_defined.toString()=== 'true'? operationalPerformanceGr.outcome_metric_reference.title.toString(): operationalPerformanceGr.outcome_metric_reference.metric.title.toString(),
					metric_description: operationalPerformanceGr.outcome_metric_reference.metric.description.toString(),
					value_enhancing_capabilities: gs.nil(operationalPerformanceGr.outcome_data_reference.outcome.value_enhancing_capabilities)? '': operationalPerformanceGr.outcome_data_reference.outcome.value_enhancing_capabilities.getDisplayValue(),
					driver: operationalPerformanceGr.outcome_data_reference.outcome.driver.getDisplayValue(),
					state: operationalPerformanceGr.state.getDisplayValue()
				});
				
			}

		}
		return objectOutcomesMap;

	},

	/**
	 * @author Kasra Goravanchi
	 * @description Gets Metrics details for a given scan outcome id.
	 *
	 * @story SFSTRY0056020
	 * @param {scanOutcomeSysid} Scan Outcome sys-id.
	 * @return metric details with appropriete baseline/current value
	 */
	getMerticByScanOutcome: function(operationalPerformanceId) {

		var metric = {};
		var baseline;
		var goal_direction;
		var operationalPerformanceGr = new GlideRecord(ImpactBizKPITables.Operational_Performance);
		operationalPerformanceGr.addQuery('sys_id', operationalPerformanceId); 
		operationalPerformanceGr.orderBy('sys_created_on');
		operationalPerformanceGr.query();

		if (operationalPerformanceGr.next() && operationalPerformanceGr.canRead()) {
			var referenceValue = operationalPerformanceGr.outcome_metric_reference.reference_value;
			var userDefined = operationalPerformanceGr.outcome_metric_reference.user_defined.toString();
			if(gs.nil(referenceValue)) {
				baseline = '';
			} else {
				var refValueGr = new GlideRecord(ImpactBizKPITables.Operational_Performance);
				refValueGr.addQuery('outcome_metric_reference', operationalPerformanceGr.outcome_metric_reference);
				refValueGr.addQuery('outcome_data_reference', operationalPerformanceGr.outcome_data_reference);
				refValueGr.addQuery('start_quarter.title', referenceValue.title);
				refValueGr.query();
				if (refValueGr.next() && refValueGr.canRead()) {
					baseline = refValueGr.value ? parseInt(refValueGr.value) : ''; 
				}
			}
			if (userDefined== "true"){
				goal_direction = !gs.nil(operationalPerformanceGr.outcome_metric_reference.metric_direction) ? operationalPerformanceGr.outcome_metric_reference.metric_direction.toString() : 'high';
			}
			else {
				/*
				goal_direction = operationalPerformanceGr.outcome_metric_reference != undefined && operationalPerformanceGr.outcome_metric_reference != null &&
				operationalPerformanceGr.outcome_metric_reference.metric != undefined && operationalPerformanceGr.outcome_metric_reference.metric != null &&
				operationalPerformanceGr.outcome_metric_reference.metric.positive_measurement_direction != undefined &&
				operationalPerformanceGr.outcome_metric_reference.metric.positive_measurement_direction != null ?
				operationalPerformanceGr.outcome_metric_reference.metric.positive_measurement_direction.toString() : 'high';
				*/
				goal_direction =  !gs.nil(operationalPerformanceGr.outcome_metric_reference.metric.positive_measurement_direction) ? operationalPerformanceGr.outcome_metric_reference.metric.positive_measurement_direction.toString() : 'high';
			}
		//}

			return {
				baseline: baseline,
				goalPercentage: operationalPerformanceGr.outcome_metric_reference.goal ? parseInt(operationalPerformanceGr.outcome_metric_reference.goal) : '',
				current: operationalPerformanceGr.value ? parseInt(operationalPerformanceGr.value) : '',
				goalDirection : goal_direction,
				title: operationalPerformanceGr.outcome_metric_reference.title.toString()
			};
		}


		return metric;

	},

	/**
	 * @author Kasra Goravanchi
	 * @description Gets start dates (with associated end dates) from x_snc_octane_bvmp_operational_performance
	 * for given customer
	 *
	 * @story SFSTRY0060598
	 * @param {customerId} company sysid
	 * @param {product} product syid
	 * @return start dates (with associated end dates) for operational performances
	 */
	getOperationalPerformance: function(customerId, product, stakeholderGroupId) {

		var gr = new GlideRecord(ImpactBizKPITables.Operational_Performance);
		var encodedQuery = 'valueISNOTEMPTY^outcome_data_reference.objective.company=' + customerId + '^outcome_data_reference.user_defined=false^outcome_data_reference.outcome.productLIKE' + product +
			'^NQvalueISNOTEMPTY^outcome_data_reference.objective.company=' + customerId + '^outcome_data_reference.user_defined=true^outcome_data_reference.objective.productsLIKE' + product;
		gr.addEncodedQuery(encodedQuery);
		if (!gs.nil(stakeholderGroupId) && stakeholderGroupId !== 'undefined' ){
			gr.addQuery('outcome_data_reference.objective.stakeholder_group', 'IN', stakeholderGroupId);
		}
		gr.orderBy('start_quarter.start_date'); //the first one will be the oldest
		gr.query();

		var res = {};
		res.years = [];
		var oldestYear;
		var oldestQuarter;
		var currentYear;
		var currentQuarter;

		if (gr.next() && gr.canRead()) {
			var period = gr.start_quarter.getDisplayValue();
			var yearQ = period.split(' ');
			if (yearQ.length == 2) {
				oldestYear = parseInt(yearQ[0]) + 1;
				oldestQuarter = yearQ[1];
			}

			gr = new GlideRecord(BusinessObjective.TABLE_TIME_PERIOD);
			gr.addQuery('current_period', true);
			gr.orderByDesc('start_date'); //newest
			gr.query();
			if (gr.next() && gr.canRead()) {
				period = gr.title.getDisplayValue();
				yearQ = period.split(' ');
				if (yearQ.length == 2) {
					currentYear = parseInt(yearQ[0]);
					currentQuarter = yearQ[1];
				}

				while (oldestYear < currentYear) {
					res.years.push({
						label: oldestYear.toString(),
						value: oldestYear
					});
					res[oldestYear] = [];

					if (oldestQuarter == 'Q1') {
						res[oldestYear].push({
							label: 'Q1',
							value: 1
						});
						oldestQuarter = 'Q2';
					}
					if (oldestQuarter == 'Q2') {
						res[oldestYear].push({
							label: 'Q2',
							value: 2
						});
						oldestQuarter = 'Q3';
					}
					if (oldestQuarter == 'Q3') {
						res[oldestYear].push({
							label: 'Q3',
							value: 3
						});
						oldestQuarter = 'Q4';
					}
					if (oldestQuarter == 'Q4') {
						res[oldestYear].push({
							label: 'Q4',
							value: 4
						});
						oldestQuarter = 'Q1';
					}
					oldestYear++;
				}

				res.years.push({
					label: currentYear.toString(),
					value: currentYear
				});
				res[currentYear] = [];
				if (oldestQuarter == 'Q1' && oldestQuarter != currentQuarter) {
					res[currentYear].push({
						label: 'Q1',
						value: 1
					});
					oldestQuarter = 'Q2';
				}
				if (oldestQuarter == 'Q2' && oldestQuarter != currentQuarter) {
					res[currentYear].push({
						label: 'Q2',
						value: 2
					});
					oldestQuarter = 'Q3';
				}
				if (oldestQuarter == 'Q3' && oldestQuarter != currentQuarter) {
					res[currentYear].push({
						label: 'Q3',
						value: 3
					});
					oldestQuarter = 'Q4';
				}
				if (oldestQuarter == 'Q4' && oldestQuarter != currentQuarter) {
					res[currentYear].push({
						label: 'Q4',
						value: 4
					});
					oldestQuarter = 'Q1';
				}

				res[currentYear].push({
					label: currentQuarter,
					value: parseInt(currentQuarter[1])
				});					
			}

		}

		return res;
	},

	/**
	 * @author Kasra Goravanchi
	 * @description Gets report quarter range for selected start quarter and type of compare period
	 *
	 * @story SFSTRY0060598
	 * @param {comparisonPeriod} comparison period: "oldest" | "prior"
	 * @param {startYear} start year
	 * @param {startQuarter} start quarter
	 * @param {customerId} company sysid
	 * @param {product} product syid
	 * @return quarter range
	 */
	getComparisonQuarterRange: function(comparisonPeriod, selectedStartYear, selectedStartQuarter, customerId, product, stakeholderGroupId) {

		var payload = this.getOperationalPerformance(customerId, product, stakeholderGroupId);
		var quarterRange = "--";
		var startYear;
		var startQuarter;
		var endYear;
		var endQuarter;

		if(comparisonPeriod == "oldest"){
			startYear = parseInt(payload.years[0].value)-1; //already sorted
			startQuarter = payload[payload.years[0].value][0].label;
		}
		else if(comparisonPeriod == "prior") {
			startYear = parseInt(selectedStartYear) - 1;
			startQuarter = selectedStartQuarter;
		}
		endYear = startQuarter == 'Q1' ? startYear : startYear + 1;
		endQuarter = startQuarter == 'Q1' ? 'Q4' :
		startQuarter == 'Q2' ? 'Q1' :
		startQuarter == 'Q3' ? 'Q2' :
		startQuarter == 'Q4' ? 'Q3' : '-';
		return {"start": startYear + " " + startQuarter,
				"end": endYear + " " + endQuarter};

	},

	getTimeperiod: function(customerId) {

		var timePeriods = [];
		var uniqueElements = [];
		var currentPeriod = new x_snc_octane_bvmp.ValueOperationalPerformance().getCurrentTimePeriod();
		var currentPeriodStr = currentPeriod.year + ' Q' + currentPeriod.qtr;
		var operationalPerformanceGr = new GlideRecord(ImpactBizKPITables.Operational_Performance);
		operationalPerformanceGr.addQuery('outcome_data_reference.objective.progress_state', '!=', 'cancelled');
		operationalPerformanceGr.addQuery('outcome_data_reference.objective.progress_state', '!=', 'on_hold');
		operationalPerformanceGr.addQuery('outcome_data_reference.objective.company', customerId);
		operationalPerformanceGr.addQuery('value', '!=', '');  //added as part of DFCT0207026
		operationalPerformanceGr.orderByDesc('start_quarter.start_date');
		operationalPerformanceGr.query();

		// Convert string to a Date object
		var yearQuarter = currentPeriodStr.split(" ");
		var quarterStartMonth = { Q1: 0, Q2: 3, Q3: 6, Q4: 9 }[yearQuarter[1]];
		var referenceDate = new Date(yearQuarter[0], quarterStartMonth, 1);

		while (operationalPerformanceGr.next() && operationalPerformanceGr.canRead()) {
			if(uniqueElements.indexOf(operationalPerformanceGr.start_quarter.title.toString()) == -1) {
				var startDateStr = operationalPerformanceGr.start_quarter.start_date.toString();
				var yearQuarterStartDate = startDateStr.split("-");
				var date = new Date(yearQuarterStartDate[0] ? yearQuarterStartDate[0] : 0, yearQuarterStartDate[1] ? yearQuarterStartDate[1]-1 : 0, yearQuarterStartDate[2] ? yearQuarterStartDate[2] : 0 );
				if(date <= referenceDate){

					timePeriods.push({
						label: operationalPerformanceGr.start_quarter.title.toString(),
						selected: false
					});
				}
			}
			uniqueElements.push(operationalPerformanceGr.start_quarter.title.toString());

		}

		return timePeriods;

	},

	getOutcomeDetailsFlyOut: function (outcomeSysID) {
		var outcome = {};

		var outcomeInstanceGR = new GlideRecordSecure('x_snc_octane_bvmp_objective_outcome_instance_data');
		if (outcomeInstanceGR.get(outcomeSysID) && outcomeInstanceGR.canRead()) {
			outcome.sysID = outcomeInstanceGR.sys_id.getDisplayValue();
			outcome.selected = true;
			outcome.title = outcomeInstanceGR.title.getDisplayValue();
			outcome.objectiveTitle = outcomeInstanceGR.objective.getDisplayValue();
			outcome.stakeholder = gs.nil(outcomeInstanceGR.objective.stakeholder_group) ? '' : outcomeInstanceGR.objective.stakeholder_group.getDisplayValue();
			outcome.custom = String(outcomeInstanceGR.user_defined).toLowerCase() === 'true';
			if (outcome.custom) {
				outcome.product = gs.nil(outcomeInstanceGR.objective.products) ? '' : outcomeInstanceGR.objective.products.getDisplayValue();
				outcome.description = outcomeInstanceGR.description.getDisplayValue();
			} else {
				outcome.valueOutcome = outcomeInstanceGR.outcome.getDisplayValue();
				outcome.product = gs.nil(outcomeInstanceGR.outcome.product) ? '' : outcomeInstanceGR.outcome.product.getDisplayValue();
				outcome.description = gs.nil(outcomeInstanceGR.outcome.description) ? '' : outcomeInstanceGR.outcome.description.getDisplayValue();
				outcome.coreCapabilities = gs.nil(outcomeInstanceGR.outcome.core_capabilities) ? '' : outcomeInstanceGR.outcome.core_capabilities.getDisplayValue().split(',');
			}
		}

		var metricInstanceGR = new GlideRecordSecure('x_snc_octane_bvmp_outcome_metric_instance_data');
		if (metricInstanceGR.get('outcome', outcomeSysID) && metricInstanceGR.canRead()) {
			if (outcome.custom) {
				outcome.metricTitle = metricInstanceGR.title.getDisplayValue();
				outcome.metricUnit = metricInstanceGR.unit.getDisplayValue();
				outcome.metricDirection = gs.nil(metricInstanceGR.getValue('metric_direction')) ?
					{
					label: '',
					value: ''
				} : {
					label: metricInstanceGR.getValue('metric_direction') == 'low' ? 'Decrease' : 'Increase',
					value: metricInstanceGR.getValue('metric_direction')
				};
			} else {
				outcome.metricTitle = metricInstanceGR.metric.getDisplayValue();
				outcome.metricBenchmarkTitle = gs.nil(metricInstanceGR.metric.benchmark_title) ? '' : metricInstanceGR.metric.benchmark_title.getDisplayValue();
				outcome.metricDescription = gs.nil(metricInstanceGR.metric.description) ? '' : metricInstanceGR.metric.description.getDisplayValue();
				outcome.canBeNormalized =  metricInstanceGR.outcome.outcome.normalizable ? true : false;
				outcome.isNormalized = metricInstanceGR.outcome.normalized ? true :false ;
				var benchmarkValue = gs.nil(metricInstanceGR.metric.benchmark_value) ? '' : metricInstanceGR.metric.benchmark_value.getDisplayValue();
				var benchmarkUnit = gs.nil(metricInstanceGR.metric.unit) ? '' : metricInstanceGR.metric.unit.getDisplayValue();
				var benchmarkDirection = gs.nil(metricInstanceGR.metric.positive_measurement_direction) ? '' : metricInstanceGR.metric.positive_measurement_direction.getDisplayValue();
				if (outcome.metricBenchmarkTitle && benchmarkValue && benchmarkUnit && benchmarkDirection) {
					benchmarkDirection = (benchmarkDirection == 'Low') ? 'reduction' : 'increase';
					outcome.metricBenchmark = benchmarkValue + '%' + ' ' + benchmarkDirection;
				}
			}
		}
		return outcome;
	},

	saveOutcomeDetailsFlyOut: function (outcome) {
		try {
			var isCustomerUser = new x_snc_octane_sp.OctaneImpactRoleUtils().isCustomerUser(gs.getUserID());
			var outcomeInstanceGR = new GlideRecordSecure('x_snc_octane_bvmp_objective_outcome_instance_data');
			if (outcomeInstanceGR.get(outcome.sysID)) {
				outcomeInstanceGR.setValue('title', outcome.title);
				if (outcome.custom && !isCustomerUser) {
					outcomeInstanceGR.setValue('description', outcome.description);
				}
				outcomeInstanceGR.update();
			}
			if (outcome.custom) {
				var metricInstanceGR = new GlideRecordSecure('x_snc_octane_bvmp_outcome_metric_instance_data');
				if (metricInstanceGR.get('outcome', outcome.sysID)) {
					metricInstanceGR.setValue('title', outcome.metricTitle);
					if (!isCustomerUser) {
						metricInstanceGR.setValue('unit', outcome.metricUnit);
						metricInstanceGR.setValue('metric_direction', outcome.metricDirection.value);
					}
					metricInstanceGR.update();
				}
			}
		} catch (error) {
			gs.addErrorMessage(gs.getMessage('impact-CJM-outcome-failed-to-be-added'));
		}
	},

	deselectOutcomeFlyOut: function (outcome) {
		try {
			var isCustomerUser = new x_snc_octane_sp.OctaneImpactRoleUtils().isCustomerUser(gs.getUserID());
			var outcomeInstanceGR = new GlideRecordSecure('x_snc_octane_bvmp_objective_outcome_instance_data');
			if (!isCustomerUser && outcomeInstanceGR.get(outcome.sysID)) {
				outcome.selected ? outcomeInstanceGR.setValue('progress', 'not_started') : outcomeInstanceGR.setValue('progress', 'deleted');
				outcomeInstanceGR.update();
			}
		} catch (error) { 
			gs.addErrorMessage(gs.getMessage('impact-CJM-outcome-instance-failed-to-be-deleted'));
		}

	},

	type: 'BusinessObjective'
};

"""

# Additional Examples

In [34]:
example_code1 ='''
"var TestAssistAjax = Class.create();
TestAssistAjax.prototype = Object.extendsObject(global.AbstractAjaxProcessor, {
            generateUnitTest: function() {
                var returnData = ;
                try {
                    var data = this.getParameter(sysparm_data);
                    if (data) {
                        data = JSON.parse(data);
                        var queue_results = this.queueTestAssistQuery(""unit_test"", data.current_table, data.current_record_id);
                        if (!gs.nil(queue_results)) {
                            returnData = JSON.stringify(queue_results);
                        }
                    }
                } catch (e) {
                    gs.error(""[TestAssistAjax] generateUnitTest Error: "" + e.toString());
                }
                return returnData;
            },
            queueTestAssistQuery: function(type, table_name, record_id) {
                    var resultObj;
                    try {
                        var scriptIncludeGr = new GlideRecord(+table_name + );
                        if (scriptIncludeGr.get(record_id + )) {
                            gs.info(""[TestAssistAjax] queueTestAssistQuery Record: "" + scriptIncludeGr.getValue(sys_id));
                            var util = new x_snc_unit_test_ge.UnitTestGenrationUtil();
                            var queryGR = util.createUnitTestRecord(scriptIncludeGr, null, null, azure_endpoint);
                            gs.info(""[TestAssistAjax] queueTestAssistQuery queryGR: "" + queryGR.getValue(sys_id));
                            if (!queryGR) {
                                return;
                            }
                            var data = {
                                source_code: queryGR.getValue(source_code),
                                user_name: gs.getUserName(),
                                type: type,
                                instance_name: gs.getProperty(instance_name),
                                source_table: table_name,
                                openai_flag: gs.getProperty(x_snc_unit_test_ge.test_assist_azure_openai_call_enabled)
                            };
                            var instance = util.getInstance();
                            var testAssistQueuePOST = new sn_ws.RESTMessageV2(""x_snc_unit_test_ge.Test Assist"", ""Queue Test Assist Query"");
                            var user_name = gs.getProperty(x_snc_unit_test_ge.test_assist_api_user_name);
                            var user_password = gs.getProperty(x_snc_unit_test_ge.test_assist_api_user_password);
                            testAssistQueuePOST.setBasicAuth(user_name, user_password);
                            testAssistQueuePOST.setStringParameter(""instance"", instance);
                            testAssistQueuePOST.setRequestBody(JSON.stringify(data));
                            var testAssistQueuePOST_response = testAssistQueuePOST.execute();
                            var testAssistQueuePOST_response_body = testAssistQueuePOST_response.getBody();
                            var testAssistQueueResult = {
                                status_code: testAssistQueuePOST_response.getStatusCode(),
                                body: JSON.parse(testAssistQueuePOST_response_body || ""{}"").result
                            };
                            progress_id = undefined; // var failed = false;                 var message = undefined;                 var status = undefined;                 if (testAssistQueueResult.status_code == 200) {                     message = testAssistQueueResult.body.message;                     status = testAssistQueueResult.body.status;                     if (testAssistQueueResult.body.status == ""failed"") {                         queryGR.setValue(""response"", testAssistQueueResult.body.message);                         queryGR.setValue(""status"", ""failed"");                         var gdt = new GlideDateTime();                         queryGR.setValue(end_time, gdt);                         // failed = true;                     } else {                         progress_id = testAssistQueueResult.body.progress_id;                         queryGR.setValue(""progress_id"", progress_id);                     }                     queryGR.update();                 } else {                     message = ""provider_instance_error"";                     status = ""failed"";                     queryGR.setValue(""response"", message);                     queryGR.setValue(""status"", status);                     var gdt = new GlideDateTime();                     queryGR.setValue(end_time, gdt);                     queryGR.update();                 }                 resultObj = {                     status_code: testAssistQueueResult.status_code,                     query_id: queryGR.getUniqueValue(),                     progress_id: progress_id,                     query_status: status,                     message: message                 };                 gs.info([TestAssistAjax]: Return Data: queueTestAssistQuery:  + JSON.stringify(resultObj));             }         } catch (e) {             gs.error(""[TestAssistAjax] queueTestAssistQuery Error: "" + e.toString());         }         return resultObj;     },     getEndpointResponse: function() {         // [explainCodeAjaxCallback] answer:  {""query_status"": ""pending"", ""status_code"":200,""query_id"":""fa8d9ea4dbc0359050be9c7ed39619fc"", ""progress_id"":""038d12a01b4cf590507f11751a4bcbd7""}         var returnData = ;         try {             var data = this.getParameter(sysparm_data);             if (data) {                 gs.info(""[TestAssistAjax] data: "" + data);                 // return returnData;                 data = JSON.parse(data);                 var progress_id = data.progress_id;                 var query_id = data.query_id;                 var query_status = data.query_status;                 var util = new x_snc_unit_test_ge.UnitTestGenrationUtil();                 var instance = util.getInstance();                 var testAssistEndpointResponseGET = new sn_ws.RESTMessageV2(""x_snc_unit_test_ge.Test Assist"", ""Get Test Assist Response"");                 var user_name = gs.getProperty(x_snc_unit_test_ge.test_assist_api_user_name);                 var user_password = gs.getProperty(x_snc_unit_test_ge.test_assist_api_user_password);                 testAssistEndpointResponseGET.setBasicAuth(user_name, user_password);                 testAssistEndpointResponseGET.setStringParameter(""instance"", instance);                 testAssistEndpointResponseGET.setStringParameter(""progress_id"", progress_id);                 var testAssistEndpointResponseGET_response = testAssistEndpointResponseGET.execute();                 var testAssistEndpointResponseGET_response_body = testAssistEndpointResponseGET_response.getBody();                 var endpointResponseResult = {                     status_code: testAssistEndpointResponseGET_response.getStatusCode(),                     body: JSON.parse(testAssistEndpointResponseGET_response_body || ""{}"").result                 };                 var progress_id = undefined;                 var resultObj = {                     status_code: endpointResponseResult.status_code,                     response: undefined,                     query_status: undefined,                     query_id: query_id,                     error_type: undefined,                     error_message: undefined                 };                 if (endpointResponseResult.status_code == 200) {                     // var result = {                     //     status: queryGR.getValue(""status""),                     //     response: undefined                     // };                     resultObj.query_status = endpointResponseResult.body.status;                     if ((resultObj.query_status == ""completed"") || (resultObj.query_status == ""failed"")) {                         resultObj.response = endpointResponseResult.body.response;                         if (resultObj.query_status == ""failed"") {                             resultObj.error_type = endpointResponseResult.body.error_type;                             resultObj.error_message = endpointResponseResult.body.error_message;                         }                     }                 } else {                     resultObj.response = ""provider_instance_error"";                     resultObj.query_status = ""failed"";                 }                 if (resultObj.query_status !== query_status) {                     var queryGR = new GlideRecord(x_snc_unit_test_ge_unit_test);                     if (queryGR.get(query_id)) {                         queryGR.setValue(""status"", resultObj.query_status);                         if (!gs.nil(resultObj.response)) {                             queryGR.setValue(""response"", resultObj.response);                             if (resultObj.query_status == ""completed"") {                                 queryGR.setValue(script, resultObj.response);                             }                         }                         if ((resultObj.query_status == ""completed"") || (resultObj.query_status == ""failed"")) {                             var gdt = new GlideDateTime();                             queryGR.setValue(end_time, gdt);                         }                         queryGR.update();                     }                 }                 gs.info(""[TestAssistAjax]: Return Data: getEndpointResponse: "" + JSON.stringify(resultObj));                 returnData = JSON.stringify(resultObj);             }         } catch (e) {             gs.error(""[TestAssistAjax] getEndpointResponse Error: "" + e.toString());         }         return returnData;     },     saveFeedback: function() {         var transactionId = this.getParameter(transactionId);         var rating = this.getParameter(rating);         var feedback = this.getParameter(feedback);         var table = this.getParameter(table);         var queryType = this.getParameter(query_type);         var query_id = this.getParameter(query_id);         gs.info(""[TestAssistAjax]: transactionId: "" + transactionId + "", rating: "" + rating + "", feedback: "" + feedback + "", table: "" + table + "", queryType: "" + queryType);         var grFdbk = new GlideRecord(x_snc_unit_test_ge_test_assist_feedback);         grFdbk.initialize();         grFdbk.setValue(user_rating, rating);         grFdbk.setValue(user_feedback, feedback);         grFdbk.setValue(user, gs.getUserID());         //grFdbk.setValue(source_table, table);         var feedbackId = grFdbk.insert();         var grQuery = new GlideRecord(x_snc_unit_test_ge_unit_test);         grQuery.addEncodedQuery(user= + gs.getUserID() + ^sys_id= + transactionId);         if (queryType == unit_test) {             grQuery.addQuery(type, unit_test);         }         grQuery.query();         if (grQuery.next()) {             grQuery.setValue(feedback, feedbackId);             grQuery.update();         }     },     updateRecordDetails: function() {         try {             gs.info(""[TestAssistAjax]: Function: updateRecordDetails"");             var query_id = this.getParameter(query_id);             var testModified = this.getParameter(testModified);             var createTest = this.getParameter(createTest);             var testScriptModified = this.getParameter(testScriptModified);             var document_id = this.getParameter(document_id);             var testCode = ;             gs.info(""[TestAssistAjax]: query_id: "" + query_id + "", testModified: "" + testModified + "", createTest: "" + createTest + "", document_id: "" + document_id + "", testScriptModified: "" + testScriptModified);             var grQuery = new GlideRecord(x_snc_unit_test_ge_unit_test);             if (grQuery.get(query_id)) {                 gs.info(""[TestAssistAjax]: Record Found: "" + grQuery.getUniqueValue());                 if (testModified == true || testModified == true) {                     grQuery.setValue(modified, true);                     grQuery.setValue(test_script_modified, testScriptModified);                     testCode = testScriptModified;                 } else {                     testCode = grQuery.getValue(script);                 }                 if (createTest == true || createTest == true) {                     var atfUtil = new x_snc_unit_test_ge.UnitTestGenerationATFUtil();                     var scriptIncludeRef = new GlideRecord(sys_script_include);                     if (scriptIncludeRef.get(document_id)) {                         var ATFTestGR = atfUtil.createATFTest(scriptIncludeRef, testCode);                         var isTestCreated;                         if (ATFTestGR) {                             isTestCreated = true;                             var link = ""/sys_atf_test.do?sys_id="" + ATFTestGR.getValue(sys_id);                             var linkMessage = ""ATF test created/updated. Please <a class=web target=_blank href = "" + link + ""> click here </a> and verify related test step(s) from Test Steps related list."";                             gs.addInfoMessage(gs.getMessage(linkMessage));                         } else {                             isTestCreated = false;                             var errorMessage = ""ATF test creation failed."";                             gs.addErrorMessage(gs.getMessage(errorMessage));                         }                         grQuery.setValue(test_created, isTestCreated);                     }                 }                 var updated = grQuery.update();                 gs.info(""[TestAssistAjax]: Record Updated: "" + updated);             }         } catch (e) {             gs.error(""[TestAssistAjax]: "" + e.toString());         }     },     getQueryRecordFieldValue: function() {         var result = false;         try {             var query_id = this.getParameter(sysparm_query_id);             var field_name = this.getParameter(sysparm_field_name);             if (query_id) {                 var queryGr = new GlideRecord(x_snc_unit_test_ge_unit_test);                 if (queryGr.get(query_id)) {                     return queryGr.getValue(+field_name+);                 }             }         } catch (e) {             gs.error(""[TestAssistAjax] getQueryRecordFieldValue Error: "" + e.toString());         }         return result;     },     type: TestAssistAjax });"
'''

In [35]:
example_code2 = '''
"var SampleSIGlobal = Class.create();
SampleSIGlobal.prototype = {
    getRecordByName: function(name) {
        var records = new GlideRecord(""sys_script_include"");
        records.addEncodedQuery(""name="" + name);
        records.query();
        if (records.next()) {
            var flag = true;
            flag = getRecordActiveStatus(records);
            if (flag == true) return records.getUniqueValue();
        }
        return false;
    },
    getRecordActiveStatus: function(record) {
        if (record.active == true) {
            return true;
        }
        return false;
    },
    type: SampleSIGlobal
};"
'''

In [38]:
example_code3 = '''
"var TestAssistAjax = Class.create();
TestAssistAjax.prototype = Object.extendsObject(global.AbstractAjaxProcessor, {
            generateUnitTest: function() {
                var returnData = ;
                try {
                    var data = this.getParameter(sysparm_data);
                    if (data) {
                        data = JSON.parse(data);
                        var queue_results = this.queueTestAssistQuery(""unit_test"", data.current_table, data.current_record_id);
                        if (!gs.nil(queue_results)) {
                            returnData = JSON.stringify(queue_results);
                        }
                    }
                } catch (e) {
                    gs.error(""[TestAssistAjax] generateUnitTest Error: "" + e.toString());
                }
                return returnData;
            },
            queueTestAssistQuery: function(type, table_name, record_id) {
                    var resultObj;
                    try {
                        var scriptIncludeGr = new GlideRecord(+table_name + );
                        if (scriptIncludeGr.get(record_id + )) {
                            gs.info(""[TestAssistAjax] queueTestAssistQuery Record: "" + scriptIncludeGr.getValue(sys_id));
                            var util = new x_snc_unit_test_ge.UnitTestGenrationUtil();
                            var queryGR = util.createUnitTestRecord(scriptIncludeGr, null, null, azure_endpoint);
                            gs.info(""[TestAssistAjax] queueTestAssistQuery queryGR: "" + queryGR.getValue(sys_id));
                            if (!queryGR) {
                                return;
                            }
                            var data = {
                                source_code: queryGR.getValue(source_code),
                                user_name: gs.getUserName(),
                                type: type,
                                instance_name: gs.getProperty(instance_name),
                                source_table: table_name,
                                openai_flag: gs.getProperty(x_snc_unit_test_ge.test_assist_azure_openai_call_enabled)
                            };
                            var instance = util.getInstance();
                            var testAssistQueuePOST = new sn_ws.RESTMessageV2(""x_snc_unit_test_ge.Test Assist"", ""Queue Test Assist Query"");
                            var user_name = gs.getProperty(x_snc_unit_test_ge.test_assist_api_user_name);
                            var user_password = gs.getProperty(x_snc_unit_test_ge.test_assist_api_user_password);
                            testAssistQueuePOST.setBasicAuth(user_name, user_password);
                            testAssistQueuePOST.setStringParameter(""instance"", instance);
                            testAssistQueuePOST.setRequestBody(JSON.stringify(data));
                            var testAssistQueuePOST_response = testAssistQueuePOST.execute();
                            var testAssistQueuePOST_response_body = testAssistQueuePOST_response.getBody();
                            var testAssistQueueResult = {
                                status_code: testAssistQueuePOST_response.getStatusCode(),
                                body: JSON.parse(testAssistQueuePOST_response_body || ""{}"").result
                            };
                            progress_id = undefined; // var failed = false;                 var message = undefined;                 var status = undefined;                 if (testAssistQueueResult.status_code == 200) {                     message = testAssistQueueResult.body.message;                     status = testAssistQueueResult.body.status;                     if (testAssistQueueResult.body.status == ""failed"") {                         queryGR.setValue(""response"", testAssistQueueResult.body.message);                         queryGR.setValue(""status"", ""failed"");                         var gdt = new GlideDateTime();                         queryGR.setValue(end_time, gdt);                         // failed = true;                     } else {                         progress_id = testAssistQueueResult.body.progress_id;                         queryGR.setValue(""progress_id"", progress_id);                     }                     queryGR.update();                 } else {                     message = ""provider_instance_error"";                     status = ""failed"";                     queryGR.setValue(""response"", message);                     queryGR.setValue(""status"", status);                     var gdt = new GlideDateTime();                     queryGR.setValue(end_time, gdt);                     queryGR.update();                 }                 resultObj = {                     status_code: testAssistQueueResult.status_code,                     query_id: queryGR.getUniqueValue(),                     progress_id: progress_id,                     query_status: status,                     message: message                 };                 gs.info([TestAssistAjax]: Return Data: queueTestAssistQuery:  + JSON.stringify(resultObj));             }         } catch (e) {             gs.error(""[TestAssistAjax] queueTestAssistQuery Error: "" + e.toString());         }         return resultObj;     },     getEndpointResponse: function() {         // [explainCodeAjaxCallback] answer:  {""query_status"": ""pending"", ""status_code"":200,""query_id"":""fa8d9ea4dbc0359050be9c7ed39619fc"", ""progress_id"":""038d12a01b4cf590507f11751a4bcbd7""}         var returnData = ;         try {             var data = this.getParameter(sysparm_data);             if (data) {                 gs.info(""[TestAssistAjax] data: "" + data);                 // return returnData;                 data = JSON.parse(data);                 var progress_id = data.progress_id;                 var query_id = data.query_id;                 var query_status = data.query_status;                 var util = new x_snc_unit_test_ge.UnitTestGenrationUtil();                 var instance = util.getInstance();                 var testAssistEndpointResponseGET = new sn_ws.RESTMessageV2(""x_snc_unit_test_ge.Test Assist"", ""Get Test Assist Response"");                 var user_name = gs.getProperty(x_snc_unit_test_ge.test_assist_api_user_name);                 var user_password = gs.getProperty(x_snc_unit_test_ge.test_assist_api_user_password);                 testAssistEndpointResponseGET.setBasicAuth(user_name, user_password);                 testAssistEndpointResponseGET.setStringParameter(""instance"", instance);                 testAssistEndpointResponseGET.setStringParameter(""progress_id"", progress_id);                 var testAssistEndpointResponseGET_response = testAssistEndpointResponseGET.execute();                 var testAssistEndpointResponseGET_response_body = testAssistEndpointResponseGET_response.getBody();                 var endpointResponseResult = {                     status_code: testAssistEndpointResponseGET_response.getStatusCode(),                     body: JSON.parse(testAssistEndpointResponseGET_response_body || ""{}"").result                 };                 var progress_id = undefined;                 var resultObj = {                     status_code: endpointResponseResult.status_code,                     response: undefined,                     query_status: undefined,                     query_id: query_id,                     error_type: undefined,                     error_message: undefined                 };                 if (endpointResponseResult.status_code == 200) {                     // var result = {                     //     status: queryGR.getValue(""status""),                     //     response: undefined                     // };                     resultObj.query_status = endpointResponseResult.body.status;                     if ((resultObj.query_status == ""completed"") || (resultObj.query_status == ""failed"")) {                         resultObj.response = endpointResponseResult.body.response;                         if (resultObj.query_status == ""failed"") {                             resultObj.error_type = endpointResponseResult.body.error_type;                             resultObj.error_message = endpointResponseResult.body.error_message;                         }                     }                 } else {                     resultObj.response = ""provider_instance_error"";                     resultObj.query_status = ""failed"";                 }                 if (resultObj.query_status !== query_status) {                     var queryGR = new GlideRecord(x_snc_unit_test_ge_unit_test);                     if (queryGR.get(query_id)) {                         queryGR.setValue(""status"", resultObj.query_status);                         if (!gs.nil(resultObj.response)) {                             queryGR.setValue(""response"", resultObj.response);                             if (resultObj.query_status == ""completed"") {                                 queryGR.setValue(script, resultObj.response);                             }                         }                         if ((resultObj.query_status == ""completed"") || (resultObj.query_status == ""failed"")) {                             var gdt = new GlideDateTime();                             queryGR.setValue(end_time, gdt);                         }                         queryGR.update();                     }                 }                 gs.info(""[TestAssistAjax]: Return Data: getEndpointResponse: "" + JSON.stringify(resultObj));                 returnData = JSON.stringify(resultObj);             }         } catch (e) {             gs.error(""[TestAssistAjax] getEndpointResponse Error: "" + e.toString());         }         return returnData;     },     saveFeedback: function() {         var transactionId = this.getParameter(transactionId);         var rating = this.getParameter(rating);         var feedback = this.getParameter(feedback);         var table = this.getParameter(table);         var queryType = this.getParameter(query_type);         var query_id = this.getParameter(query_id);         gs.info(""[TestAssistAjax]: transactionId: "" + transactionId + "", rating: "" + rating + "", feedback: "" + feedback + "", table: "" + table + "", queryType: "" + queryType);         var grFdbk = new GlideRecord(x_snc_unit_test_ge_test_assist_feedback);         grFdbk.initialize();         grFdbk.setValue(user_rating, rating);         grFdbk.setValue(user_feedback, feedback);         grFdbk.setValue(user, gs.getUserID());         //grFdbk.setValue(source_table, table);         var feedbackId = grFdbk.insert();         var grQuery = new GlideRecord(x_snc_unit_test_ge_unit_test);         grQuery.addEncodedQuery(user= + gs.getUserID() + ^sys_id= + transactionId);         if (queryType == unit_test) {             grQuery.addQuery(type, unit_test);         }         grQuery.query();         if (grQuery.next()) {             grQuery.setValue(feedback, feedbackId);             grQuery.update();         }     },     updateRecordDetails: function() {         try {             gs.info(""[TestAssistAjax]: Function: updateRecordDetails"");             var query_id = this.getParameter(query_id);             var testModified = this.getParameter(testModified);             var createTest = this.getParameter(createTest);             var testScriptModified = this.getParameter(testScriptModified);             var document_id = this.getParameter(document_id);             var testCode = ;             gs.info(""[TestAssistAjax]: query_id: "" + query_id + "", testModified: "" + testModified + "", createTest: "" + createTest + "", document_id: "" + document_id + "", testScriptModified: "" + testScriptModified);             var grQuery = new GlideRecord(x_snc_unit_test_ge_unit_test);             if (grQuery.get(query_id)) {                 gs.info(""[TestAssistAjax]: Record Found: "" + grQuery.getUniqueValue());                 if (testModified == true || testModified == true) {                     grQuery.setValue(modified, true);                     grQuery.setValue(test_script_modified, testScriptModified);                     testCode = testScriptModified;                 } else {                     testCode = grQuery.getValue(script);                 }                 if (createTest == true || createTest == true) {                     var atfUtil = new x_snc_unit_test_ge.UnitTestGenerationATFUtil();                     var scriptIncludeRef = new GlideRecord(sys_script_include);                     if (scriptIncludeRef.get(document_id)) {                         var ATFTestGR = atfUtil.createATFTest(scriptIncludeRef, testCode);                         var isTestCreated;                         if (ATFTestGR) {                             isTestCreated = true;                             var link = ""/sys_atf_test.do?sys_id="" + ATFTestGR.getValue(sys_id);                             var linkMessage = ""ATF test created/updated. Please <a class=web target=_blank href = "" + link + ""> click here </a> and verify related test step(s) from Test Steps related list."";                             gs.addInfoMessage(gs.getMessage(linkMessage));                         } else {                             isTestCreated = false;                             var errorMessage = ""ATF test creation failed."";                             gs.addErrorMessage(gs.getMessage(errorMessage));                         }                         grQuery.setValue(test_created, isTestCreated);                     }                 }                 var updated = grQuery.update();                 gs.info(""[TestAssistAjax]: Record Updated: "" + updated);             }         } catch (e) {             gs.error(""[TestAssistAjax]: "" + e.toString());         }     },     getQueryRecordFieldValue: function() {         var result = false;         try {             var query_id = this.getParameter(sysparm_query_id);             var field_name = this.getParameter(sysparm_field_name);             if (query_id) {                 var queryGr = new GlideRecord(x_snc_unit_test_ge_unit_test);                 if (queryGr.get(query_id)) {                     return queryGr.getValue(+field_name+);                 }             }         } catch (e) {             gs.error(""[TestAssistAjax] getQueryRecordFieldValue Error: "" + e.toString());         }         return result;     },     type: TestAssistAjax });"
'''

In [36]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(example_code1)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

input prompt size: 668
<class 'dict'>
(function() {
    describe('TestAssistAjax', function() {
        var testAssistAjax;
        beforeEach(function() {
            testAssistAjax = new TestAssistAjax();
        });
        it('should return empty string if no data is provided', function() {
            var result = testAssistAjax.generateUnitTest();
            expect(result).toEqual('');
        });
        it('should return empty string if queue_results is null', function() {
            var data = {
                current_table: 'table_name',
                current_record_id: 'record_id'
            };
            spyOn(testAssistAjax, 'queueTestAssistQuery').and.returnValue(null);
            testAssistAjax.getParameter = jasmine.createSpy('getParameter').and.returnValue(JSON.stringify(data));
            var result = testAssistAjax.generateUnitTest();
            expect(result).toEqual('');
        });
        it('should return queue_results if queue_results is not null', fu

In [37]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(example_code2)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

input prompt size: 235
<class 'dict'>
(function(outputs, steps, params, stepResult, assertEqual) {
    
    describe('Validate SampleSIGlobal SI functions', function() {
        it('test getRecordByName function with valid name', function() {
            var sampleSIGlobal = new SampleSIGlobal();
            var record = sampleSIGlobal.getRecordByName('test_script_include');
            expect(record).not.toBe(false);
        });
        it('test getRecordByName function with invalid name', function() {
            var sampleSIGlobal = new SampleSIGlobal();
            var record = sampleSIGlobal.getRecordByName('invalid_script_include');
            expect(record).toBe(false);
        });
        it('test getRecordActiveStatus function with active record', function() {
            var sampleSIGlobal = new SampleSIGlobal();
            var record = new GlideRecord('sys_script_include');
            record.get('test_script_include');
            var activeStatus = sampleSIGlobal.getReco

In [39]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

#small_code,code,large_code
model =Inference(faiss_model_azure)
code = model.code_clean(example_code3)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")

if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

input prompt size: 668
<class 'dict'>
(function() {
    describe('TestAssistAjax', function() {
        var testAssistAjax;
        beforeEach(function() {
            testAssistAjax = new TestAssistAjax();
        });
        it('should return empty string if no data is provided', function() {
            var result = testAssistAjax.generateUnitTest();
            expect(result).toEqual('');
        });
        it('should return empty string if queue_results is null', function() {
            var data = {
                current_table: 'table_name',
                current_record_id: 'record_id'
            };
            spyOn(testAssistAjax, 'queueTestAssistQuery').and.returnValue(null);
            testAssistAjax.getParameter = jasmine.createSpy('getParameter').and.returnValue(JSON.stringify(data));
            var result = testAssistAjax.generateUnitTest();
            expect(result).toEqual('');
        });
        it('should return queue_results if queue_results is not null', fu

In [27]:
start = time.time()
time.sleep(10)
end = time.time()
(end - start)

10.010379076004028